In [ ]:
### Imports & static constants setup - this section shouldn't need changed much at all (for SRS, options will be different on other reports)...
import os 
from datetime import datetime as dt 
from time import sleep
import json 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

### Set some static lists to work with when choosing parameters 
### for reference / future - parameter options: 
report_by_options = ['County','Agency','Tag']
report_type_options = [
    'Law Enforcement Officers Killed or Assaulted',
    'Return A',
    'Supplemental Homicide',
    'Return of Arson Offenses',
    'Supplement Return A',
    'Human Trafficking',
    'ASRE Adult Arrests',
    'ASRE Adult Victims',
    'ASRE Juvenile Arrests',
    'ASRE Juvenile Victims',
    'Person Charged',
]
county_options = [
    '001 - Adams County',
    '002 - Allegheny County',
    '003 - Armstrong County',
    '004 - Beaver County',
    '005 - Bedford County',
    '006 - Berks County',
    '007 - Blair County',
    '008 - Bradford County',
    '009 - Bucks County',
    '010 - Butler County',
    '011 - Cambria County',
    '012 - Cameron County',
    '013 - Carbon County',
    '014 - Centre County',
    '015 - Chester County',
    '016 - Clarion County',
    '017 - Clearfield County',
    '018 - Clinton County',
    '019 - Columbia County',
    '020 - Crawford County',
    '021 - Cumberland County',
    '022 - Dauphin County',
    '023 - Delaware County',
    '024 - Elk County',
    '025 - Erie County',
    '026 - Fayette County',
    '027 - Forest County',
    '028 - Franklin County',
    '029 - Fulton County',
    '030 - Greene County',
    '031 - Huntingdon County',
    '032 - Indiana County',
    '033 - Jefferson County',
    '034 - Juniata County',
    '035 - Lackawanna County',
    '036 - Lancaster County',
    '037 - Lawrence County',
    '038 - Lebanon County',
    '039 - Lehigh County',
    '040 - Luzerne County',
    '041 - Lycoming County',
    '042 - McKean County',
    '043 - Mercer County',
    '044 - Mifflin County',
    '045 - Monroe County',
    '046 - Montgomery County',
    '047 - Montour County',
    '048 - Northampton County',
    '049 - Northumberland County',
    '050 - Perry County',
    '051 - Pike County',
    '052 - Potter County',
    '053 - Schuylkill County',
    '054 - Snyder County',
    '055 - Somerset County',
    '056 - Sullivan County',
    '057 - Susquehanna County',
    '058 - Tioga County',
    '059 - Union County',
    '060 - Venango County',
    '061 - Warren County',
    '062 - Washington County',
    '063 - Wayne County',
    '064 - Westmoreland County',
    '065 - Wyoming County',
    '066 - York County',
    '067 - Philadelphia County',
]
agency_options = [
    "PA0010000 - ADAMS COUNTY SHERIFFS",
    "PA0010100 - GETTYSBURG BOROUGH",
    "PA0010200 - LITTLESTOWN BOROUGH",
    "PA0010300 - CONEWAGO TOWNSHIP (ADAMS COUNTY)",
    "PA0010400 - MCSHERRYSTOWN BOROUGH",
    "PA0010500 - CUMBERLAND TOWNSHIP (ADAMS COUNTY)",
    "PA0010600 - HAMILTONBAN TOWNSHIP (COVERED BY PSP - TROOP H - GETTYSBURG STATION - PAPSP3100)",
    "PA0010700 - NEW OXFORD BOROUGH (COVERED BY EASTERN ADAMS REGIONAL - PA0011800)",
    "PA0010800 - EAST BERLIN BOROUGH",
    "PA0011000 - (INACTIVE) ADAMS COUNTY DETECTIVES",
    "PA0011100 - (INACTIVE) BERWICK TOWNSHIP (COVERED BY PSP - TROOP H - GETTYSBURG STATION - PAPSP3100)",
    "PA0011200 - LIBERTY TOWNSHIP (ADAMS COUNTY)",
    "PA0011300 - READING TOWNSHIP (ADAMS COUNTY)",
    "PA0011500 - BIGLERVILLE BOROUGH",
    "PA0011600 - BONNEAUVILLE BOROUGH",
    "PA0011800 - EASTERN ADAMS REGIONAL",
    "PA0012000 - FREEDOM TOWNSHIP (ADAMS COUNTY) (COVERED BY LIBERTY TOWNSHIP - PA0011200)",
    "PA0012100 - BENDERSVILLE BOROUGH",
    "PA0012200 - FAIRFIELD BOROUGH (COVERED BY CARROLL VALLEY BOROUGH - PA0012300)",
    "PA0012300 - CARROLL VALLEY BOROUGH",
    "PA0012400 - LATIMORE TOWNSHIP",
    "PA0012500 - ABBOTTSTOWN BOROUGH",
    "PA001BF0X - DCNR - BUREAU OF FORESTRY (ADAMS COUNTY) ",
    "PA001DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (ADAMS COUNTY)",
    "PA0020000 - ALLEGHENY COUNTY SHERIFFS",
    "PA0020100 - BALDWIN BOROUGH",
    "PA0020200 - BELLEVUE BOROUGH",
    "PA0020300 - BETHEL PARK BOROUGH",
    "PA0020400 - BRENTWOOD BOROUGH",
    "PA0020500 - CARNEGIE BOROUGH",
    "PA0020600 - CLAIRTON CITY",
    "PA0020700 - CORAOPOLIS BOROUGH",
    "PA0020800 - DORMONT BOROUGH",
    "PA0020900 - DUQUESNE CITY",
    "PA0020A00 - BALDWIN TOWNSHIP",
    "PA0020B00 - COLLIER TOWNSHIP",
    "PA0020C00 - (INACTIVE) EAST DEER TOWNSHIP (COVERED BY ALLEGHENY COUNTY REGIONAL - PA0021U00)",
    "PA0020D00 - ELIZABETH TOWNSHIP",
    "PA0020E00 - HARRISON TOWNSHIP",
    "PA0020F00 - KENNEDY TOWNSHIP",
    "PA0020G00 - RESERVE TOWNSHIP",
    "PA0020J00 - SOUTH FAYETTE TOWNSHIP",
    "PA0020K00 - WHITE OAK BOROUGH",
    "PA0020L00 - WILKINS TOWNSHIP",
    "PA0020M00 - WILMERDING BOROUGH (COVERED BY PITCAIRN PA0028300)",
    "PA0020N00 - (INACTIVE) ALEPPO TOWNSHIP (COVERED BY OHIO TOWNSHIP - PA0021A00)",
    "PA0020P00 - FAWN TOWNSHIP",
    "PA0020R00 - FINDLAY TOWNSHIP",
    "PA0020S00 - (INACTIVE) FORWARD TOWNSHIP (COVERED BY ELIZABETH TOWNSHIP - PA0020D00)",
    "PA0020T00 - HARMAR TOWNSHIP",
    "PA0020V00 - INDIANA TOWNSHIP",
    "PA0020W00 - KILBUCK TOWNSHIP (COVERED BY OHIO TOWNSHIP - PA0021A00)",
    "PA0020X00 - NEVILLE TOWNSHIP (COVERED BY OHIO TOWNSHIP - PA0021A00)",
    "PA0020Y00 - NORTH FAYETTE TOWNSHIP",
    "PA0020Z00 - OHARA TOWNSHIP",
    "PA0021000 - FOREST HILLS BOROUGH",
    "PA0021100 - GREEN TREE BOROUGH",
    "PA0021200 - HOMESTEAD BOROUGH",
    "PA0021300 - MCKEESPORT CITY",
    "PA0021400 - MCKEES ROCKS BOROUGH",
    "PA0021500 - MILLVALE BOROUGH",
    "PA0021600 - MONROEVILLE BOROUGH",
    "PA0021700 - MOUNT LEBANON TOWNSHIP",
    "PA0021800 - OAKMONT BOROUGH",
    "PA0021900 - PENN HILLS TOWNSHIP",
    "PA0021A00 - OHIO TOWNSHIP",
    "PA0021B00 - NORTHERN REGIONAL (ALLEGHENY COUNTY)",
    "PA0021C00 - RICHLAND TOWNSHIP (ALLEGHENY COUNTY) (COVERED BY NORTHERN REGIONAL - PA0021B00)",
    "PA0021D00 - WEST DEER TOWNSHIP",
    "PA0021E00 - SOUTH PARK TOWNSHIP",
    "PA0021H00 - CRESCENT TOWNSHIP",
    "PA0021I00 - LEET TOWNSHIP",
    "PA0021J00 - ALLEGHENY COUNTY HOUSING AUTHORITY",
    "PA0021N00 - UNIVERSITY OF PITTSBURGH - MAIN CAMPUS",
    "PA0021P00 - PSU - GREATER ALLEGHENY CAMPUS",
    "PA0021R00 - MARSHALL TOWNSHIP (COVERED BY NORTHERN REGIONAL - PA0021B00)",
    "PA0021S00 - SPRINGDALE TOWNSHIP (COVERED BY ALLEGHENY VALLEY REGIONAL - PA0021U00)",
    "PA0021U00 - ALLEGHENY VALLEY REGIONAL",
    "PA0022100 - PLEASANT HILLS BOROUGH",
    "PA0022200 - ROSS TOWNSHIP",
    "PA0022300 - SEWICKLEY BOROUGH",
    "PA0022400 - SEWICKLEY HEIGHTS BOROUGH",
    "PA0022500 - UPPER ST. CLAIR TOWNSHIP",
    "PA0022600 - WHITEHALL BOROUGH",
    "PA0022700 - WILKINSBURG BOROUGH",
    "PA0022800 - ALLEGHENY COUNTY POLICE",
    "PA0022900 - AVALON BOROUGH",
    "PA0022A00 - FRAZER TOWNSHIP",
    "PA0022D00 - ALLEGHENY COUNTY PORT AUTHORITY",
    "PA0022F9E - CARNEGIE-MELLON UNIVERSITY",
    "PA0022H9E - CHATHAM UNIVERSITY",
    "PA0023000 - (INACTIVE) BRADDOCK BOROUGH (COVERED BY EASTERN REGIONAL MON VALLEY - PA002B800)",
    "PA0023100 - EDGEWOOD BOROUGH",
    "PA0023200 - GLASSPORT BOROUGH",
    "PA0023300 - HAMPTON TOWNSHIP",
    "PA0023400 - MCCANDLESS TOWNSHIP",
    "PA0023500 - MOON TOWNSHIP",
    "PA0023600 - MUNHALL BOROUGH",
    "PA0023800 - (INACTIVE) NORTH BRADDOCK BOROUGH (COVERED BY EASTERN REGIONAL MON VALLEY PA002B800)",
    "PA0023900 - NORTH VERSAILLES TOWNSHIP",
    "PA0024000 - ROBINSON TOWNSHIP (ALLEGHENY COUNTY)",
    "PA0024100 - SCOTT TOWNSHIP (ALLEGHENY COUNTY)",
    "PA0024200 - SHALER TOWNSHIP",
    "PA0024300 - STOWE TOWNSHIP",
    "PA0024400 - SWISSVALE BOROUGH",
    "PA0024500 - TURTLE CREEK BOROUGH",
    "PA0024600 - WEST HOMESTEAD BOROUGH",
    "PA0024700 - WEST MIFFLIN BOROUGH",
    "PA0024800 - ASPINWALL BOROUGH",
    "PA0024900 - BELL ACRES BOROUGH",
    "PA0025000 - (INACTIVE) BEN AVON BOROUGH (COVERED BY AVALON BOROUGH - PA0022900 )",
    "PA0025100 - (INACTIVE) BEN AVON HEIGHTS (COVERED BY OHIO TOWNSHIP - PA0021A00)",
    "PA00251VS - (INACTIVE) HILLEL ACADEMY POLICE DEPARTMENT ",
    "PA0025200 - (INACTIVE) BLAWNOX BOROUGH (COVERED BY OHARA TOWNSHIP - PA0020Z00)",
    "PA0025300 - (INACTIVE) BRACKENRIDGE BOROUGH (COVERED BY TARENTUM BOROUGH - PA0029200)",
    "PA0025400 - BRADDOCK HILLS BOROUGH",
    "PA0025500 - (INACTIVE) BRADFORD WOODS BOROUGH (COVERED BY NORTHERN REGIONAL - PA0021B00)",
    "PA0025600 - BRIDGEVILLE BOROUGH",
    "PA0025700 - CASTLE SHANNON BOROUGH",
    "PA0025900 - (INACTIVE) CHESWICK BOROUGH (COVERED BY ALLEGHENY VALLEY REGIONAL - PA0021U00)",
    "PA0026000 - CHURCHILL BOROUGH",
    "PA0026100 - CRAFTON BOROUGH",
    "PA0026200 - DRAVOSBURG BOROUGH (COVERED BY MCKEESPORT CITY - PA0021300)",
    "PA0026300 - EAST MCKEESPORT BOROUGH",
    "PA0026400 - EAST PITTSBURGH BOROUGH (COVERED BY EASTERN REGIONAL MON VALLEY - PA002B800)",
    "PA0026500 - EDGEWORTH BOROUGH",
    "PA0026600 - (INACTIVE) ELIZABETH BOROUGH (COVERED BY ELIZABETH TOWNSHIP - PA0020D00)",
    "PA0026700 - EMSWORTH BOROUGH (COVERED BY OHIO TOWNSHIP - PA0021A00)",
    "PA0026800 - ETNA BOROUGH",
    "PA0026900 - FOX CHAPEL BOROUGH",
    "PA0027000 - FRANKLIN PARK BOROUGH",
    "PA0027300 - HEIDELBERG BOROUGH",
    "PA0027400 - INGRAM BOROUGH",
    "PA0027500 - JEFFERSON HILLS BOROUGH",
    "PA0027600 - LEETSDALE BOROUGH",
    "PA0027700 - LIBERTY BOROUGH",
    "PA0027800 - LINCOLN BOROUGH",
    "PA0027900 - MOUNT OLIVER BOROUGH",
    "PA0028000 - OAKDALE BOROUGH",
    "PA0028200 - GLEN OSBORNE BOROUGH (COVERED BY SEWICKLEY BOROUGH - PA0022300)",
    "PA0028300 - PITCAIRN BOROUGH",
    "PA0028500 - PLUM BOROUGH",
    "PA0028600 - PORT VUE BOROUGH",
    "PA0028700 - (INACTIVE) RANKIN BOROUGH (COVERED BY EASTERN REGIONAL MON VALLEY PA002B800)",
    "PA0028800 - ROSSLYN FARMS BOROUGH (COVERED BY SCOTT TOWNSHIP - PA0024100)",
    "PA0029000 - SHARPSBURG BOROUGH",
    "PA0029100 - SPRINGDALE BOROUGH",
    "PA0029200 - TARENTUM BOROUGH",
    "PA0029400 - VERONA BOROUGH",
    "PA0029500 - VERSAILLES BOROUGH",
    "PA0029700 - WEST ELIZABETH BOROUGH (COVERED BY ELIZABETH TOWNSHIP - PA0020D00)",
    "PA0029800 - WEST VIEW BOROUGH",
    "PA0029900 - WHITAKER BOROUGH",
    "PA0029A9E - DUQUESNE UNIVERSITY",
    "PA0029D9E - CARLOW UNIVERSITY",
    "PA0029K9E - POINT PARK UNIVERSITY",
    "PA002A500 - ALLEGHENY COUNTY COMMUNITY COLLEGE",
    "PA002A600 - DCNR - POINT STATE PARK",
    "PA002B800 - EASTERN REGIONAL MON VALLEY",
    "PA002BF0X - DCNR - BUREAU OF FORESTRY (ALLEGHENY COUNTY)",
    "PA002DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (ALLEGHENY COUNTY)",
    "PA0030100 - APOLLO BOROUGH",
    "PA0030200 - (INACTIVE) FORD CITY BOROUGH (COVERED BY SOUTHERN ARMSTRONG REGIONAL - PA0033100)",
    "PA0030300 - (INACTIVE) FREEPORT BOROUGH (COVERED BY SOUTHERN ARMSTRONG REGIONAL - PA0033100)",
    "PA0030400 - KITTANNING BOROUGH",
    "PA0030500 - LEECHBURG BOROUGH",
    "PA0030800 - KISKIMINETAS TOWNSHIP",
    "PA0030900 - NORTH APOLLO BOROUGH (COVERED BY KISKIMINETAS TOWNSHIP - PA0030800)",
    "PA0031000 - PARKS TOWNSHIP",
    "PA0031100 - RURAL VALLEY BOROUGH",
    "PA0031200 - SOUTH BUFFALO TOWNSHIP",
    "PA0031300 - (INACTIVE) GILPIN TOWNSHIP (COVERED BY SOUTHERN ARMSTRONG REGIONAL - PA0033100)",
    "PA0031400 - MANOR TOWNSHIP (ARMSTRONG COUNTY)",
    "PA0031500 - (INACTIVE) NORTH BUFFALO TOWNSHIP (COVERED BY PSP - TROOP D - KITTANNING STATION - PAPSP4600)",
    "PA0031800 - COWANSHANNOCK TOWNSHIP (COVERED BY PSP - TROOP D - KITTANNING STATION - PAPSP4600)",
    "PA0032000 - WEST KITTANNING BOROUGH",
    "PA0032100 - (INACTIVE) BETHEL TOWNSHIP (ARMSTRONG COUNTY) (COVERED BY MANOR TOWNSHIP - PA0031400)",
    "PA0032200 - EAST FRANKLIN TOWNSHIP",
    "PA0032300 - PARKER CITY",
    "PA0032500 - WORTHINGTON BOROUGH",
    "PA0032800 - (INACTIVE) ELDERTON BOROUGH (COVERED BY PSP - TROOP D - KITTANNING STATION - PAPSP4600)",
    "PA0033100 - SOUTHERN ARMSTRONG REGIONAL POLICE DEPARTMENT",
    "PA003BF0X - DCNR - BUREAU OF FORESTRY (ARMSTRONG COUNTY)",
    "PA003DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (ARMSTRONG COUNTY)",
    "PA0040000 - BEAVER COUNTY SHERIFFS",
    "PA0040100 - ALIQUIPPA CITY",
    "PA0040200 - AMBRIDGE BOROUGH",
    "PA0040300 - BEAVER COUNTY DETECTIVES",
    "PA0040400 - BEAVER BOROUGH",
    "PA0040500 - BEAVER FALLS CITY",
    "PA0040600 - MIDLAND BOROUGH",
    "PA0040700 - ROCHESTER BOROUGH",
    "PA0040800 - (INACTIVE) BADEN BOROUGH (COVERED BY BEAVER VALLEY REGIONAL - PA0046400)",
    "PA0040900 - BRIGHTON TOWNSHIP",
    "PA0041100 - ECONOMY BOROUGH",
    "PA0041200 - (INACTIVE) FREEDOM BOROUGH (COVERED BY BEAVER VALLEY REGIONAL - PA0046400)",
    "PA0041300 - HARMONY TOWNSHIP",
    "PA0041400 - HOPEWELL TOWNSHIP",
    "PA0041500 - MONACA BOROUGH",
    "PA0041600 - NEW BRIGHTON BOROUGH",
    "PA0041700 - NORTH SEWICKLEY TOWNSHIP",
    "PA0041800 - CENTER TOWNSHIP",
    "PA0041900 - CHIPPEWA TOWNSHIP",
    "PA0042000 - (INACTIVE) CONWAY BOROUGH (COVERED BY BEAVER VALLEY REGIONAL - PA0046400)",
    "PA0042300 - INDEPENDENCE TOWNSHIP",
    "PA0042400 - INDUSTRY BOROUGH (COVERED BY BEAVER BOROUGH - PA0040400)",
    "PA0042500 - PATTERSON TOWNSHIP",
    "PA0042800 - RACCOON TOWNSHIP",
    "PA0042900 - (INACTIVE) ROCHESTER TOWNSHIP (COVERED BY BEAVER VALLEY REGIONAL - PA0046400)",
    "PA0043100 - WHITE TOWNSHIP (COVERED BY PATTERSON TOWNSHIP - PA0042500)",
    "PA0043200 - BRIDGEWATER BOROUGH",
    "PA0043300 - KOPPEL BOROUGH",
    "PA0043400 - EAST ROCHESTER BOROUGH (COVERED BY ROCHESTER BOROUGH - PA0040700)",
    "PA0043500 - VANPORT TOWNSHIP (COVERED BY BEAVER BOROUGH - PA0040400)",
    "PA0043600 - FRANKLIN TOWNSHIP (BEAVER COUNTY)",
    "PA0043800 - NEW SEWICKLEY TOWNSHIP",
    "PA0043900 - SOUTH BEAVER TOWNSHIP",
    "PA0044100 - POTTER TOWNSHIP (COVERED BY CENTER TOWNSHIP - PA0041800)",
    "PA0044200 - (INACTIVE) BIG BEAVER BOROUGH (COVERED BY BEAVER FALLS CITY - PA0040500)",
    "PA0044400 - MARION TOWNSHIP (BEAVER COUNTY)",
    "PA0044900 - SOUTH HEIGHTS BOROUGH",
    "PA0045000 - WEST MAYFIELD BOROUGH (COVERED BY BEAVER FALLS CITY - PA0040500)",
    "PA0045100 - SHIPPINGPORT BOROUGH",
    "PA0045200 - OHIOVILLE BOROUGH",
    "PA0045300 - PSU - BEAVER CAMPUS",
    "PA0045400 - DARLINGTON TOWNSHIP",
    "PA0045600 - DCNR - RACCOON CREEK STATE PARK",
    "PA0046400 - BEAVER VALLEY REGIONAL",
    "PA004BF0X - DCNR - BUREAU OF FORESTRY (BEAVER COUNTY) ",
    "PA004DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (BEAVER COUNTY)",
    "PA0050000 - BEDFORD COUNTY SHERIFFS",
    "PA0050100 - BEDFORD BOROUGH",
    "PA0050200 - EVERETT BOROUGH",
    "PA0050400 - SAXTON BOROUGH",
    "PA0050600 - LIBERTY TOWNSHIP (BEDFORD COUNTY) (COVERED BY SAXTON BOROUGH - PA0050400)",
    "PA0050700 - HOPEWELL BOROUGH (COVERED BY PSP - TROOP G - BEDFORD STATION - PAPSP0400)",
    "PA0050900 - DCNR - BLUE KNOB STATE PARK",
    "PA0051000 - DCNR - SHAWNEE STATE PARK",
    "PA005BF0X - DCNR - BUREAU OF FORESTRY (BEDFORD COUNTY) ",
    "PA005DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (BEDFORD COUNTY)",
    "PA0060000 - BERKS COUNTY SHERIFFS",
    "PA0060100 - BIRDSBORO BOROUGH",
    "PA0060200 - (INACTIVE) BOYERTOWN BOROUGH (COVERED BY EASTERN BERKS REGIONAL - PA0067100)",
    "PA0060300 - CUMRU TOWNSHIP",
    "PA0060400 - EXETER TOWNSHIP (BERKS COUNTY)",
    "PA0060500 - FLEETWOOD BOROUGH",
    "PA0060600 - HAMBURG BOROUGH",
    "PA0060700 - KENHORST BOROUGH (COVERED BY READING CITY - PA0061400)",
    "PA0060800 - KUTZTOWN BOROUGH",
    "PA0060900 - LAURELDALE BOROUGH",
    "PA0061000 - LEESPORT BOROUGH (COVERED BY NORTHERN BERKS REGIONAL - PA0062800)",
    "PA0061100 - MOHNTON BOROUGH",
    "PA0061200 - MOUNT PENN BOROUGH (COVERED BY CENTRAL BERKS REGIONAL - PA0063500)",
    "PA0061300 - MUHLENBERG TOWNSHIP",
    "PA0061400 - READING CITY",
    "PA0061500 - ROBESONIA BOROUGH (COVERED BY SOUTH HEIDELBERG TOWNSHIP - PA0064700)",
    "PA0061600 - SHILLINGTON BOROUGH",
    "PA0061700 - SINKING SPRING BOROUGH",
    "PA0061800 - SPRING TOWNSHIP (BERKS COUNTY)",
    "PA0061900 - WESTERN BERKS REGIONAL (COVERED BY SOUTH HEIDELBERG TOWNSHIP - PA0064700)",
    "PA0062100 - WEST READING BOROUGH",
    "PA0062200 - WYOMISSING BOROUGH",
    "PA0062400 - CAERNARVON TOWNSHIP",
    "PA0062500 - (INACTIVE) COLEBROOKDALE TOWNSHIP (COVERED BY EASTERN BERKS REGIONAL - PA0067100)",
    "PA0062800 - NORTHERN BERKS REGIONAL",
    "PA0062900 - ROBESON TOWNSHIP",
    "PA0063000 - BALLY BOROUGH",
    "PA0063100 - BERN TOWNSHIP",
    "PA0063200 - BERNVILLE BOROUGH",
    "PA0063300 - (INACTIVE) CENTRE TOWNSHIP (COVERED BY PSP - TROOP L - HAMBURG STATION - PAPSP3500)",
    "PA0063400 - DOUGLASS TOWNSHIP (BERKS COUNTY)",
    "PA0063500 - CENTRAL BERKS REGIONAL",
    "PA0063600 - LOWER HEIDELBERG TOWNSHIP",
    "PA0063700 - OLEY TOWNSHIP (COVERED BY CENTRAL BERKS REGIONAL - PA0063500)",
    "PA0063900 - SHOEMAKERSVILLE BOROUGH (COVERED BY PSP - TROOP L - HAMBURG STATION - PAPSP3500)",
    "PA0064100 - (INACTIVE) BERKS-LEHIGH REGIONAL (COVERED BY PSP - TROOP L - READING STATION - PAPSP7200)",
    "PA0064200 - BETHEL TOWNSHIP (BERKS COUNTY)",
    "PA0064300 - BRECKNOCK TOWNSHIP (BERKS COUNTY)",
    "PA0064400 - (INACTIVE) HEIDELBERG TOWNSHIP (BERKS COUNTY) (COVERED BY SOUTH HEIDELBERG TOWNSHIP - PA0064700)",
    "PA0064700 - SOUTH HEIDELBERG TOWNSHIP",
    "PA0064900 - BERKS COUNTY DETECTIVES (PA006013A)",
    "PA0065000 - AMITY TOWNSHIP",
    "PA0065200 - WOMELSDORF BOROUGH",
    "PA0065400 - MARION TOWNSHIP (BERKS COUNTY) (COVERED BY TULPEHOCKEN TOWNSHIP - PA0066000)",
    "PA0065700 - KUTZTOWN UNIVERSITY",
    "PA0065900 - PSU - BERKS CAMPUS",
    "PA0066000 - TULPEHOCKEN TOWNSHIP",
    "PA0066200 - TILDEN TOWNSHIP",
    "PA0066500 - DCNR - FRENCH CREEK STATE PARK",
    "PA006679E - (INACTIVE) ALBRIGHT UNIVERSITY",
    "PA0067100 - EASTERN BERKS REGIONAL",
    "PA006BF0X - DCNR - BUREAU OF FORESTRY (BERKS COUNTY) ",
    "PA006DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (BERKS COUNTY)",
    "PA0070000 - BLAIR COUNTY SHERIFFS",
    "PA0070100 - ALTOONA CITY",
    "PA0070200 - HOLLIDAYSBURG BOROUGH",
    "PA0070300 - LOGAN TOWNSHIP",
    "PA0070400 - TYRONE BOROUGH",
    "PA0070500 - BELLWOOD BOROUGH",
    "PA0070600 - ROARING SPRING BOROUGH",
    "PA0070700 - ALLEGHENY TOWNSHIP (BLAIR COUNTY)",
    "PA0070800 - DUNCANSVILLE BOROUGH",
    "PA0070900 - MARTINSBURG BOROUGH",
    "PA0071000 - WILLIAMSBURG BOROUGH",
    "PA0071100 - GREENFIELD TOWNSHIP (BLAIR COUNTY)",
    "PA0071400 - PSU - ALTOONA CAMPUS",
    "PA0071500 - BLAIR TOWNSHIP",
    "PA0071700 - NORTH WOODBURY TOWNSHIP",
    "PA0071800 - FREEDOM TOWNSHIP (BLAIR COUNTY)",
    "PA0071900 - DCNR - CANOE CREEK STATE PARK",
    "PA007BF0X - DCNR - BUREAU OF FORESTRY (BLAIR COUNTY) ",
    "PA007DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (BLAIR COUNTY)",
    "PA0080000 - BRADFORD COUNTY SHERIFFS",
    "PA0080100 - ATHENS BOROUGH",
    "PA0080200 - CANTON BOROUGH",
    "PA0080300 - SAYRE BOROUGH",
    "PA0080400 - TOWANDA BOROUGH",
    "PA0080500 - TROY BOROUGH",
    "PA0080700 - SOUTH WAVERLY BOROUGH (COVERED BY SAYRE BOROUGH - PA0080300)",
    "PA0080800 - ATHENS TOWNSHIP",
    "PA0081200 - (INACTIVE) NORTH EAST POLICE (COVERED BY TROOP P - TOWANDA STATION - PAPSP8500)",
    "PA0081700 - LERAYSVILLE BOROUGH (COVERED BY NORTHEAST POLICE - PA0081200)",
    "PA0081800 - ULSTER TOWNSHIP (COVERED BY PSP - TROOP P - TOWANDA STATION - PAPSP8500)",
    "PA0082000 - DCNR - MT. PISGAH STATE PARK",
    "PA008BF0X - DCNR - BUREAU OF FORESTRY (BRADFORD COUNTY) ",
    "PA008DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (BRADFORD COUNTY)",
    "PA0090000 - BUCKS COUNTY SHERIFFS",
    "PA0090100 - BENSALEM TOWNSHIP",
    "PA0090200 - BRISTOL BOROUGH",
    "PA0090300 - BRISTOL TOWNSHIP",
    "PA0090400 - DOYLESTOWN BOROUGH (COVERED BY CENTRAL BUCKS REGIONAL - PA0096300)",
    "PA0090500 - FALLS TOWNSHIP (BUCKS COUNTY)",
    "PA0090700 - LOWER MAKEFIELD TOWNSHIP",
    "PA0090800 - LOWER SOUTHAMPTON TOWNSHIP",
    "PA0090900 - MIDDLETOWN TOWNSHIP",
    "PA0091000 - MORRISVILLE BOROUGH",
    "PA0091100 - NEW HOPE BOROUGH",
    "PA0091200 - NEWTOWN TOWNSHIP (BUCKS COUNTY)",
    "PA0091300 - NORTHAMPTON TOWNSHIP",
    "PA0091400 - PENNDEL BOROUGH",
    "PA0091500 - PERKASIE BOROUGH",
    "PA0091600 - QUAKERTOWN BOROUGH",
    "PA0091700 - PENNRIDGE REGIONAL",
    "PA0091800 - SOLEBURY TOWNSHIP",
    "PA0091900 - TULLYTOWN BOROUGH",
    "PA0092000 - UPPER SOUTHAMPTON TOWNSHIP",
    "PA0092100 - WARMINSTER TOWNSHIP",
    "PA0092200 - YARDLEY BOROUGH",
    "PA0092300 - UPPER MAKEFIELD TOWNSHIP",
    "PA0092500 - HULMEVILLE BOROUGH",
    "PA0092600 - (INACTIVE) CHALFONT BOROUGH (COVERED BY CENTRAL BUCKS REGIONAL - PA0096300)",
    "PA0092700 - DOYLESTOWN TOWNSHIP",
    "PA0092800 - EAST ROCKHILL TOWNSHIP (COVERED BY PENNRIDGE REGIONAL - PA0091700)",
    "PA0092900 - HILLTOWN TOWNSHIP",
    "PA0093000 - DCNR - NESHAMINY STATE PARK",
    "PA0093200 - NEW BRITAIN TOWNSHIP",
    "PA0093300 - DCNR - DELAWARE CANAL STATE PARK",
    "PA0093500 - SPRINGFIELD TOWNSHIP (BUCKS COUNTY)",
    "PA0093600 - WARWICK TOWNSHIP (BUCKS COUNTY)",
    "PA0093800 - WEST ROCKHILL TOWNSHIP (COVERED BY PENNRIDGE REGIONAL - PA0091700)",
    "PA0093900 - NEW BRITAIN BOROUGH (COVERED BY CENTRAL BUCKS REGIONAL - PA0096300)",
    "PA0094000 - DCNR - NOCKAMIXON STATE PARK",
    "PA0094100 - LANGHORNE MANOR BOROUGH",
    "PA0094200 - NOCKAMIXON TOWNSHIP (COVERED BY PSP - TROOP M - DUBLIN STATION - PAPSP1600)",
    "PA0094300 - PLUMSTEAD TOWNSHIP",
    "PA0094400 - WRIGHTSTOWN TOWNSHIP (COVERED BY NEWTOWN TOWNSHIP - PA0091200)",
    "PA0094500 - WARRINGTON TOWNSHIP",
    "PA0094700 - BUCKINGHAM TOWNSHIP",
    "PA0094800 - BUCKS COUNTY DETECTIVES",
    "PA0094900 - DCNR - TYLER STATE PARK",
    "PA0095100 - TINICUM TOWNSHIP (BUCKS COUNTY)",
    "PA0095200 - IVYLAND BOROUGH",
    "PA0095300 - BEDMINSTER TOWNSHIP",
    "PA0095400 - BUCKS COUNTY PARK POLICE",
    "PA0095700 - DUBLIN BOROUGH",
    "PA0096000 - NEWTOWN BOROUGH",
    "PA0096100 - LANGHORNE BOROUGH",
    "PA0096200 - RICHLAND TOWNSHIP (BUCKS COUNTY)",
    "PA0096300 - CENTRAL BUCKS REGIONAL",
    "PA009BF0X - DCNR - BUREAU OF FORESTRY (BUCKS COUNTY) ",
    "PA009DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (BUCKS COUNTY)",
    "PA0100000 - BUTLER COUNTY SHERIFFS",
    "PA0100100 - BUTLER CITY",
    "PA0100200 - BUTLER TOWNSHIP (BUTLER COUNTY)",
    "PA0100300 - SLIPPERY ROCK BOROUGH",
    "PA0100400 - ZELIENOPLE BOROUGH",
    "PA0100700 - EVANS CITY BOROUGH",
    "PA0100800 - MARS BOROUGH",
    "PA0100900 - MIDDLESEX TOWNSHIP (BUTLER COUNTY)",
    "PA0101000 - PENN TOWNSHIP (BUTLER COUNTY)",
    "PA0101100 - SLIPPERY ROCK UNIVERSITY",
    "PA0101200 - CRANBERRY TOWNSHIP",
    "PA0101700 - BUTLER COUNTY DETECTIVES",
    "PA0101800 - BUFFALO TOWNSHIP",
    "PA0101900 - SAXONBURG BOROUGH",
    "PA0102200 - JACKSON TOWNSHIP (BUTLER COUNTY)",
    "PA0102300 - CALLERY BOROUGH",
    "PA0102500 - LANCASTER TOWNSHIP (BUTLER COUNTY)",
    "PA0102800 - (INACTIVE) PORTERSVILLE BOROUGH (COVERED BY PSP - TROOP D BUTLER - PAPSP0900)",
    "PA0102900 - HARRISVILLE BOROUGH",
    "PA0103000 - PROSPECT BOROUGH",
    "PA0103300 - ADAMS TOWNSHIP (BUTLER COUNTY)",
    "PA0103700 - BUTLER COUNTY COMMUNITY COLLEGE",
    "PA0103800 - DCNR - MORAINE STATE PARK",
    "PA0103900 - DCNR - JENNINGS ENVIRONMENTAL EDUCATIONAL CENTER",
    "PA0104000 - (INACTIVE) EVANS CITY - SEVEN FIELDS REGIONAL (RETIRED ORI)",
    "PA010BF0X - DCNR - BUREAU OF FORESTRY (BUTLER COUNTY) ",
    "PA010DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (BUTLER COUNTY)",
    "PA0110000 - CAMBRIA COUNTY SHERIFFS",
    "PA0110100 - NORTHERN CAMBRIA BOROUGH",
    "PA0110200 - CONEMAUGH TOWNSHIP (CAMBRIA COUNTY)",
    "PA0110300 - EBENSBURG BOROUGH",
    "PA0110400 - JOHNSTOWN CITY",
    "PA0110500 - RICHLAND TOWNSHIP (CAMBRIA COUNTY)",
    "PA0110700 - CRESSON BOROUGH",
    "PA0110800 - DALE BOROUGH (COVERED BY JOHNSTOWN CITY - PA0110400)",
    "PA0110900 - EAST CONEMAUGH BOROUGH",
    "PA0111000 - GALLITZIN BOROUGH",
    "PA0111100 - GEISTOWN BOROUGH",
    "PA0111200 - NANTY GLO BOROUGH",
    "PA0111300 - PATTON BOROUGH",
    "PA0111400 - PORTAGE BOROUGH",
    "PA0111500 - SOUTH FORK BOROUGH ( COVERED BY PSP - TROOP A - EBENSBURG - PAPSP2200)",
    "PA0111600 - SOUTHMONT BOROUGH (COVERED BY WEST HILLS REGIONAL - PA0115900)",
    "PA0111800 - UPPER YODER TOWNSHIP",
    "PA0111900 - BLACKLICK TOWNSHIP",
    "PA0112000 - CAMBRIA TOWNSHIP",
    "PA0112100 - CARROLLTOWN BOROUGH",
    "PA0112300 - CRESSON TOWNSHIP",
    "PA0112600 - EAST TAYLOR TOWNSHIP",
    "PA0112800 - ELDER TOWNSHIP (COVERED BY HASTINGS BOROUGH - PA0113100)",
    "PA0112900 - FERNDALE BOROUGH",
    "PA0113000 - FRANKLIN BOROUGH (COVERED BY PSP - TROOP A - EBENSBURG STATION - PAPSP2200)",
    "PA0113100 - HASTINGS BOROUGH",
    "PA0113200 - JACKSON TOWNSHIP (CAMBRIA COUNTY)",
    "PA0113300 - LILLY BOROUGH (COVERED BY PSP - TROOP A - EBENSBURG STATION - PAPSP2200)",
    "PA0113500 - LORETTO BOROUGH",
    "PA0113700 - SANKERTOWN BOROUGH",
    "PA0114000 - STONYCREEK TOWNSHIP",
    "PA0114200 - SUMMERHILL TOWNSHIP",
    "PA0114300 - WEST CARROLL TOWNSHIP (COVERED BY CARROLLTOWN BOROUGH - PA0112100)",
    "PA0114800 - LOWER YODER TOWNSHIP (COVERED BY WEST HILLS REGIONAL - PA0115900)",
    "PA0115000 - SUSQUEHANNA TOWNSHIP (CAMBRIA COUNTY) (COVERED BY PSP - TROOP A - EBENSBURG STATION - PAPSP2200)",
    "PA0115100 - CAMBRIA COUNTY DISTRICT ATTORNEYS OFFICE ",
    "PA0115200 - ADAMS TOWNSHIP (CAMBRIA COUNTY)",
    "PA0115300 - CROYLE TOWNSHIP",
    "PA0115400 - (INACTIVE) ALLEGHENY TOWNSHIP (CAMBRIA COUNTY) (COVERED BY PSP - TROOP A - EBENSBURG - PAPSP2200)",
    "PA0115500 - (INACTIVE) ASHVILLE BOROUGH (COVERED BY PSP - TROOP A - EBENSBURG STATION - PAPSP2200)",
    "PA0115600 - VINTONDALE BOROUGH",
    "PA0115800 - GALLITZIN TOWNSHIP",
    "PA0115900 - WEST HILLS REGIONAL",
    "PA0116000 - TUNNELL HILL BOROUGH (COVERED BY GALLITZIN BOROUGH - PA0111000)",
    "PA0116100 - WASHINGTON TOWNSHIP (CAMBRIA COUNTY) (COVERED BY PSP TROOP A EBENSBURG - PAPSP2200)",
    "PA0116500 - UNIVERSITY OF PITTSBURGH - JOHNSTOWN CAMPUS ",
    "PA0116800 - DCNR - PRINCE GALLITZIN STATE PARK",
    "PA0116900 - FOREST RANGERS GALLITZIN STATE FOREST DISTRICT 6",
    "PA011809E - MOUNT ALOYSIUS UNIVERSITY",
    "PA011BF0X - DCNR - BUREAU OF FORESTRY (CAMBRIA COUNTY) ",
    "PA011DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (CAMBRIA COUNTY)",
    "PA0120100 - EMPORIUM BOROUGH",
    "PA0120300 - DCNR - SIZERVILLE STATE PARK",
    "PA0120500 - DCNR - SINNEMAHONING STATE PARK",
    "PA012BF0X - DCNR - BUREAU OF FORESTRY (CAMERON COUNTY) ",
    "PA012DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (CAMERON COUNTY)",
    "PA0130000 - CARBON COUNTY SHERIFFS OFFICE",
    "PA0130200 - JIM THORPE BOROUGH",
    "PA0130300 - LANSFORD BOROUGH",
    "PA0130400 - LEHIGHTON BOROUGH",
    "PA0130500 - PALMERTON BOROUGH",
    "PA0130600 - SUMMIT HILL BOROUGH",
    "PA0130700 - WEATHERLY BOROUGH",
    "PA0130800 - FRANKLIN TOWNSHIP (CARBON COUNTY)",
    "PA0130900 - KIDDER TOWNSHIP",
    "PA0131000 - MAHONING TOWNSHIP (CARBON COUNTY)",
    "PA0131100 - NESQUEHONING BOROUGH",
    "PA0131200 - BEAVER MEADOWS BOROUGH",
    "PA0131500 - EAST PENN TOWNSHIP (COVERED BY PSP - TROOP N - LEHIGHTON STATION - PAPSP4800)",
    "PA0131600 - WEISSPORT BOROUGH",
    "PA0132000 - DCNR - BELTZVILLE STATE PARK",
    "PA0132100 - DCNR - HICKORY RUN STATE PARK",
    "PA013BF0X - DCNR - BUREAU OF FORESTRY (CARBON COUNTY) ",
    "PA013DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (CARBON COUNTY)",
    "PA0140000 - CENTRE COUNTY SHERIFFS",
    "PA0140100 - BELLEFONTE BOROUGH",
    "PA0140200 - PHILIPSBURG BOROUGH (COVERED BY PSP - TROOP G - ROCKVIEW STATION - PAPSP7400)",
    "PA0140300 - STATE COLLEGE BOROUGH",
    "PA0140500 - SPRING TOWNSHIP (CENTRE COUNTY)",
    "PA0140700 - FERGUSON TOWNSHIP",
    "PA0140900 - PATTON TOWNSHIP",
    "PA0141100 - PSU - MAIN CAMPUS",
    "PA0141700 - DCNR - BLACK MOSHANNON STATE PARK",
    "PA0141800 - DCNR - BALD EAGLE STATE PARK",
    "PA014BF0X - DCNR - BUREAU OF FORESTRY (CENTRE COUNTY) ",
    "PA014DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (CENTRE COUNTY)",
    "PA0150000 - CHESTER COUNTY SHERIFFS",
    "PA0150100 - COATESVILLE CITY",
    "PA0150200 - DOWNINGTOWN BOROUGH",
    "PA0150300 - EASTTOWN TOWNSHIP",
    "PA0150400 - EAST WHITELAND TOWNSHIP",
    "PA0150500 - KENNETT SQUARE BOROUGH",
    "PA0150600 - MALVERN BOROUGH",
    "PA0150700 - NORTH COVENTRY TOWNSHIP",
    "PA0150800 - OXFORD BOROUGH",
    "PA0150900 - PARKESBURG BOROUGH",
    "PA0151000 - PHOENIXVILLE BOROUGH",
    "PA0151100 - SPRING CITY BOROUGH",
    "PA0151200 - TREDYFFRIN TOWNSHIP",
    "PA0151300 - WEST CHESTER BOROUGH",
    "PA0151400 - WEST GOSHEN TOWNSHIP",
    "PA0151500 - WEST GROVE BOROUGH (COVERED BY SOUTHERN CHESTER COUNTY REGIONAL - PA0156700)",
    "PA0151600 - WILLISTOWN TOWNSHIP",
    "PA0151700 - CALN TOWNSHIP",
    "PA0151800 - (INACTIVE) ATGLEN BOROUGH (COVERED BY CHRISTIANA BOROUGH - PA0361800)",
    "PA0151900 - (INACTVE) AVONDALE BOROUGH (COVERED BY SOUTHERN CHESTER COUNTY REGIONAL - PA0156700)",
    "PA0152000 - BIRMINGHAM TOWNSHIP",
    "PA0152100 - EAST FALLOWFIELD TOWNSHIP",
    "PA0152200 - SCHUYLKILL TOWNSHIP (CHESTER COUNTY)",
    "PA0152300 - SOUTH COATESVILLE BOROUGH",
    "PA0152400 - THORNBURY TOWNSHIP (COVERED BY PSP - TROOP J - AVONDALE - PAPSP0200)",
    "PA0152500 - UPPER UWCHLAN TOWNSHIP",
    "PA0152700 - WESTTOWN-EAST GOSHEN REGIONAL",
    "PA0152800 - UWCHLAN TOWNSHIP",
    "PA0152900 - HONEY BROOK BOROUGH",
    "PA0153200 - EAST PIKELAND TOWNSHIP",
    "PA0153300 - EAST VINCENT TOWNSHIP",
    "PA0153400 - HIGHLAND TOWNSHIP (COVERED BY PARKESBURG BOROUGH - PA0150900)",
    "PA0153500 - NEW GARDEN TOWNSHIP (COVERED BY SOUTHERN CHESTER COUNTY REGIONAL - PA0156700)",
    "PA0153700 - WEST FALLOWFIELD TOWNSHIP",
    "PA0153800 - WEST SADSBURY TOWNSHIP",
    "PA0153900 - WEST VINCENT TOWNSHIP",
    "PA0154000 - WEST WHITELAND TOWNSHIP",
    "PA0154200 - HONEY BROOK TOWNSHIP (COVERED BY PSP - TROOP J - EMBREEVILLE STATION - PAPSP2700)",
    "PA0154400 - VALLEY TOWNSHIP",
    "PA0154500 - CHESTER COUNTY DETECTIVES",
    "PA0154600 - WEST CHESTER UNIVERSITY",
    "PA0154700 - LINCOLN UNIVERSITY - MAIN CAMPUS",
    "PA0154800 - EAST COVENTRY TOWNSHIP",
    "PA0154900 - EAST BRANDYWINE TOWNSHIP",
    "PA0155000 - SADSBURY TOWNSHIP",
    "PA0155200 - WEST CALN TOWNSHIP",
    "PA0155300 - WEST NOTTINGHAM TOWNSHIP",
    "PA0155500 - WEST BRANDYWINE TOWNSHIP",
    "PA0155700 - WEST PIKELAND TOWNSHIP",
    "PA0155900 - DCNR - WHITE CLAY CREEK STATE PARK",
    "PA0156000 - DCNR - MARSH CREEK STATE PARK",
    "PA0156100 - FOREST RANGERS WILLIAM PENN STATE FOREST DISTRICT 17",
    "PA0156200 - EAST MARLBOROUGH TOWNSHIP",
    "PA0156500 - KENNETT TOWNSHIP",
    "PA0156600 - PSU - GREAT VALLEY CAMPUS",
    "PA0156700 - SOUTHERN CHESTER COUNTY REGIONAL",
    "PA015AG00 - (INACTIVE) CHESTER COUNTY ATTORNEY GENERALS OFFICE",
    "PA015BF0X - DCNR - BUREAU OF FORESTRY (CHESTER COUNTY) ",
    "PA015DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (CHESTER COUNTY)",
    "PA0160000 - CLARION COUNTY SHERIFFS",
    "PA0160100 - CLARION BOROUGH",
    "PA0160200 - (INACTIVE) NEW BETHLEHEM BOROUGH (COVERED BY SOUTHERN CLARION COUNTY REGIONAL - PA0161600)",
    "PA0160300 - EAST BRADY BOROUGH (COVERED BY SOUTHERN CLARION COUNTY REGIONAL - PA0161600)",
    "PA0160400 - KNOX BOROUGH",
    "PA0160500 - RIMERSBURG BOROUGH PA0160500 (COVERED BY SOUTHERN CLARION COUNTY REGIONAL - PA0161600)",
    "PA0160600 - CLARION COUNTY DETECTIVES",
    "PA0160700 - CLARION UNIVERSITY OF PENNSYLVANIA",
    "PA0161100 - DCNR - COOK FOREST STATE PARK",
    "PA0161500 - FOREST RANGERS CLEAR CREEK STATE FOREST DISTRICT 8",
    "PA0161600 - SOUTHERN CLARION REGIONAL POLICE DEPARTMENT ",
    "PA016BF0X - DCNR - BUREAU OF FORESTRY (CLARION COUNTY) ",
    "PA016DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (CLARION COUNTY)",
    "PA0170000 - (INACTIVE) CLEARFIELD COUNTY SHERIFFS",
    "PA0170100 - (INACTIVE) CLEARFIELD BOROUGH (COVERED BY CLEARFIELD REGIONAL - PA0172300)",
    "PA0170200 - DUBOIS CITY",
    "PA0170300 - CURWENSVILLE BOROUGH",
    "PA0170500 - (INACTIVE) CHESTER HILL BOROUGH (COVERED BY PSP - TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0170600 - HOUTZDALE BOROUGH (COVERED BY PSP - TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0170700 - OSCEOLA MILLS BOROUGH (COVERED BY PSP - TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0170800 - SANDY TOWNSHIP",
    "PA0171100 - (INACTIVE) LAWRENCE TOWNSHIP (CLEARFIELD COUNTY) (COVERED BY CLEARFIELD REGIONAL - PA0172300)",
    "PA0171300 - (INACTIVE) COALPORT BOROUGH (COVERED BY PSP - TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0171400 - DECATUR TOWNSHIP (COVERED BY TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0171600 - MORRIS-COOPER REGIONAL (COVERED BY TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0171800 - DCNR - PARKER DAM STATE PARK",
    "PA0171900 - FOREST RANGERS MOSHANNON STATE FOREST DISTRICT 9",
    "PA0172100 - PSU - DUBOIS CAMPUS",
    "PA0172200 - (INACTIVE) BIGLER TOWNSHIP REGIONAL(COVERED BY PSP - TROOP C - CLEARFIELD STATION - PAPSP1300)",
    "PA0172300 - CLEARFIELD REGIONAL POLICE DEPARTMENT",
    "PA017BF0X - DCNR - BUREAU OF FORESTRY (CLEARFIELD COUNTY)",
    "PA017DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (CLEARFIELD COUNTY)",
    "PA0180100 - LOCK HAVEN CITY",
    "PA0180200 - RENOVO BOROUGH",
    "PA0180300 - (INACTIVE) AVIS BOROUGH (COVERED BY PINE CREEK TOWNSHIP - PA0181600)",
    "PA0180500 - DUNNSTABLE TOWNSHIP (COVERED BY PINE CREEK TOWNSHIP - PA0181600)",
    "PA0180600 - WOODWARD TOWNSHIP",
    "PA0180700 - FLEMINGTON BOROUGH (COVERED BY PSP - TROOP F - LAMAR STATION - PAPSP5000)",
    "PA0180900 - MILL HALL BOROUGH",
    "PA0181000 - (INACTIVE) CASTANEA TOWNSHIP (COVERED BY MILL HALL BOROUGH - PA0180900)",
    "PA0181100 - (INACTIVE) CLINTON COUNTY DETECTIVES",
    "PA0181200 - LOCK HAVEN UNIVERSITY OF PENNSYLVANIA - MAIN CAMPUS",
    "PA0181500 - (INACTIVE) LAMAR TOWNSHIP (COVERED BY MILL HALL BOROUGH - PA0180900)",
    "PA0181600 - PINE CREEK TOWNSHIP",
    "PA0182500 - DCNR - KETTLE CREEK STATE PARK",
    "PA0182600 - FOREST RANGERS SPROUL STATE FOREST DISTRICT 10",
    "PA018BF0X - DCNR - BUREAU OF FORESTRY (CLINTON COUNTY) ",
    "PA018DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (CLINTON COUNTY)",
    "PA0190100 - BENTON BOROUGH",
    "PA0190200 - BERWICK BOROUGH",
    "PA0190300 - BLOOMSBURG BOROUGH",
    "PA0190400 - CATAWISSA BOROUGH",
    "PA0190500 - BRIAR CREEK TOWNSHIP",
    "PA0190600 - MILLVILLE BOROUGH (COVERED BY HEMLOCK TOWNSHIP - PA0191400)",
    "PA0190800 - SCOTT TOWNSHIP (COLUMBIA COUNTY)",
    "PA0190900 - BLOOMSBURG UNIVERSITY OF PENNSYLVANIA",
    "PA0191000 - SOUTH CENTRE TOWNSHIP",
    "PA0191200 - MONTOUR TOWNSHIP",
    "PA0191300 - NORTH CENTRE TOWNSHIP (COVERED BY BRIAR CREEK TOWNSHIP - PA0190500)",
    "PA0191400 - HEMLOCK TOWNSHIP",
    "PA0191500 - LOCUST TOWNSHIP",
    "PA0191700 - (INACTIVE) BENTON TOWNSHIP (COVERED BY PSP - TROOP N - BLOOMSBURG - PAPSP0700)",
    "PA0191800 - (INACTIVE) CATAWISSA TOWNSHIP (COVERED BY MONTOUR TOWNSHIP - PA0191200)",
    "PA0191900 - (INACTIVE) CONYNGHAM TOWNSHIP (COVERED BY PSP - TROOP N - BLOOMSBURG STATION - PAPSP0700)",
    "PA0192200 - MAIN TOWNSHIP",
    "PA0192300 - ORANGE TOWNSHIP (COVERED BY ORANGE TOWNSHIP - PA0192400)",
    "PA0192400 - ORANGE TOWNSHIP POLICE DEPARTMENT",
    "PA0192500 - FRANKLIN TOWNSHIP (COLUMBIA COUNTY) (COVERED BY LOCUST TOWNSHIP - PA0191500)",
    "PA0192600 - MADISON TOWNSHIP (COVERED BY HEMLOCK TOWNSHIP - PA0191400)",
    "PA0192800 - STILLWATER BOROUGH (COVERED BY PSP - TROOP N - BLOOMSBURG STATION - PAPSP0700)",
    "PA0192900 - GREENWOOD TOWNSHIP (COVERED BY PSP - TROOP N - BLOOMSBURG STATION - PAPSP0700)",
    "PA0193000 - ROARING CREEK TOWNSHIP (COVERED BY LOCUST TOWNSHIP - PA0191500)",
    "PA019BF0X - DCNR - BUREAU OF FORESTRY (COLUMBIA COUNTY) ",
    "PA019DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (COLUMBIA COUNTY)",
    "PA0200100 - MEADVILLE CITY",
    "PA0200200 - TITUSVILLE CITY",
    "PA0200300 - CAMBRIDGE SPRINGS BOROUGH",
    "PA0200500 - COCHRANTON BOROUGH",
    "PA0200700 - (INACTIVE) CONNEAUTVILLE BOROUGH (COVERED BY PSP - TROOP E - MEADVILLE STATION - PAPSP5500)",
    "PA0200800 - LINESVILLE BOROUGH (INACTIVE) ( COVERED BY CONNEAUT LAKE REGIONAL POLICE DEPARTMENT - PA0201800",
    "PA0200900 - DCNR - PYMATUNING STATE PARK",
    "PA0201100 - (INACTIVE) VERNON TOWNSHIP (COVERED BY MEADVILLE CITY - PA0200100)",
    "PA0201400 - WEST MEAD TOWNSHIP",
    "PA0201800 - CONNEAUT LAKE REGIONAL",
    "PA0201900 - UNIVERSITY OF PITTSBURGH - TITUSVILLE CAMPUS",
    "PA020239E - ALLEGHENY COLLEGE",
    "PA020BF0X - DCNR - BUREAU OF FORESTRY (CRAWFORD COUNTY) ",
    "PA020DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (CRAWFORD COUNTY)",
    "PA0210000 - CUMBERLAND COUNTY SHERIFFS",
    "PA0210100 - CAMP HILL BOROUGH",
    "PA0210200 - CARLISLE BOROUGH",
    "PA0210300 - EAST PENNSBORO TOWNSHIP",
    "PA0210400 - HAMPDEN TOWNSHIP",
    "PA0210500 - WEST SHORE REGIONAL",
    "PA0210600 - LOWER ALLEN TOWNSHIP",
    "PA0210700 - MECHANICSBURG BOROUGH",
    "PA0210800 - NEW CUMBERLAND BOROUGH",
    "PA0210900 - SHIPPENSBURG BOROUGH",
    "PA0211000 - SHIREMANSTOWN BOROUGH",
    "PA0211200 - WORMLEYSBURG BOROUGH (COVERED BY WEST SHORE REGIONAL - PA0210500)",
    "PA0211300 - MIDDLESEX TOWNSHIP (CUMBERLAND COUNTY)",
    "PA0211400 - MOUNT HOLLY SPRINGS BOROUGH",
    "PA0211500 - NORTH MIDDLETON TOWNSHIP",
    "PA0211600 - SHIPPENSBURG UNIVERSITY",
    "PA0211700 - SILVER SPRING TOWNSHIP",
    "PA0211800 - UPPER ALLEN TOWNSHIP",
    "PA0211900 - NEWVILLE BOROUGH (COVERED BY PSP - TROOP H - CARLISLE STATION - PAPSP1000)",
    "PA0212100 - CUMBERLAND COUNTY DETECTIVES (COVERED BY CUMBERLAND COUNTY D.A. - PA021013A)",
    "PA0212300 - DCNR - COLONEL DENNING STATE PARK",
    "PA0212500 - DCNR - PINE GROVE FURNACE STATE PARK",
    "PA021269E - DICKINSON COLLEGE",
    "PA0213500 - DCNR - KINGS GAP ENVIRONMENTAL EDUCATION CENTER",
    "PA0213800 - PSU - CARLISLE CAMPUS",
    "PA021BF0X - DCNR - BUREAU OF FORESTRY (CUMBERLAND COUNTY)",
    "PA021DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (CUMBERLAND COUNTY)",
    "PA021SP0X - PSP - TROOP H - HARRISBURG STATION (CUMBERLAND COUNTY)",
    "PA0220100 - ELIZABETHVILLE BOROUGH (COVERED BY PSP - TROOP H - LYKENS STATION PAPSP5100)",
    "PA0220200 - HARRISBURG CITY",
    "PA0220300 - HUMMELSTOWN BOROUGH",
    "PA0220400 - LOWER PAXTON TOWNSHIP",
    "PA0220500 - MIDDLETOWN BOROUGH",
    "PA0220600 - PAXTANG BOROUGH (COVERED BY SWATARA TOWNSHIP - PA0220900)",
    "PA0220700 - STEELTON BOROUGH",
    "PA0220800 - SUSQUEHANNA TOWNSHIP (DAUPHIN COUNTY)",
    "PA0220900 - SWATARA TOWNSHIP",
    "PA0221000 - HIGHSPIRE BOROUGH",
    "PA0221100 - (INACTIVE) LYKENS BOROUGH (COVERED BY PSP - TROOP H - LYKENS STATION - PAPSP5100)",
    "PA0221200 - MILLERSBURG BOROUGH",
    "PA0221300 - PENBROOK BOROUGH",
    "PA0221400 - WILLIAMSTOWN BOROUGH (COVERED BY PSP - TROOP H - LYKENS STATION PAPSP5100)",
    "PA0221500 - DERRY TOWNSHIP (DAUPHIN COUNTY)",
    "PA0221600 - HALIFAX BOROUGH",
    "PA0221700 - LOWER SWATARA TOWNSHIP",
    "PA0221800 - WICONISCO TOWNSHIP",
    "PA0222100 - ROYALTON BOROUGH",
    "PA0222200 - HALIFAX TOWNSHIP (COVERED BY PSP - TROOP H - LYKENS STATION - PAPSP5100)",
    "PA0222300 - CAPITOL POLICE",
    "PA0222400 - (INACTIVE) BUTLER COUNTY ATTORNEY GENERAL ",
    "PA0222600 - DAUPHIN COUNTY DETECTIVES",
    "PA0222700 - DAUPHIN BOROUGH (COVERED BY PSP - TROOP H - HARRISBURG STATION - PAPSP3600)",
    "PA0222800 - PSU - HARRISBURG CAMPUS",
    "PA0223200 - SUSQUEHANNA AREA REGIONAL AIRPORT AUTHORITY (HBG INTL AIRPORT)",
    "PA0223400 - PENNSYLVANIA FISH AND BOAT COMMISSION",
    "PA0224200 - BOF FIRE PROTECTION DIVISION HARRISBURG ",
    "PA022AG00 - (INACTIVE) DAUPHIN COUNTY ATTORNEY GENERALS OFFICE",
    "PA022BF0X - DCNR - BUREAU OF FORESTRY (DAUPHIN COUNTY) ",
    "PA022DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (DAUPHIN COUNTY)",
    "PA022SP00 - 123 TEST",
    "PA0230000 - (INACTIVE) DELAWARE COUNTY SHERIFF",
    "PA0230100 - ASTON TOWNSHIP",
    "PA0230200 - BETHEL TOWNSHIP (DELAWARE COUNTY)",
    "PA0230300 - BROOKHAVEN BOROUGH",
    "PA0230400 - CHESTER CITY",
    "PA0230500 - CLIFTON HEIGHTS BOROUGH",
    "PA0230600 - COLLINGDALE BOROUGH",
    "PA0230700 - COLWYN BOROUGH",
    "PA0230800 - DARBY BOROUGH",
    "PA0230900 - DARBY TOWNSHIP",
    "PA0231000 - EAST LANSDOWNE BOROUGH",
    "PA0231100 - EDDYSTONE BOROUGH",
    "PA0231200 - FOLCROFT BOROUGH",
    "PA0231300 - GLENOLDEN BOROUGH",
    "PA0231400 - HAVERFORD TOWNSHIP",
    "PA0231500 - LANSDOWNE BOROUGH",
    "PA0231600 - LOWER CHICHESTER TOWNSHIP",
    "PA0231700 - MARPLE TOWNSHIP",
    "PA0231800 - MEDIA BOROUGH",
    "PA0231900 - MILLBOURNE BOROUGH",
    "PA0232000 - MARCUS HOOK BOROUGH",
    "PA0232100 - MORTON BOROUGH",
    "PA0232200 - NETHER PROVIDENCE TOWNSHIP",
    "PA0232300 - NEWTOWN TOWNSHIP (DELAWARE COUNTY)",
    "PA0232400 - NORWOOD BOROUGH",
    "PA0232500 - PROSPECT PARK BOROUGH",
    "PA0232600 - RADNOR TOWNSHIP",
    "PA0232700 - RIDLEY PARK BOROUGH",
    "PA0232800 - RIDLEY TOWNSHIP",
    "PA0233000 - SHARON HILL BOROUGH",
    "PA0233100 - SPRINGFIELD TOWNSHIP (DELAWARE COUNTY)",
    "PA0233200 - SWARTHMORE BOROUGH",
    "PA0233300 - TINICUM TOWNSHIP (DELAWARE COUNTY)",
    "PA0233400 - TRAINER BOROUGH",
    "PA0233500 - UPLAND BOROUGH",
    "PA0233600 - UPPER CHICHESTER TOWNSHIP",
    "PA0233700 - UPPER DARBY TOWNSHIP",
    "PA0233800 - UPPER PROVIDENCE TOWNSHIP (DELAWARE COUNTY) ",
    "PA0233900 - YEADON BOROUGH",
    "PA0234000 - ALDAN BOROUGH",
    "PA0234200 - CHESTER TOWNSHIP",
    "PA0234400 - DELAWARE COUNTY PARK POLICE",
    "PA0234500 - PARKSIDE BOROUGH",
    "PA0234800 - DELAWARE COUNTY DISTRICT ATTORNEY - CRIMINAL INVESTIGATIVE DIVISION",
    "PA0235000 - CHEYNEY UNIVERSITY OF PENNSYLVANIA",
    "PA0235300 - DCNR - RIDLEY CREEK STATE PARK",
    "PA0235500 - PSU - BRANDYWINE CAMPUS",
    "PA023579E - VILLANOVA UNIVERSITY",
    "PA0235900 - CHESTER HOUSING AUTHORITY POLICE",
    "PA023BF0X - DCNR - BUREAU OF FORESTRY (DELAWARE COUNTY) ",
    "PA023DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (DELAWARE COUNTY)",
    "PA0240000 - ELK COUNTY SHERIFFS",
    "PA0240100 - JOHNSONBURG BOROUGH",
    "PA0240200 - RIDGWAY BOROUGH",
    "PA0240300 - SAINT MARYS CITY",
    "PA0242500 - DCNR - BENDIGO STATE PARK",
    "PA0242600 - FOREST RANGERS ELK STATE FOREST DISTRICT 13 ",
    "PA024BF0X - DCNR - BUREAU OF FORESTRY (ELK COUNTY)",
    "PA024DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (ELK COUNTY)",
    "PA0250000 - ERIE COUNTY SHERIFFS",
    "PA0250100 - CORRY CITY",
    "PA0250200 - ERIE CITY",
    "PA0250300 - MILLCREEK TOWNSHIP (ERIE COUNTY)",
    "PA0250400 - NORTH EAST BOROUGH",
    "PA0250500 - WESLEYVILLE BOROUGH",
    "PA0250600 - LAWRENCE PARK TOWNSHIP",
    "PA0250700 - UNION CITY BOROUGH",
    "PA0250800 - (INACTIVE) ALBION BOROUGH (COVERED BY PSP - TROOP E - GIRARD STATION (PAPSP3300)",
    "PA0250900 - EDINBORO BOROUGH",
    "PA0251100 - GIRARD BOROUGH",
    "PA0251200 - LAKE CITY BOROUGH",
    "PA0251300 - DCNR - PRESQUE ISLE STATE PARK",
    "PA0251600 - ERIE COUNTY DETECTIVES",
    "PA0251700 - PENNWEST UNIVERSITY - EDINBORO POLICE DEPARTMENT",
    "PA0251900 - PSU - BEHREND CAMPUS",
    "PA0252200 - ERIE INTERNATIONAL AIRPORT",
    "PA025239E - MERCYHURST UNIVERSITY",
    "PA025249E - GANNON UNIVERSITY",
    "PA025BF0X - DCNR - BUREAU OF FORESTRY (ERIE COUNTY) ",
    "PA025DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (ERIE COUNTY)",
    "PA0260100 - BROWNSVILLE BOROUGH",
    "PA0260200 - CONNELLSVILLE CITY",
    "PA0260300 - UNIONTOWN CITY",
    "PA0260400 - SOUTHWEST REGIONAL POLICE",
    "PA0260500 - MASONTOWN BOROUGH",
    "PA0260700 - (INACTIVE) SOUTH CONNELLSVILLE BOROUGH (COVERED BY CONNELLSVILLE CITY PA0260200)",
    "PA0260800 - WASHINGTON TOWNSHIP (FAYETTE COUNTY)",
    "PA0261000 - DUNBAR BOROUGH",
    "PA0261100 - EVERSON BOROUGH",
    "PA0261200 - FAIRCHANCE BOROUGH (COVERED BY PSP - TROOP B - UNIONTOWN - PAPSP8800)",
    "PA0261300 - FAYETTE CITY BOROUGH (COVERED BY PSP - TROOP B - BELLE VERNON STATION - PAPSP0500)",
    "PA0261500 - LUZERNE TOWNSHIP",
    "PA0261600 - MENALLEN TOWNSHIP (COVERED BY GERMAN TOWNSHIP - PA0264100)",
    "PA0261800 - PERRYOPOLIS BOROUGH",
    "PA0261900 - POINT MARION BOROUGH",
    "PA0262000 - REDSTONE TOWNSHIP",
    "PA0262100 - FAYETTE COUNTY DETECTIVES",
    "PA0262400 - SOUTH UNION TOWNSHIP (COVERED BY PSP - TROOP B - UNIONTOWN STATION - PAPSP8800)",
    "PA0262500 - (INACTIVE) SMITHFIELD BOROUGH (COVERED BY PSP - TROOP B - UNIONTOWN STATION - PAPSP8800)",
    "PA0262800 - NEWELL BOROUGH (COVERED BY SOUTHWEST REGIONAL POLICE - PA0260400)",
    "PA0262900 - WHARTON TOWNSHIP (COVERED BY PSP - TROOP B - UNIONTOWN STATION - PAPSP8800)",
    "PA0263600 - DCNR - OHIOPYLE STATE PARK",
    "PA026389P - (INACTIVE) UNIONTOWN HOSPITAL",
    "PA0264000 - PSU - FAYETTE CAMPUS",
    "PA0264100 - GERMAN TOWNSHIP",
    "PA0264200 - FAYETTE COUNTY CAREER AND TECHNICAL INSTITUTE",
    "PA026BF0X - DCNR - BUREAU OF FORESTRY (FAYETTE COUNTY) ",
    "PA026DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (FAYETTE COUNTY)",
    "PA027BF0X - DCNR - BUREAU OF FORESTRY (FOREST COUNTY) ",
    "PA027DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (FOREST COUNTY)",
    "PA0280000 - FRANKLIN COUNTY SHERIFFS",
    "PA0280100 - CHAMBERSBURG BOROUGH",
    "PA0280200 - GREENCASTLE BOROUGH",
    "PA0280300 - WAYNESBORO BOROUGH",
    "PA0280400 - MERCERSBURG BOROUGH",
    "PA0280600 - WASHINGTON TOWNSHIP (FRANKLIN COUNTY)",
    "PA0281000 - PSU - MONT ALTO CAMPUS",
    "PA0281100 - DCNR - CALEDONIA STATE PARK",
    "PA0281200 - FOREST RANGERS MICHAUX STATE FOREST DISTRICT 1",
    "PA0281400 - FRANKLIN COUNTY DRUG TASK FORCE",
    "PA028BF0X - DCNR - BUREAU OF FORESTRY (FRANKLIN COUNTY) ",
    "PA028DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (FRANKLIN COUNTY)",
    "PA0290100 - MCCONNELLSBURG BOROUGH (COVERED BY PSP - TROOP G - MCCONNELLSBURG STATION - PAPSP5400)",
    "PA0290300 - DCNR - COWANS GAP STATE PARK",
    "PA0290400 - FOREST RANGERS BUCHANAN STATE FOREST DISTRICT 2",
    "PA029BF0X - DCNR - BUREAU OF FORESTRY (FULTON COUNTY) ",
    "PA029DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (FULTON COUNTY)",
    "PA0300000 - GREENE COUNTY SHERIFFS",
    "PA0300100 - WAYNESBURG BOROUGH",
    "PA0300200 - (INACTIVE) CARMICHAELS BOROUGH (COVERED BY CUMBERLAND TOWNSHIP PA0300400)",
    "PA0300300 - GREENE COUNTY DISTRICT ATTORNEYS OFFICE ",
    "PA0300400 - CUMBERLAND TOWNSHIP GREENE COUNTY)",
    "PA0300500 - PERRY TOWNSHIP (GREENE COUNTY) (COVERED BY GREENE COUNTY REGIONAL - PA0301200)",
    "PA0300600 - WAYNE TOWNSHIP (GREENE COUNTY) (COVERED BY PSP - TROOP B - WAYNESBURG - PAPSP9200)",
    "PA0300800 - DCNR - RYERSON STATION STATE PARK",
    "PA0301000 - SOUTHWEST REGIONAL - GREENE CO. (COVERED BY SOUTHWEST REGIONAL POLICE - PA0260400)",
    "PA0301100 - MORRIS TOWNSHIP (GREENE COUNTY) (COVERED BY GREENE COUNTY REGIONAL - PA0310200)",
    "PA0301200 - (INACTIVE) GREENE COUNTY REGIONAL - GREENE COUNTY",
    "PA030BF0X - DCNR - BUREAU OF FORESTRY (GREENE COUNTY) ",
    "PA030DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (GREENE COUNTY)",
    "PA030SP0X - PSP - TROOP T - NEWVILLE STATION (FULTON COUNTY)",
    "PA0310000 - (INACTIVE) HUNTINGDON COUNTY SHERIFFS",
    "PA0310100 - HUNTINGDON BOROUGH",
    "PA0310200 - MOUNT UNION BOROUGH",
    "PA0310400 - JUNIATA VALLEY REGIONAL (COVERED BY PSP - TROOP G - HUNTINGDON STATION - PAPSP4100)",
    "PA0310900 - DCNR - TROUGH CREEK STATE PARK",
    "PA0311000 - DCNR - GREENWOOD FURNACE STATE PARK",
    "PA0311200 - FOREST RANGERS ROTHROCK STATE FOREST DISTRICT 5",
    "PA031139E - JUNIATA COLLEGE",
    "PA031BF0X - DCNR - BUREAU OF FORESTRY (HUNTINGDON COUNTY)",
    "PA031DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (HUNTINGDON COUNTY)",
    "PA0320000 - INDIANA COUNTY SHERIFFS",
    "PA0320100 - BLAIRSVILLE BOROUGH",
    "PA0320200 - HOMER CITY BOROUGH",
    "PA0320300 - INDIANA BOROUGH",
    "PA0320400 - CLYMER BOROUGH",
    "PA0320500 - CHERRY TREE BOROUGH",
    "PA0320700 - SALTSBURG BOROUGH",
    "PA0320900 - INDIANA COUNTY DETECTIVES",
    "PA0321000 - INDIANA UNIVERSITY OF PENNSYLVANIA - MAIN CAMPUS",
    "PA0321500 - MARION CENTER BOROUGH (COVERED BY CLYMER BOROUGH - PA0320400)",
    "PA0321600 - DCNR - YELLOW CREEK STATE PARK",
    "PA032BF0X - DCNR - BUREAU OF FORESTRY (INDIANA COUNTY) ",
    "PA032DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (INDIANA COUNTY)",
    "PA0330000 - JEFFERSON COUNTY SHERIFFS",
    "PA0330100 - BROCKWAY BOROUGH",
    "PA0330200 - BROOKVILLE BOROUGH",
    "PA0330300 - PUNXSUTAWNEY BOROUGH",
    "PA0330400 - REYNOLDSVILLE BOROUGH",
    "PA0330500 - (INACTIVE) BIG RUN BOROUGH (COVERED BY PSP - TROOP C - PUNXSUTAWNEY STATION - PAPSPC170X)",
    "PA0330600 - FALLS CREEK BOROUGH (COVERED BY PSP - TROOP C - DUBOIS STATION - PASPC340X)",
    "PA0330700 - SYKESVILLE BOROUGH",
    "PA0331200 - SUMMERVILLE BOROUGH (COVERED BY PSP - TROOP C - PUNXSUTAWNEY STATION - PAPSP7000)",
    "PA0331300 - DCNR - CLEAR CREEK STATE PARK",
    "PA0331400 - INDIANA UNIVERSITY OF PENNSYLVANIA - PUNXSUTAWNEY CAMPUS",
    "PA033BF0X - DCNR - BUREAU OF FORESTRY (JEFFERSON COUNTY)",
    "PA033DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (JEFFERSON COUNTY)",
    "PA0340000 - JUNIATA COUNTY SHERIFF",
    "PA0340400 - MIFFLIN BOROUGH",
    "PA0340500 - MIFFLINTOWN BOROUGH (COVERED BY PSP - TROOP G - LEWISTOWN STATION - PAPSP4900)",
    "PA0340700 - PORT ROYAL BOROUGH (COVERED BY PSP - TROOP G - LEWISTOWN STATION - PAPSP4900)",
    "PA034BF0X - DCNR - BUREAU OF FORESTRY (JUNIATA COUNTY) ",
    "PA034DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (JUNIATA COUNTY)",
    "PA0350100 - CARBONDALE CITY",
    "PA0350200 - DUNMORE BOROUGH",
    "PA0350300 - OLD FORGE BOROUGH",
    "PA0350400 - SCRANTON CITY",
    "PA0350500 - ARCHBALD BOROUGH",
    "PA0350600 - BLAKELY BOROUGH",
    "PA0350700 - CLARKS SUMMIT BOROUGH",
    "PA0350800 - DICKSON CITY BOROUGH",
    "PA0350900 - JERMYN BOROUGH",
    "PA0351100 - OLYPHANT BOROUGH",
    "PA0351200 - TAYLOR BOROUGH",
    "PA0351300 - THROOP BOROUGH",
    "PA0351500 - DALTON BOROUGH",
    "PA0351600 - JESSUP BOROUGH",
    "PA0351700 - MAYFIELD BOROUGH",
    "PA0351800 - MOSCOW BOROUGH",
    "PA0351900 - SOUTH ABINGTON TOWNSHIP",
    "PA0352000 - VANDLING BOROUGH (COVERED BY CARBONDALE CITY - PA0350100)",
    "PA0352100 - DCNR - LACKAWANNA STATE PARK",
    "PA0352300 - MOOSIC BOROUGH",
    "PA0352400 - WAVERLY TOWNSHIP",
    "PA0352600 - NEWTON TOWNSHIP (COVERED BY SOUTH ABINGTON TOWNSHIP - PA0351900)",
    "PA0352700 - SCOTT TOWNSHIP (LACKAWANNA COUNTY)",
    "PA0352800 - LACKAWANNA COUNTY DETECTIVES",
    "PA0353000 - GREENFIELD TOWNSHIP (LACKAWANNA COUNTY) ",
    "PA0353100 - ROARING BROOK TOWNSHIP",
    "PA0353300 - COVINGTON TOWNSHIP",
    "PA0353500 - CARBONDALE TOWNSHIP",
    "PA0353600 - JEFFERSON TOWNSHIP (LACKAWANNA COUNTY)",
    "PA0353700 - PSU - SCRANTON CAMPUS",
    "PA035389E - NORFOLK SOUTHERN - LACKAWANNA COUNTY",
    "PA035399E - UNIVERSITY OF SCRANTON",
    "PA0354100 - FOREST RANGERS PINCHOT STATE FOREST DISTRICT 11",
    "PA035BF0X - DCNR - BUREAU OF FORESTRY (LACKAWANNA COUNTY)",
    "PA035DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (LACKAWANNA COUNTY)",
    "PA0360000 - LANCASTER COUNTY SHERIFFS",
    "PA0360100 - COLUMBIA BOROUGH",
    "PA0360200 - ELIZABETHTOWN BOROUGH",
    "PA0360300 - EPHRATA BOROUGH",
    "PA0360400 - EPHRATA TOWNSHIP (COVERED BY EPHRATA BOROUGH - PA0360300)",
    "PA0360500 - LANCASTER CITY",
    "PA0360600 - LITITZ BOROUGH",
    "PA0360700 - MANHEIM BOROUGH",
    "PA0360800 - MANHEIM TOWNSHIP",
    "PA0360900 - (INACTIVE) MARIETTA BOROUGH (COVERED BY SUSQUEHANNA REGIONAL - PA0363800)",
    "PA0361000 - MOUNT JOY BOROUGH",
    "PA0361100 - NEW HOLLAND BOROUGH",
    "PA0361200 - LANCASTER TOWNSHIP (LANCASTER COUNTY) (COVERED BY MANHEIM TOWNSHIP - PA0360800)",
    "PA0361300 - MILLERSVILLE BOROUGH",
    "PA0361400 - WEST LAMPETER TOWNSHIP",
    "PA0361500 - (INACTIVE) ADAMSTOWN BOROUGH (COVERED BY EPHRATA BOROUGH - PA0360300)",
    "PA0361600 - (INACTIVE) AKRON BOROUGH (COVERED BY WEST EARL TOWNSHIP - PA0364800)",
    "PA0361800 - CHRISTIANA BOROUGH",
    "PA0362000 - DENVER BOROUGH (COVERED BY EAST COCALICO TOWNSHIP - PA0363700)",
    "PA0362100 - EARL TOWNSHIP (COVERED BY NEW HOLLAND BOROUGH - PA0361100",
    "PA0362300 - EAST HEMPFIELD TOWNSHIP",
    "PA0362400 - EAST PETERSBURG BOROUGH (COVERED BY NORTHERN LANCASTER COUNTY REGIONAL - PA0365800)",
    "PA0362500 - MANOR TOWNSHIP (LANCASTER COUNTY)",
    "PA0362600 - NORTHWEST LANCASTER COUNTY REGIONAL",
    "PA0362700 - (INACTIVE) PEQUEA TOWNSHIP",
    "PA0362900 - QUARRYVILLE BOROUGH",
    "PA0363000 - STRASBURG BOROUGH",
    "PA0363200 - WARWICK TOWNSHIP (LANCASTER COUNTY) (COVERED BY NORTHERN LANCASTER CO. REG. - PA0365800)",
    "PA0363400 - WEST HEMPFIELD TOWNSHIP",
    "PA0363500 - PEQUEA TOWNSHIP (LANCASTER COUNTY)",
    "PA0363600 - (INACTIVE) CONOY TOWNSHIP (COVERED BY SUSQUEHANNA REGIONAL (LANCASTER COUNTY) PA0363800)",
    "PA0363700 - EAST COCALICO TOWNSHIP",
    "PA0363800 - SUSQUEHANNA REGIONAL",
    "PA0363900 - EAST EARL TOWNSHIP",
    "PA0364000 - EAST LAMPETER TOWNSHIP",
    "PA0364100 - PENN TOWNSHIP (LANCASTER COUNTY) (COVERED BY NORTHERN LANCASTER COUNTY REG. - PA0365800)",
    "PA0364200 - UPPER LEACOCK TOWNSHIP (COVERED BY EAST LAMPETER TOWNSHIP - PA0364000)",
    "PA0364300 - WEST COCALICO TOWNSHIP (COVERED BY EPHRATA BOROUGH - PA0360300)",
    "PA0364400 - WEST DONEGAL TOWNSHIP (COVERED BY NORTHWEST LANCASTER COUNTY REGIONAL - PA0362600)",
    "PA0364700 - MOUNTVILLE BOROUGH (COVERED BY WEST HEMPFIELD TOWNSHIP - PA0363400)",
    "PA0364800 - WEST EARL TOWNSHIP",
    "PA0365100 - MILLERSVILLE UNIVERSITY",
    "PA0365200 - (INACTIVE) CLAY TOWNSHIP (COVERED BY NORTHERN LANCASTER COUNTY REGIONAL - PA0365800)",
    "PA0365300 - (INACTIVE) BRECKNOCK TOWNSHIP (LANCASTER COUNTY) (COVERED BY PSP - TROOP J - LANCASTER - PAPSP4700)",
    "PA0365800 - NORTHERN LANCASTER COUNTY REGIONAL",
    "PA0365900 - DCNR - SUSQUEHANNOCK STATE PARK",
    "PA0366200 - LANCASTER TOWNSHIP POLICE DEPARTMENT",
    "PA036BF0X - DCNR - BUREAU OF FORESTRY (LANCASTER COUNTY)",
    "PA036DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (LANCASTER COUNTY)",
    "PA0370000 - LAWRENCE COUNTY SHERIFFS",
    "PA0370100 - ELLWOOD CITY BOROUGH",
    "PA0370200 - NEW CASTLE CITY",
    "PA0370300 - SHENANGO TOWNSHIP (LAWRENCE COUNTY)",
    "PA0370400 - BESSEMER BOROUGH",
    "PA0370500 - NESHANNOCK TOWNSHIP",
    "PA0370600 - NEW WILMINGTON BOROUGH",
    "PA0370700 - NORTH BEAVER TOWNSHIP",
    "PA0370800 - UNION TOWNSHIP (LAWRENCE COUNTY)",
    "PA0370900 - PERRY TOWNSHIP (LAWRENCE COUNTY) (COVERED BY PSP - TROOP D - NEW CASTLE - PAPSP6200)",
    "PA0371000 - WAYNE TOWNSHIP (LAWRENCE COUNTY)",
    "PA0371200 - LAWRENCE COUNTY DETECTIVES",
    "PA0371300 - SOUTH NEW CASTLE BOROUGH (COVERED BY SHENANGO TOWNSHIP - PA0370300)",
    "PA0371400 - PULASKI TOWNSHIP",
    "PA0371700 - WAMPUM BOROUGH",
    "PA0371800 - NEW BEAVER BOROUGH",
    "PA0371900 - (INACTIVE) HICKORY TOWNSHIP (COVERED BY PSP - TROOP D - NEW CASTLE - PAPSP6200)",
    "PA0372000 - MAHONING TOWNSHIP (LAWRENCE COUNTY)",
    "PA0372100 - LITTLE BEAVER TOWNSHIP",
    "PA037239E - (INACTIVE) WESTMINSTER COLLEGE (ORI RETIRED)",
    "PA037BF0X - DCNR - BUREAU OF FORESTRY (LAWRENCE COUNTY) ",
    "PA037DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (LAWRENCE COUNTY)",
    "PA0380100 - (INACTIVE) ANNVILLE TOWNSHIP (COVERED BY WESTERN LEBANON COUNTY REGIONAL - PA0383300)",
    "PA0380200 - LEBANON CITY",
    "PA0380300 - (INACTIVE) PALMYRA BOROUGH (COVERED BY WESTERN LEBANON COUNTY REGIONAL - PA0383300)",
    "PA0380400 - RICHLAND BOROUGH (COVERED BY PSP - TROOP L - JONESTOWN STATION - PAPSP4300)",
    "PA0380500 - MYERSTOWN BOROUGH (COVERED BY PSP - TROOP L - JONESTOWN STATION - PAPSP4300)",
    "PA0380600 - SOUTH LEBANON TOWNSHIP",
    "PA0380700 - (INACTIVE) BETHEL TOWNSHIP (LEBANON COUNTY) (COVERED BY PSP - TROOP L - JONESTOWN - PAPSP4300)",
    "PA0380800 - DCNR - MEMORIAL LAKE STATE PARK",
    "PA0380900 - CORNWALL BOROUGH",
    "PA0381000 - HEIDELBERG TOWNSHIP (LEBANON COUNTY) (COVERED BY PSP - TROOP L - JONESTOWN - PAPSP4300)",
    "PA0381100 - MOUNT GRETNA BOROUGH (COVERED BY CORNWALL BOROUGH - PA0380900)",
    "PA0381200 - (INACTIVE) NORTH CORNWALL TOWNSHIP (COVERED BY LEBANON COUNTY REGIONAL - PA0384100)",
    "PA0381300 - NORTH LEBANON TOWNSHIP (COVERED BY LEBANON COUNTY REGIONAL - PA0384100)",
    "PA0381400 - CLEONA BOROUGH POLICE DEPARTMENT",
    "PA0381500 - NORTH ANNVILLE TOWNSHIP (COVERED BY CLEONA BOROUGH - PA0381400)",
    "PA0381600 - NORTH LONDONDERRY TOWNSHIP",
    "PA0381700 - SOUTH ANNVILLE TOWNSHIP",
    "PA0381900 - WEST CORNWALL TOWNSHIP (COVERED BY CORWALL BOROUGH - PA0380900)",
    "PA0382100 - LEBANON COUNTY DETECTIVES",
    "PA0382200 - SOUTH LONDONDERRY TOWNSHIP",
    "PA0382300 - WEST LEBANON TOWNSHIP (COVERED BY LEBANON COUNTY REGIONAL - PA0384100)",
    "PA0382400 - MILLCREEK TOWNSHIP (LEBANON COUNTY)",
    "PA0382500 - FORT INDIANTOWN GAP POLICE",
    "PA0382800 - LEBANON VALLEY COLLEGE",
    "PA0383300 - WESTERN LEBANON COUNTY REGIONAL POLICE DEPARTMENT",
    "PA0384100 - LEBANON COUNTY REGIONAL POLICE DEPARTMENT ",
    "PA038BF0X - DCNR - BUREAU OF FORESTRY (LEBANON COUNTY) ",
    "PA038DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (LEBANON COUNTY)",
    "PA0390000 - LEHIGH COUNTY SHERIFFS",
    "PA0390100 - ALLENTOWN CITY",
    "PA0390200 - CATASAUQUA BOROUGH",
    "PA0390300 - COOPERSBURG BOROUGH",
    "PA0390400 - EMMAUS BOROUGH",
    "PA0390500 - FOUNTAIN HILL BOROUGH",
    "PA0390600 - SALISBURY TOWNSHIP",
    "PA0390700 - SLATINGTON BOROUGH",
    "PA0390800 - WHITEHALL TOWNSHIP",
    "PA0390900 - COPLAY BOROUGH",
    "PA0391100 - UPPER SAUCON TOWNSHIP",
    "PA0391200 - MACUNGIE BOROUGH",
    "PA0391300 - SOUTH WHITEHALL TOWNSHIP",
    "PA0391400 - ALBURTIS BOROUGH",
    "PA0391500 - LOWER MILFORD TOWNSHIP (COVERED BY PSP - TROOP M - FOGELSVILLE STATION - PAPSP2900)",
    "PA0391900 - (INACTIVE) LEHIGH COUNTY DETECTIVES",
    "PA0392100 - LEHIGH NORTHAMPTON AIRPORT AUTHORITY",
    "PA039249E - DESALES UNIVERSITY",
    "PA0392700 - UPPER MACUNGIE TOWNSHIP",
    "PA0392900 - LEHIGH CAREER AND TECHINCAL INSTITUTE",
    "PA0393100 - PSU - LEHIGH VALLEY CAMPUS",
    "PA039BF0X - DCNR - BUREAU OF FORESTRY (LEHIGH COUNTY) ",
    "PA039DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (LEHIGH COUNTY)",
    "PA039PP01 - CEDAR CREST COLLEGE",
    "PA0400100 - DALLAS BOROUGH",
    "PA0400200 - DUPONT BOROUGH",
    "PA0400300 - DURYEA BOROUGH",
    "PA0400400 - EDWARDSVILLE BOROUGH",
    "PA0400500 - FREELAND BOROUGH",
    "PA0400600 - HANOVER TOWNSHIP (LUZERNE COUNTY)",
    "PA0400700 - HAZLETON CITY",
    "PA0400800 - KINGSTON BOROUGH",
    "PA0400900 - LUZERNE BOROUGH",
    "PA0401000 - NANTICOKE CITY",
    "PA0401100 - PITTSTON CITY",
    "PA0401200 - PLYMOUTH BOROUGH",
    "PA0401300 - WILKES-BARRE CITY",
    "PA0401400 - (INACTIVE) WYOMING BOROUGH (LUZERNE COUNTY) (COVERED BY WYOMING AREA REGIONAL - PA0408200)",
    "PA0401500 - ASHLEY BOROUGH",
    "PA0401600 - AVOCA BOROUGH",
    "PA0401700 - (INACTIVE) EXETER BOROUGH (LUZERNE COUNTY) (COVERED BY WYOMING AREA REGIONAL - PA0408200)",
    "PA0401800 - FORTY FORT BOROUGH",
    "PA0401900 - LARKSVILLE BOROUGH",
    "PA0402000 - NEWPORT TOWNSHIP (LUZERNE COUNTY)",
    "PA0402100 - PLAINS TOWNSHIP",
    "PA0402200 - SUGAR NOTCH BOROUGH",
    "PA0402300 - SWOYERSVILLE BOROUGH",
    "PA0402400 - WEST HAZLETON BOROUGH",
    "PA0402500 - (INACTIVE) WEST PITTSTON BOROUGH (LUZERNE COUNTY) (COVERED BY WYOMING AREA REGIONAL - PA0408200)",
    "PA0402600 - (INACTIVE) WEST WYOMING BOROUGH (LUZERNE COUNTY) (COVERED BY WYOMING AREA REGIONAL - PA0408200)",
    "PA0402700 - WILKES-BARRE TOWNSHIP",
    "PA0402900 - CONYNGHAM BOROUGH",
    "PA0403000 - DCNR - RICKETTS GLEN STATE PARK",
    "PA0403100 - FAIRVIEW TOWNSHIP (LUZERNE COUNTY)",
    "PA0403200 - LAKE TOWNSHIP (COVERED BY PSP - TROOP P - WILKES-BARRE STATION - PAPSP9300)",
    "PA0403300 - LEHMAN TOWNSHIP POLICE DEPARTMENT",
    "PA0403400 - NESCOPECK BOROUGH",
    "PA0403500 - PLYMOUTH TOWNSHIP (LUZERNE COUNTY) (COVERED BY PSP - TROOP P - WILKES-BARRE - PAPSP9300)",
    "PA0403700 - SHICKSHINNY BOROUGH (COVERED BY PSP - TROOP P - SHICKSHINNY STATION - PAPSP7900)",
    "PA0403800 - WHITE HAVEN BOROUGH",
    "PA0403900 - WRIGHT TOWNSHIP",
    "PA0404000 - HARVEYS LAKE BOROUGH",
    "PA0404100 - JACKSON TOWNSHIP (LUZERNE COUNTY)",
    "PA0404300 - PRINGLE BOROUGH (COVERED BY KINGSTON BOROUGH - PA0400800)",
    "PA0404400 - (INACTIVE) EXETER TOWNSHIP (LUZERNE COUNTY) (COVERED BY WYOMING AREA REGIONAL - PA0408200)",
    "PA0404500 - JENKINS TOWNSHIP",
    "PA0404700 - SALEM TOWNSHIP",
    "PA0404900 - PITTSTON TOWNSHIP",
    "PA0405000 - DALLAS TOWNSHIP",
    "PA0405100 - KINGSTON TOWNSHIP",
    "PA0405500 - LUZERNE COUNTY DETECTIVES",
    "PA0405600 - BUTLER TOWNSHIP (LUZERNE COUNTY)",
    "PA0405700 - RICE TOWNSHIP",
    "PA0406000 - SUGARLOAF TOWNSHIP",
    "PA0406100 - COURTDALE BOROUGH",
    "PA0406200 - LAFLIN BOROUGH (COVERED BY PSP - TROOP P - WILKES-BARRE STATION - PAPSP9300)",
    "PA0406500 - HUGHESTOWN BOROUGH",
    "PA0406700 - NUANGOLA BOROUGH (COVERED BY RICE TOWNSHIP - PA0405700)",
    "PA0407100 - PSU - HAZLETON CAMPUS",
    "PA0407200 - PSU - WILKES-BARRE CAMPUS",
    "PA0407500 - DCNR - FRANCES SLOCUM STATE PARK",
    "PA0407800 - (DELETE)",
    "PA0408200 - WYOMING AREA REGIONAL POLICE DEPARTMENT ",
    "PA040859E - WILKES UNIVERSITY",
    "PA0408900 - (INACTIVE) WILKES-BARRE AREA CAREER TECHNICAL CENTER POLICE DEPARTMENT",
    "PA040BF0X - DCNR - BUREAU OF FORESTRY (LUZERNE COUNTY) ",
    "PA040DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (LUZERNE COUNTY)",
    "PA0410000 - LYCOMING COUNTY SHERIFFS",
    "PA0410100 - (INACTIVE) TIADAGHTON VALLEY REGIONAL (COVERED BY LYCOMING REGIONAL POLICE DEPARTMENT - PA0412800)",
    "PA0410200 - MONTOURSVILLE BOROUGH",
    "PA0410300 - MUNCY BOROUGH",
    "PA0410400 - SOUTH WILLIAMSPORT BOROUGH",
    "PA0410500 - WILLIAMSPORT CITY",
    "PA0410700 - PENNSYLVANIA COLLEGE OF TECHNOLOGY",
    "PA0410800 - HUGHESVILLE BOROUGH",
    "PA0410900 - (INACTIVE) MONTGOMERY BOROUGH (COVERED BY PSP - TROOP F - MONTOURSVILLE STATION - PAPSP6000)",
    "PA0411000 - DUBOISTOWN BOROUGH (COVERED BY SOUTH WILLIAMSPORT BOROUGH - PA0410400)",
    "PA0411500 - (INACTIVE) OLD LYCOMING TOWNSHIP (COVERED BY LYCOMING REGIONAL POLICE DEPARTMENT - PA0412800)",
    "PA0411700 - MUNCY TOWNSHIP",
    "PA0411800 - PORTER TOWNSHIP (COVERED BY LYCOMING REGIONAL POLICE DEPARTMENT - PA0412800)",
    "PA0412100 - DCNR - LITTLE PINE STATE PARK",
    "PA0412200 - FOREST RANGERS TIADAGHTON STATE FOREST DISTRICT 12",
    "PA0412800 - LYCOMING REGIONAL POLICE DEPARTMENT",
    "PA041BF0X - DCNR - BUREAU OF FORESTRY (LYCOMING COUNTY)",
    "PA041DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (LYCOMING COUNTY)",
    "PA0420100 - BRADFORD CITY",
    "PA0420200 - KANE BOROUGH",
    "PA0420300 - PORT ALLEGANY BOROUGH",
    "PA0420400 - BRADFORD TOWNSHIP",
    "PA0420500 - FOSTER TOWNSHIP (MCKEAN COUNTY)",
    "PA0420600 - MOUNT JEWETT BOROUGH (COVERED BY PSP - TROOP C - LEWIS RUN STATION - PAPSP4400)",
    "PA0420700 - (INACTIVE) OTTO-ELDRED REGIONAL",
    "PA0420800 - SMETHPORT BOROUGH",
    "PA0421000 - OTTO TOWNSHIP POLICE DEPARTMENT",
    "PA0421100 - MCKEAN COUNTY DETECTIVES",
    "PA0421200 - LEWIS RUN BOROUGH (COVERED BY PSP - TROOP C - LEWIS RUN STATION - PAPSP4400)",
    "PA0421500 - UNIVERSITY OF PITTSBURGH - BRADFORD CAMPUS ",
    "PA0421700 - (DELETE)",
    "PA042BF0X - DCNR - BUREAU OF FORESTRY (MCKEAN COUNTY) ",
    "PA042DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (MCKEAN COUNTY)",
    "PA0430000 - MERCER COUNTY SHERIFFS OFFICE",
    "PA0430100 - (INACTIVE) SOUTHWEST MERCER COUNTY REGIONAL",
    "PA0430200 - GREENVILLE BOROUGH",
    "PA0430300 - GROVE CITY BOROUGH",
    "PA0430400 - HERMITAGE CITY",
    "PA0430500 - MERCER BOROUGH",
    "PA0430600 - SHARON CITY",
    "PA0430700 - SHARPSVILLE BOROUGH",
    "PA0430800 - PYMATUNING TOWNSHIP",
    "PA0430900 - HEMPFIELD TOWNSHIP (MERCER COUNTY)",
    "PA0431000 - SANDY LAKE BOROUGH",
    "PA0431100 - SHENANGO TOWNSHIP (MERCER COUNTY)",
    "PA0431200 - WHEATLAND BOROUGH (COVERED BY HERMITAGE CITY - PA0430400)",
    "PA0431300 - WEST MIDDLESEX BOROUGH (COVERED BY SHENANGO TOWNSHIP - PA0431100)",
    "PA0431400 - STONEBORO BOROUGH",
    "PA0431500 - JEFFERSON TOWNSHIP (MERCER COUNTY)",
    "PA0431700 - MERCER COUNTY DETECTIVE BUREAU",
    "PA0432100 - JAMESTOWN BOROUGH",
    "PA0432200 - (INACTIVE) WEST SALEM TOWNSHIP (COVERED BY PSP - TROOP D - MERCER STATION - PAPSP5700)",
    "PA0432400 - SOUTH PYMATUNING TOWNSHIP",
    "PA043259E - THIEL COLLEGE",
    "PA0432600 - DCNR - MAURICE K. GODDARD STATE PARK",
    "PA0432700 - PSU - SHENANGO VALLEY CAMPUS",
    "PA0432800 - FARRELL CITY",
    "PA043BF0X - DCNR - BUREAU OF FORESTRY (MERCER COUNTY) ",
    "PA043DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (MERCER COUNTY)",
    "PA0440100 - MIFFLIN COUNTY REGIONAL",
    "PA0440200 - (INACTIVE) BURNHAM BOROUGH (COVERED BY MIFFLIN COUNTY REGIONAL - PA0440100)",
    "PA0440300 - DERRY TOWNSHIP (MIFFLIN COUNTY) (COVERED BY MIFFLIN COUNTY REGIONAL - PA0440100)",
    "PA0440600 - ARMAGH TOWNSHIP",
    "PA0440700 - GRANVILLE TOWNSHIP",
    "PA0440900 - UNION TOWNSHIP (MIFFLIN COUNTY) (COVERED BY MIFFLIN COUNTY REGIONAL - PA0440100)",
    "PA0441100 - DCNR - REEDS GAP STATE PARK",
    "PA0441200 - LEWISTOWN BOROUGH",
    "PA044BF0X - DCNR - BUREAU OF FORESTRY (MIFFLIN COUNTY) ",
    "PA044DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (MIFFLIN COUNTY)",
    "PA0450000 - MONROE COUNTY SHERIFFS",
    "PA0450100 - DELAWARE WATER GAP BOROUGH",
    "PA045013A - (INACTIVE) MONROE COUNTY CRIMINAL INVESTIGATION DIVISION",
    "PA0450200 - EAST STROUDSBURG BOROUGH (COVERED BY STROUD AREA REGIONAL - PA0450800)",
    "PA0450300 - STROUDSBURG BOROUGH (COVERED BY STROUD AREA REGIONAL - PA0450800)",
    "PA0450400 - (INACTIVE) BARRETT TOWNSHIP (COVERED BY POCONO MOUNTAIN REGIONAL - PA0450900)",
    "PA0450500 - (INACTIVE) COOLBAUGH TOWNSHIP (COVERED BY POCONO MOUNTAIN REGIONAL - PA0450900)",
    "PA0450600 - MOUNT POCONO BOROUGH (COVERED BY POCONO MOUNTAIN REGIONAL - PA0450900)",
    "PA0450700 - POCONO TOWNSHIP",
    "PA0450800 - STROUD AREA REGIONAL",
    "PA0450900 - POCONO MOUNTAIN REGIONAL",
    "PA0451100 - EAST STROUDSBURG UNIVERSITY",
    "PA0451200 - DCNR - TOBYHANNA STATE PARK",
    "PA0451300 - TUNKHANNOCK TOWNSHIP (MONROE COUNTY) (COVERED BY POCONO MOUNTAIN REGIONAL - PA0450900)",
    "PA0451500 - FOREST RANGERS DELAWARE STATE FOREST DISTRICT 19",
    "PA045BF0X - DCNR - BUREAU OF FORESTRY (MONROE COUNTY) ",
    "PA045DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (MONROE COUNTY)",
    "PA0460000 - MONTGOMERY COUNTY SHERIFFS",
    "PA0460100 - ABINGTON TOWNSHIP",
    "PA0460200 - AMBLER BOROUGH",
    "PA0460300 - BRIDGEPORT BOROUGH",
    "PA0460400 - BRYN ATHYN BOROUGH",
    "PA0460500 - CHELTENHAM TOWNSHIP",
    "PA0460600 - COLLEGEVILLE BOROUGH",
    "PA0460700 - CONSHOHOCKEN BOROUGH",
    "PA0460800 - EAST NORRITON TOWNSHIP",
    "PA0460900 - HATBORO BOROUGH",
    "PA0461000 - HORSHAM TOWNSHIP",
    "PA0461100 - JENKINTOWN BOROUGH",
    "PA0461200 - LANSDALE BOROUGH",
    "PA0461300 - LOWER GWYNEDD TOWNSHIP",
    "PA0461400 - LOWER MERION TOWNSHIP",
    "PA0461500 - LOWER MORELAND TOWNSHIP",
    "PA0461600 - LOWER POTTSGROVE TOWNSHIP",
    "PA0461700 - LOWER PROVIDENCE TOWNSHIP",
    "PA0461800 - MONTGOMERY TOWNSHIP",
    "PA0461900 - NORRISTOWN BOROUGH",
    "PA0462000 - NORTH WALES BOROUGH",
    "PA0462100 - PLYMOUTH TOWNSHIP (MONTGOMERY COUNTY)",
    "PA0462200 - POTTSTOWN BOROUGH",
    "PA0462300 - SPRINGFIELD TOWNSHIP (MONTGOMERY COUNTY) ",
    "PA0462400 - ROCKLEDGE BOROUGH",
    "PA0462500 - ROYERSFORD BOROUGH",
    "PA0462600 - SOUDERTON BOROUGH",
    "PA0462700 - UPPER DUBLIN TOWNSHIP",
    "PA0462800 - UPPER GWYNEDD TOWNSHIP",
    "PA0462900 - UPPER MERION TOWNSHIP",
    "PA0463000 - UPPER MORELAND TOWNSHIP",
    "PA0463100 - WEST NORRITON TOWNSHIP",
    "PA0463200 - WHITEMARSH TOWNSHIP",
    "PA0463300 - WHITPAIN TOWNSHIP",
    "PA0463500 - NARBERTH BOROUGH",
    "PA0463600 - WEST CONSHOHOCKEN BOROUGH",
    "PA0463700 - TELFORD BOROUGH",
    "PA0463800 - UPPER PROVIDENCE TOWNSHIP (MONTGOMERY COUNTY)",
    "PA0463900 - DCNR - EVANSBURG STATE PARK",
    "PA0464100 - HATFIELD TOWNSHIP",
    "PA0464200 - LOWER FREDERICK TOWNSHIP",
    "PA0464300 - LOWER SALFORD TOWNSHIP",
    "PA0464500 - SCHWENKSVILLE BOROUGH (COVERED BY PSP - TROOP K - SKIPPACK STATION - PAPSP7600)",
    "PA0464600 - UPPER POTTSGROVE TOWNSHIP",
    "PA0464800 - WEST POTTSGROVE TOWNSHIP",
    "PA0464900 - DOUGLASS TOWNSHIP (MONTGOMERY COUNTY)",
    "PA0465000 - PROD TESTING AGENCY",
    "PA0465100 - FRANCONIA TOWNSHIP",
    "PA0465200 - MONTGOMERY COUNTY DETECTIVES",
    "PA0465300 - LIMERICK TOWNSHIP",
    "PA0465400 - NEW HANOVER TOWNSHIP",
    "PA0465600 - DCNR - FORT WASHINGTON STATE PARK",
    "PA0465700 - MARLBOROUGH TOWNSHIP",
    "PA0465900 - TOWAMENCIN TOWNSHIP",
    "PA0466000 - UPPER PERK",
    "PA0466200 - PSU - ABINGTON CAMPUS",
    "PA0466900 - EAST GREENVILLE BOROUGH",
    "PA046BF0X - DCNR - BUREAU OF FORESTRY (MONTGOMERY COUNTY)",
    "PA046DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (MONTGOMERY COUNTY)",
    "PA0470100 - DANVILLE BOROUGH",
    "PA0470200 - MAHONING TOWNSHIP (MONTOUR COUNTY)",
    "PA0470300 - MONTOUR COUNTY DETECTIVES",
    "PA047BF0X - DCNR - BUREAU OF FORESTRY (MONTOUR COUNTY) ",
    "PA047DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (MONTOUR COUNTY)",
    "PA0480000 - NORTHAMPTON COUNTY SHERIFFS",
    "PA0480100 - BANGOR BOROUGH",
    "PA0480200 - COLONIAL REGIONAL",
    "PA0480300 - BETHLEHEM CITY",
    "PA0480400 - EASTON CITY",
    "PA0480500 - HELLERTOWN BOROUGH",
    "PA0480600 - LEHIGH TOWNSHIP (NORTHAMPTON COUNTY)",
    "PA0480700 - NAZARETH BOROUGH",
    "PA0480800 - NORTHAMPTON BOROUGH",
    "PA0480900 - PALMER TOWNSHIP",
    "PA0481000 - WILSON BOROUGH",
    "PA0481100 - PEN ARGYL BOROUGH (COVERED BY SLATE BELT REGIONAL - PA0484800)",
    "PA0481200 - BETHLEHEM TOWNSHIP",
    "PA0481300 - BUSHKILL TOWNSHIP",
    "PA0481400 - EAST BANGOR BOROUGH",
    "PA0481500 - FORKS TOWNSHIP",
    "PA0481600 - LOWER MOUNT BETHEL TOWNSHIP (COVERED BY PSP TROOP M - BELFAST STATION - PAPSP2100)",
    "PA0481700 - LOWER SAUCON TOWNSHIP",
    "PA0481800 - MOORE TOWNSHIP",
    "PA0481900 - NORTH CATASAUQUA BOROUGH",
    "PA0482000 - PLAINSFIELD TOWNSHIP (COVERED BY SLATE BELT REGIONAL - PA0484800)",
    "PA0482100 - PORTLAND BOROUGH",
    "PA0482200 - ROSETO BOROUGH",
    "PA0482300 - TATAMY BOROUGH",
    "PA0482500 - WASHINGTON TOWNSHIP (NORTHAMPTON COUNTY) ",
    "PA0482600 - WIND GAP BOROUGH (COVERED BY SLATE BELT REGIONAL - PA0484800)",
    "PA0482800 - WALNUTPORT BOROUGH",
    "PA0483000 - NORTHAMPTON COUNTY DETECTIVES",
    "PA0483100 - FREEMANSBURG BOROUGH",
    "PA0483200 - UPPER NAZARETH TOWNSHIP",
    "PA0483400 - (INACTIVE) STOCKERTOWN BOROUGH (COVERED BY PSP - TROOP M - BELFAST STATION - PAPSP2100)",
    "PA048359E - LEHIGH UNIVERSITY",
    "PA048369E - MORAVIAN COLLEGE",
    "PA0483800 - DCNR - JACOBSBURG ENVIRONMENTAL EDUCATION CENTER",
    "PA048439E - LAFAYETTE COLLEGE",
    "PA0484800 - SLATE BELT REGIONAL",
    "PA0484900 - WEST EASTON BOROUGH",
    "PA048BF0X - DCNR - BUREAU OF FORESTRY (NORTHAMPTON COUNTY)",
    "PA048DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (NORTHAMPTON COUNTY)",
    "PA0490000 - NORTHUMBERLAND COUNTY SHERIFFS",
    "PA0490100 - MILTON BOROUGH",
    "PA0490200 - MOUNT CARMEL BOROUGH",
    "PA0490300 - NORTHUMBERLAND BOROUGH",
    "PA0490400 - SHAMOKIN CITY",
    "PA0490500 - SUNBURY CITY",
    "PA0490600 - COAL TOWNSHIP",
    "PA0490700 - KULPMONT BOROUGH",
    "PA0490900 - ZERBE TOWNSHIP",
    "PA0491200 - RALPHO TOWNSHIP",
    "PA0491300 - RIVERSIDE BOROUGH",
    "PA0491400 - WATSONTOWN BOROUGH",
    "PA0491600 - POINT TOWNSHIP",
    "PA0491700 - (INACTIVE) NORTHUMBERLAND COUNTY DETECTIVES",
    "PA0491800 - MOUNT CARMEL TOWNSHIP",
    "PA0491900 - (INACTIVE) LOWER MAHANOY TOWNSHIP (COVERED BY PSP - TROOP F - STONINGTON PAPSP7800)",
    "PA0492100 - DCNR - SHIKELLAMY STATE PARK",
    "PA049BF0X - DCNR - BUREAU OF FORESTRY (NORTHUMBERLAND COUNTY)",
    "PA049DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (NORTHUMBERLAND COUNTY)",
    "PA0500100 - MARYSVILLE BOROUGH",
    "PA0500200 - DUNCANNON BOROUGH (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA0500300 - NEWPORT BOROUGH (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA05004VS - RYE TOWNSHIP (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA0500900 - (INACTIVE) BLOOMFIELD BOROUGH (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA0501100 - PENN TOWNSHIP (PERRY COUNTY) (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA0501400 - OLIVER TOWNSHIP (COVERED BY PSP - TROOP H - NEWPORT STATION - PAPSP1800)",
    "PA0501500 - DCNR - LITTLE BUFFALO STATE PARK",
    "PA0501600 - FOREST RANGERS TUSCARORA STATE FOREST DISTRICT 3",
    "PA050BF0X - DCNR - BUREAU OF FORESTRY (PERRY COUNTY) ",
    "PA050DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (PERRY COUNTY)",
    "PA0510000 - (INACTIVE) PHILADELPHIA COUNTY SHERIFFS (COVERED BY - PHILADELPHIA CITY - PAPEP0000)",
    "PA0510400 - UNIVERSITY OF PENNSYLVANIA",
    "PA0510600 - TEMPLE UNIVERSITY - MAIN CAMPUS",
    "PA0510700 - (INACTIVE) SEPTA TRANSIT (COVERED BY PHILADELPHIA CITY - PAPEP0000)",
    "PA0511300 - (INACTIVE) PHILADELPHIA HOUSING AUTHORITY ",
    "PA051189E - DREXEL UNIVERSITY - UNIVERSITY CITY CAMPUS ",
    "PA0512600 - DCNR - BENJAMIN RUSH STATE PARK",
    "PA051AG00 - (INACTIVE) PHILADELPHIA COUNTY ATTORNEY GENERALS OFFICE",
    "PA051BF0X - DCNR - BUREAU OF FORESTRY (PHILADELPHIA COUNTY)",
    "PA051DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (PHILADELPHIA COUNTY)",
    "PA0520000 - PIKE COUNTY SHERIFFS",
    "PA0520100 - MATAMORAS BOROUGH (COVERED BY EASTERN PIKE REGIONAL - PA0520400)",
    "PA0520200 - MILFORD BOROUGH",
    "PA0520300 - PIKE COUNTY DETECTIVES",
    "PA0520400 - EASTERN PIKE REGIONAL",
    "PA0520600 - SHOHOLA TOWNSHIP (COVERED BY PSP - TROOP R - BLOOMING GROVE STATION - PAPSP5800)",
    "PA0520700 - DCNR - PROMISED LAND STATE PARK",
    "PA052BF0X - DCNR - BUREAU OF FORESTRY (PIKE COUNTY) ",
    "PA052DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (PIKE COUNTY)",
    "PA0530100 - COUDERSPORT BOROUGH",
    "PA0530200 - GALETON BOROUGH",
    "PA0530400 - SHINGLEHOUSE BOROUGH",
    "PA0530600 - (INACTIVE) AUSTIN BOROUGH (COVERED BY PSP - TROOP F - COUDERSPORT STATION - PAPSP1500)",
    "PA0531300 - (INACTIVE) SWEDEN TOWNSHIP (COVERED BY PSP - TROOP F - COUDERSPORT STATION - PAPSP1500)",
    "PA0531400 - DCNR - OLE BULL STATE PARK",
    "PA0531500 - DCNR - LYMAN RUN STATE PARK",
    "PA0531600 - FOREST RANGERS SUSQUEHANNOCK STATE FOREST DISTRICT 15",
    "PA053BF0X - DCNR - BUREAU OF FORESTRY (POTTER COUNTY) ",
    "PA053DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (POTTER COUNTY)",
    "PA0540000 - SCHUYLKILL COUNTY SHERIFFS",
    "PA054019E - READING NORTHERN RAILROAD POLICE (BERKS) ",
    "PA0540200 - FRACKVILLE BOROUGH",
    "PA0540300 - (INACTIVE) GIRARDVILLE BOROUGH (COVERED BY PSP - TROOP L - FRACKVILLE STATION - PAPSP5200)",
    "PA0540400 - MCADOO BOROUGH",
    "PA0540500 - MAHANOY CITY BOROUGH",
    "PA0540600 - MINERSVILLE BOROUGH",
    "PA0540700 - POTTSVILLE CITY",
    "PA0540800 - SAINT CLAIR BOROUGH",
    "PA0540900 - SCHUYLKILL HAVEN BOROUGH",
    "PA0541000 - SHENANDOAH BOROUGH",
    "PA0541100 - TAMAQUA BOROUGH",
    "PA0541200 - ASHLAND BOROUGH",
    "PA0541300 - (INACTIVE) COALDALE BOROUGH (COVERED BY PSP - TROOP L - FRACKVILLE STATION - PAPSP5200)",
    "PA0541400 - GILBERTON BOROUGH (COVERED BY FRACKVILLE BOROUGH - PA0540200)",
    "PA0541500 - HEGINS TOWNSHIP",
    "PA0541600 - (INACTIVE) NEW PHILADELPHIA BOROUGH (COVERED BY BLYTHE TOWNSHIP - PA0544900)",
    "PA0541700 - PORT CARBON BOROUGH",
    "PA0541800 - DELANO TOWNSHIP (COVERED BY RYAN TOWNSHIP - PA0543400)",
    "PA0542000 - MAHANOY TOWNSHIP",
    "PA0542100 - MECHANICSVILLE BOROUGH (COVERED BY PORT CARBON BOROUGH - PA0541700)",
    "PA0542200 - ORWIGSBURG BOROUGH",
    "PA0542300 - PALO ALTO BOROUGH (COVERED BY PORT CARBON BOROUGH - PA0541700)",
    "PA0542400 - PINE GROVE BOROUGH",
    "PA0542600 - RINGTOWN BOROUGH",
    "PA0542700 - SCHUYLKILL TOWNSHIP -SCHUYLKILL COUNTY ( COVERED BY PSP - TROOP L - FRACKVILLE STATION - PAPSP5200)",
    "PA0542800 - TOWER CITY BOROUGH",
    "PA0542900 - (INACTIVE) TREMONT BOROUGH (COVERED BY PSP - TROOP L - SCHUYLKILL HAVEN STATION - PAPSP7500)",
    "PA0543000 - UNION TOWNSHIP (SCHUYLKILL COUNTY)",
    "PA0543100 - BUTLER TOWNSHIP (SCHUYLKILL COUNTY)",
    "PA0543200 - (INACTIVE) CASS TOWNSHIP (COVERED BY PSP - TROOP L - FRACKVILLE STATION)",
    "PA0543400 - RYAN TOWNSHIP",
    "PA0543500 - SCHUYLKILL COUNTY DETECTIVES",
    "PA0543600 - RUSH TOWNSHIP",
    "PA0543700 - WEST MAHANOY TOWNSHIP",
    "PA0543800 - KLINE TOWNSHIP",
    "PA0543900 - (INACTIVE) AUBURN BOROUGH (COVERED BY PSP - TROOP L - SCHUYLKILL HAVEN - PAPSP7500)",
    "PA0544100 - NORWEGIAN TOWNSHIP (COVERED BY PSP - TROOP L - SCHUYLKILL HAVEN STATION - PAPSP7500)",
    "PA0544200 - NORTH UNION TOWNSHIP",
    "PA0544300 - WEST PENN TOWNSHIP",
    "PA0544500 - WALKER TOWNSHIP",
    "PA0544700 - BRANCH TOWNSHIP",
    "PA0544800 - (INACTIVE) NEW CASTLE TOWNSHIP (COVERED BY PSP - TROOP L - FRACKVILLE STATION - PAPSP5200)",
    "PA0544900 - BLYTHE TOWNSHIP",
    "PA0545000 - REILLY TOWNSHIP",
    "PA0545100 - EAST NORWEIGIAN TOWNSHIP (COVERED BY SAINT CLAIR BOROUGH - PA0540800)",
    "PA0545200 - PSU - SCHUYLKILL CAMPUS",
    "PA0545300 - EAST UNION TOWNSHIP",
    "PA0545400 - DCNR - TUSCARORA STATE PARK",
    "PA0545500 - FOREST RANGERS WEISER STATE FOREST DISTRICT 18",
    "PA0545700 - FOSTER TOWNSHIP (SCHUYLKILL COUNTY)",
    "PA054BF0X - DCNR - BUREAU OF FORESTRY (SCHUYLKILL COUNTY)",
    "PA054DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (SCHUYLKILL COUNTY)",
    "PA0550000 - SNYDER COUNTY SHERIFFS OFFICE",
    "PA0550100 - SELINSGROVE BOROUGH",
    "PA0550200 - MIDDLEBURG BOROUGH",
    "PA0550300 - SHAMOKIN DAM BOROUGH",
    "PA0550400 - (INACTIVE) SNYDER COUNTY DETECTIVES",
    "PA0550700 - SPRING TOWNSHIP (SNYDER COUNTY) (COVERED BY PSP - TROOP F - SELINSGROVE - PAPSP7700)",
    "PA055BF0X - DCNR - BUREAU OF FORESTRY (SNYDER COUNTY) ",
    "PA055DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (SNYDER COUNTY)",
    "PA0560100 - MEYERSDALE BOROUGH",
    "PA0560200 - SOMERSET BOROUGH",
    "PA0560300 - WINDBER BOROUGH",
    "PA0560400 - BERLIN BOROUGH",
    "PA0560500 - BOSWELL BOROUGH",
    "PA0560600 - CENTRAL CITY BOROUGH",
    "PA0560700 - CONEMAUGH TOWNSHIP (SOMERSET COUNTY)",
    "PA0560800 - (INACTIVE) CONFLUENCE BOROUGH (COVERED BY PSP - TROOP A - SOMERSET STATION - PAPSP8000)",
    "PA0560900 - HOOVERSVILLE BOROUGH",
    "PA0561100 - JENNERSTOWN BOROUGH",
    "PA0561300 - PAINT BOROUGH (COVERED BY WINDBER BOROUGH - PA0560300)",
    "PA0561400 - (INACTIVE) SHADE TOWNSHIP (COVERED BY PSP - TROOP A - SOMERSET - PAPSP8000)",
    "PA0561700 - INDIAN LAKE BOROUGH",
    "PA0561800 - ROCKWOOD BOROUGH",
    "PA0561900 - SEVEN SPRINGS BOROUGH (COVERED BY PSP - TROOP A - SOMERSET STATION - PAPSP8000)",
    "PA0562000 - PAINT TOWNSHIP (COVERED BY PSP - TROOP A - SOMERSET STATION - PAPSP8000)",
    "PA0562100 - GARRETT BOROUGH (COVERED BY PSP - TROOP A - SOMERSET STATION - PAPSP8000)",
    "PA0562500 - ELK LICK TOWNSHIP",
    "PA0562600 - SUMMIT TOWNSHIP",
    "PA0562700 - DCNR - KOOSER STATE PARK",
    "PA0562800 - DCNR - LAUREL RIDGE STATE PARK",
    "PA0562900 - DCNR - LAUREL HILL STATE PARK",
    "PA056BF0X - DCNR - BUREAU OF FORESTRY (SOMERSET COUNTY)",
    "PA056DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (SOMERSET COUNTY)",
    "PA0570400 - DCNR - WORLDS END STATE PARK",
    "PA0570500 - FOREST RANGERS LOYALSOCK STATE FOREST DISTRICT 20",
    "PA057BF0X - DCNR - BUREAU OF FORESTRY (SULLIVAN COUNTY)",
    "PA057DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (SULLIVAN COUNTY)",
    "PA0580000 - SUSQUEHANNA COUNTY SHERIFFS (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0580100 - (INACTIVE) SUSQUEHANNA DEPOT BOROUGH (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0580200 - FOREST CITY BOROUGH",
    "PA0580300 - LANESBORO BOROUGH",
    "PA0580400 - MONTROSE BOROUGH",
    "PA0580500 - OAKLAND BOROUGH (COVERED BY SAXONBURG BOROUGH - PA0101900)",
    "PA0580600 - HALLSTEAD BOROUGH (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0580800 - CLIFFORD TOWNSHIP (COVERED BY GREENFIELD TOWNSHIP - PA0353000)",
    "PA0580900 - (INACTIVE) SUSQUEHANNA COUNTY DETECTIVES ",
    "PA0581100 - LITTLE MEADOWS BOROUGH (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0581300 - SILVER LAKE TOWNSHIP",
    "PA0581500 - HOP BOTTOM BOROUGH (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0581800 - UNION DALE BOROUGH (COVERED BY PSP - TROOP R - GIBSON STATION - PAPSP6300)",
    "PA0581900 - GREAT BEND BOROUGH",
    "PA058BF0X - DCNR - BUREAU OF FORESTRY (SUSQUEHANNA COUNTY)",
    "PA058DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (SUSQUEHANNA COUNTY)",
    "PA0590000 - TIOGA COUNTY SHERIFFS",
    "PA0590100 - WELLSBORO BOROUGH",
    "PA0590200 - MANSFIELD BOROUGH",
    "PA0590300 - BLOSSBURG BOROUGH",
    "PA0590400 - ELKLAND BOROUGH",
    "PA0590500 - KNOXVILLE BOROUGH (COVERED BY PSP - TROOP F - MANSFIELD STATION - PAPSP5300)",
    "PA0590600 - LAWRENCE TOWNSHIP (TIOGA COUNTY)",
    "PA0590700 - WESTFIELD BOROUGH",
    "PA0590800 - TIOGA COUNTY DETECTIVES",
    "PA0590900 - MANSFIELD UNIVERSITY OF PENNSYLVANIA",
    "PA0591100 - LAWRENCEVILLE BOROUGH (COVERED BY LAWRENCE TOWNSHIP - PA0590600)",
    "PA0591200 - (INACTIVE) TIOGA BOROUGH (COVERED BY PSP - TROOP F - MANSFIELD STATION - PAPSP5300)",
    "PA0591700 - OSCEOLA TOWNSHIP",
    "PA0591800 - GAINES TOWNSHIP",
    "PA0591900 - NELSON TOWNSHIP (COVERED BY LAWRENCE TOWNSHIP - PA0590600)",
    "PA0592000 - DCNR - HILLS CREEK STATE PARK",
    "PA0592100 - DCNR - LEONARD HARRISON STATE PARK",
    "PA0592200 - FOREST RANGERS TIOGA STATE FOREST DISTRICT 16",
    "PA059BF0X - DCNR - BUREAU OF FORESTRY (TIOGA COUNTY) ",
    "PA059DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (TIOGA COUNTY)",
    "PA0600000 - UNION COUNTY SHERIFFS",
    "PA0600100 - LEWISBURG BOROUGH (COVERED BY BUFFALO VALLEY REGIONAL - PA0601100)",
    "PA0600200 - EAST BUFFALO TOWNSHIP (COVERED BY BUFFALO VALLEY REGIONAL - PA0601100)",
    "PA0600300 - MIFFLINBURG BOROUGH",
    "PA0600400 - (INACTIVE) UNION COUNTY DETECTIVES",
    "PA0600600 - NEW BERLIN BOROUGH",
    "PA0600800 - HARTLETON BOROUGH (COVERED BY PSP - TROOP - F - MILTON STATION UNION COUNTY - PASPF590X-PSP)",
    "PA0600900 - DCNR - RAYMOND B. WINTER STATE PARK",
    "PA0601000 - FOREST RANGERS BALD EAGLE STATE FOREST DISTRICT 7",
    "PA0601100 - BUFFALO VALLEY REGIONAL POLICE",
    "PA060BF0X - DCNR - BUREAU OF FORESTRY (UNION COUNTY) ",
    "PA060DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (UNION COUNTY)",
    "PA0610100 - FRANKLIN CITY",
    "PA0610200 - OIL CITY",
    "PA0610500 - SUGARCREEK BOROUGH",
    "PA0611100 - EMLENTON BOROUGH",
    "PA0611400 - POLK BOROUGH",
    "PA0611500 - DCNR - OIL CREEK STATE PARK",
    "PA061BF0X - DCNR - BUREAU OF FORESTRY (VENANGO COUNTY) ",
    "PA061DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (VENANGO COUNTY)",
    "PA0620000 - WARREN COUNTY SHERIFFS",
    "PA0620100 - WARREN CITY",
    "PA0620200 - CONEWANGO TOWNSHIP",
    "PA0620300 - SHEFFIELD TOWNSHIP (COVERED BY PSP - TROOP E - WARREN STATION - PAPSP8900)",
    "PA0620500 - TIDIOUTE BOROUGH (COVERED BY PSP - TROOP E - WARREN STATION - PAPSP8900)",
    "PA0620600 - YOUNGSVILLE BOROUGH",
    "PA0620700 - WARREN COUNTY DETECTIVES",
    "PA0621200 - DCNR - CHAPMAN STATE PARK",
    "PA0621300 - FOREST RANGERS CORNPLANTER STATE FOREST DISTRICT 14",
    "PA062BF0X - DCNR - BUREAU OF FORESTRY (WARREN COUNTY) ",
    "PA062DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (WARREN COUNTY)",
    "PA0630000 - WASHINGTON COUNTY SHERIFFS",
    "PA0630100 - CANONSBURG BOROUGH",
    "PA0630200 - (INACTIVE) CHARLEROI BOROUGH (COVERED BY CHARLEROI REGIONAL - PA0636000)",
    "PA0630300 - MONONGAHELA CITY",
    "PA0630400 - PETERS TOWNSHIP",
    "PA0630500 - WASHINGTON CITY",
    "PA0630600 - DONORA BOROUGH",
    "PA0630700 - CALIFORNIA BOROUGH",
    "PA0630900 - MCDONALD BOROUGH",
    "PA0631000 - BENTLEYVILLE BOROUGH",
    "PA0631200 - BEALLSVILLE BOROUGH - (PSP - TROOP B - BELLE VERNON STATION - PASPB630X (WASHINGTON COUNTY)",
    "PA0631300 - (INACTIVE) BURGETTSTOWN BOROUGH (COVERED BY SMITH TOWNSHIP - PA0633200)",
    "PA0631400 - CARROLL TOWNSHIP (WASHINGTON COUNTY)",
    "PA0631500 - CECIL TOWNSHIP",
    "PA0631600 - CENTERVILLE BOROUGH",
    "PA0631700 - CHARTIERS TOWNSHIP",
    "PA0631800 - (INACTIVE) CLAYSVILLE BOROUGH (COVERED BY DONEGAL TOWNSHIP - PA0632100)",
    "PA0631900 - (INACTIVE) COAL CENTER BOROUGH (COVERED BY CALIFORNIA BOROUGH - PA0630700)",
    "PA0632000 - (INACTIVE) COKEBURG BOROUGH (COVERED BY PSP - TROOP B - WASHINGTON STATION - PAPSP9100)",
    "PA0632100 - DONEGAL TOWNSHIP",
    "PA0632200 - EAST BETHLEHEM TOWNSHIP (COVERED BY CENTERVILLE BOROUGH - PA0631600)",
    "PA0632300 - EAST WASHINGTON BOROUGH",
    "PA0632400 - FALLOWFIELD TOWNSHIP (COVERED BY PSP - TROOP B - BELLE VERNON STATION - PASPB630X)",
    "PA0632600 - (INACTIVE) HOUSTON BOROUGH (COVERED BY CANONSBURG BOROUGH - PA0630100)",
    "PA0632700 - (INACTIVE) INDEPENDENCE TOWNSHIP (COVERED BY DONEGAL TOWNSHIP - PA0632100)",
    "PA0632800 - LONG BRANCH BOROUGH (COVERED BY CALIFORNIA BOROUGH - PA0630700)",
    "PA0632900 - NORTH CHARLEROI BOROUGH (COVERED BY CHARLEROI REGIONAL - PA0636000)",
    "PA0633000 - NORTH FRANKLIN TOWNSHIP (COVERED BY PSP - TROOP B - WASHINGTON STATION - PAPSP9100)",
    "PA0633100 - ROSCOE BOROUGH (COVERED BY R.E.S.A. REGIONAL - PA0634900)",
    "PA0633200 - SMITH TOWNSHIP",
    "PA0633300 - SOUTH STRABANE TOWNSHIP",
    "PA0633500 - UNION TOWNSHIP (WASHINGTON COUNTY) (COVERED BY MONONGAHELA CITY - PA0630300)",
    "PA0633600 - WEST BROWNSVILLE BOROUGH",
    "PA0633800 - (INACTIVE) MIDWAY BOROUGH (COVERED BY MCDONALD BOROUGH - PA0630900)",
    "PA0633900 - HANOVER TOWNSHIP (WASHINGTON COUNTY)",
    "PA0634000 - MOUNT PLEASANT TOWNSHIP",
    "PA0634100 - NORTH STRABANE TOWNSHIP",
    "PA0634300 - WASHINGTON COUNTY DISTRICT ATTORNEY OFFICE ",
    "PA0634400 - PENNWEST UNIVERSITY - CALIFORNIA POLICE DEPARTMENT",
    "PA0634500 - WEST ALEXANDER BOROUGH (COVERED BY DONEGAL TOWNSHIP - PA0632100)",
    "PA0634600 - ELL-CO REGIONAL (COVERED BY PSP - TROOP B - WASHINGTON STATION - PAPSP9100)",
    "PA0634700 - (INACTIVE) ROBINSON TOWNSHIP (WASHINGTON COUNTY) (COVERED BY MCDONALD BOROUGH - PA0630900)",
    "PA0634900 - R.E.S.A. REGIONAL",
    "PA0635000 - WEST PIKE RUN TOWNSHIP",
    "PA0635300 - JEFFERSON TOWNSHIP (WASHINGTON COUNTY) ",
    "PA0635600 - WASHINGTON COUNTY ALTERNATIVE EDUCATION (COVERED BY WASHINGTON CITY - PA0630500)",
    "PA063589E - WASHINGTON &amp; JEFFERSON COLLEGE",
    "PA0635900 - (INACTIVE) SOUTHWEST REGIONAL (WASHINGTON COUNTY)",
    "PA0636000 - CHARLEROI REGIONAL",
    "PA0638200 - GREENE WASHINGTON REGIONAL",
    "PA063BF0X - DCNR - BUREAU OF FORESTRY (WASHINGTON COUNTY)",
    "PA063DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (WASHINGTON COUNTY)",
    "PA0640000 - WAYNE COUNTY SHERIFFS",
    "PA0640100 - HONESDALE BOROUGH",
    "PA0640200 - HAWLEY BOROUGH",
    "PA0640300 - WAYMART BOROUGH",
    "PA0640400 - LEHIGH TOWNSHIP (WAYNE COUNTY)",
    "PA0640500 - WAYNE COUNTY DETECTIVES",
    "PA064BF0X - DCNR - BUREAU OF FORESTRY (WAYNE COUNTY) ",
    "PA064DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (WAYNE COUNTY)",
    "PA0650000 - WESTMORELAND COUNTY SHERIFFS",
    "PA0650100 - MURRYSVILLE TOWNSHIP",
    "PA0650200 - GREENSBURG CITY",
    "PA0650300 - IRWIN BOROUGH",
    "PA0650400 - JEANNETTE CITY",
    "PA0650500 - LATROBE CITY",
    "PA0650600 - LOWER BURRELL CITY",
    "PA0650700 - MONESSEN CITY",
    "PA0650800 - MOUNT PLEASANT BOROUGH",
    "PA0650900 - NORTH HUNTINGDON TOWNSHIP",
    "PA0651000 - NEW KENSINGTON CITY",
    "PA0651100 - PENN TOWNSHIP (WESTMORELAND COUNTY)",
    "PA0651200 - SCOTTDALE BOROUGH",
    "PA0651300 - VANDERGRIFT BOROUGH",
    "PA0651400 - ARNOLD CITY",
    "PA0651500 - DERRY BOROUGH",
    "PA0651700 - LIGONIER TOWNSHIP (COVERED BY LIGONIER VALLEY REGIONAL - PA0657200)",
    "PA0651900 - NORTH BELLE VERNON BOROUGH",
    "PA0652100 - SOUTH GREENSBURG BOROUGH",
    "PA0652300 - SOUTHWEST GREENSBURG BOROUGH",
    "PA0652400 - TRAFFORD BOROUGH",
    "PA0652500 - WEST NEWTON BOROUGH",
    "PA0652700 - LIGONIER BOROUGH (COVERED BY LIGONIER VALLEY REGIONAL - PA0657200)",
    "PA0652800 - ALLEGHENY TOWNSHIP (WESTMORELAND COUNTY) ",
    "PA0652900 - AVONMORE BOROUGH",
    "PA0653000 - (INACTIVE) BOLIVAR BOROUGH (COVERED BY PSP - TROOP A - GREENSBURG - PAPSP3400)",
    "PA0653300 - (INACTIVE) NEW FLORENCE BOROUGH (COVERED BY SAINT CLAIR TOWNSHIP - PA0655300)",
    "PA0653400 - NORTH IRWIN BOROUGH (COVERED BY IRWIN BOROUGH - PA0650300)",
    "PA0653500 - PENN BOROUGH",
    "PA0653600 - (INACTIVE) SEWARD BOROUGH (COVERED BY PSP TROOP A - GREENSBURG STATION - PAPSP3400)",
    "PA0653700 - UPPER BURRELL TOWNSHIP",
    "PA0653800 - DELMONT BOROUGH",
    "PA0654000 - MANOR BOROUGH",
    "PA0654200 - (INACTIVE) WEST LEECHBURG BOROUGH (COVERED BY ALLEGHENY TOWNSHIP - PA0652800)",
    "PA0654400 - WESTMORELAND COUNTY DETECTIVES",
    "PA0654500 - ROSTRAVER TOWNSHIP",
    "PA0654900 - WASHINGTON TOWNSHIP (WESTMORELAND COUNTY) ",
    "PA0655100 - HYDE PARK BOROUGH (COVERED BY PSP - TROOP A - KISKI VALLEY STATION - PAPSP4500)",
    "PA0655200 - WESTMORELAND COUNTY PARK POLICE",
    "PA0655300 - SAINT CLAIR TOWNSHIP",
    "PA0655500 - (INACTIVE) BELL TOWNSHIP (COVERED BY PSP - TROOP A - KISKI VALLEY STATION - PAPSP4500)",
    "PA0655600 - UNIVERSITY OF PITTSBURGH - GREENSBURG CAMPUS",
    "PA0655800 - SUTERSVILLE BOROUGH (COVERED BY PSP - TROOP A - GREENSBURG - PAPSP3400)",
    "PA0656000 - DCNR - KEYSTONE STATE PARK",
    "PA0656100 - DCNR - LINN RUN STATE PARK",
    "PA0656200 - FOREST RANGERS FORBES STATE FOREST DISTRICT 4",
    "PA0656300 - SMITHTON BOROUGH (COVERED BY PSP - TROOP B - BELLE VERNON STATION (WESTMORELAND COUNTY - PASPB640X)",
    "PA0656800 - PSU - NEW KENSINGTON CAMPUS",
    "PA0657200 - LIGONIER VALLEY REGIONAL",
    "PA065BF0X - DCNR - BUREAU OF FORESTRY (WESTMORELAND COUNTY)",
    "PA065DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 3 (WESTMORELAND COUNTY)",
    "PA0660000 - WYOMING COUNTY SHERIFFS",
    "PA0660100 - FACTORYVILLE BOROUGH (COVERED BY DALTON BOROUGH - PA0351500)",
    "PA0660200 - MESHOPPEN BOROUGH",
    "PA0660400 - TUNKHANNOCK BOROUGH",
    "PA0660500 - WYOMING COUNTY DETECTIVES",
    "PA0660600 - MEHOOPANY TOWNSHIP (COVERED BY MESHOPPEN BOROUGH - PA0660200)",
    "PA0660800 - FALLS TOWNSHIP (WYOMING COUNTY) (COVERED BY TUNKHANNOCK TOWNSHIP - PA0661200)",
    "PA0660900 - LACEYVILLE BOROUGH (COVERED BY PSP - TROOP P - TUNKHANNOCK - PAPSP8700)",
    "PA0661100 - OVERFIELD TOWNSHIP (TUNKHANNOCK TOWNSHIP (WYOMING COUNTY) - PA0661200)",
    "PA0661200 - TUNKHANNOCK TOWNSHIP (WYOMING COUNTY)",
    "PA0661500 - DCNR - VOSBURG NECK STATE PARK",
    "PA066BF0X - DCNR - BUREAU OF FORESTRY (WYOMING COUNTY) ",
    "PA066DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 1 (WYOMING COUNTY)",
    "PA0670000 - YORK COUNTY SHERIFFS",
    "PA0670100 - HANOVER BOROUGH",
    "PA0670200 - YORK CITY",
    "PA0670300 - (INACTIVE) SPRING GARDEN TOWNSHIP (INACTIVE) (COVERED BY YORK COUNTY REGIONAL - PA0676600)",
    "PA0670400 - SPRINGETTSBURY TOWNSHIP",
    "PA0670500 - DALLASTOWN BOROUGH (COVERED BY YORK COUNTY REGIONAL - PA0676600)",
    "PA0670600 - FAIRVIEW TOWNSHIP (YORK COUNTY)",
    "PA0670700 - DCNR - CODORUS STATE PARK",
    "PA0670800 - RED LION BOROUGH (COVERED BY PSP - TROOP J - YORK STATION - PAPSP9400)",
    "PA0670900 - WEST MANCHESTER TOWNSHIP",
    "PA0671000 - WEST YORK BOROUGH",
    "PA0671100 - DCNR - GIFFORD PINCHOT STATE PARK",
    "PA0671200 - (INACTIVE) CONEWAGO TOWNSHIP (YORK COUNTY) (COVERED BY NORTHERN YORK REGIONAL - PA0671500)",
    "PA0671500 - NORTHERN YORK REGIONAL",
    "PA0671600 - (INACTIVE) NORTHEASTERN REGIONAL (COVERED BY YORK COUNTY REGIONAL POLICE - PA0676600)",
    "PA0672000 - HELLAM TOWNSHIP",
    "PA0672100 - JACKSON TOWNSHIP (YORK COUNTY) (COVERED BY NORTHERN YORK REGIONAL - PA0671500)",
    "PA0672300 - NEW FREEDOM BOROUGH (COVERED BY SOUTHERN REGIONAL (YORK COUNTY) - PA0674800)",
    "PA0672500 - (INACTIVE) SOUTHWESTERN REGIONAL (YORK COUNTY)",
    "PA0672700 - WEST MANHEIM TOWNSHIP",
    "PA0672800 - WINDSOR BOROUGH (COVERED BY PSP - TROOP J - YORK STATION - PAPSP9400)",
    "PA0672900 - WINDSOR TOWNSHIP (COVERED BY YORK COUNTY REGIONAL - PA0676600)",
    "PA0673000 - WRIGHTSVILLE BOROUGH (COVERED BY HELLAM TOWNSHIP - PA0672000)",
    "PA0673200 - (INACTIVE) YORK AREA REGIONAL (COVERED BY YORK COUNTY REGIONAL - PA0676600)",
    "PA0673500 - GLEN ROCK BOROUGH (COVERED BY SOUTHERN REGIONAL - PA0674800)",
    "PA0673600 - MOUNT WOLF BOROUGH (COVERED BY YORK COUNTY REGIONAL - PA0676600",
    "PA0673900 - HEIDELBERG TOWNSHIP (YORK COUNTY) (COVERED BY NORTHERN YORK REGIONAL - PA0671500)",
    "PA0674100 - PENN TOWNSHIP (YORK COUNTY)",
    "PA0674300 - STEWARTSTOWN BOROUGH (COVERED BY SOUTHERN REGIONAL - PA0674800)",
    "PA0674500 - YORK COUNTY DETECTIVES",
    "PA0674600 - CARROLL TOWNSHIP (YORK COUNTY)",
    "PA0674700 - NEWBERRY TOWNSHIP",
    "PA0674800 - SOUTHERN REGIONAL (YORK COUNTY)",
    "PA0675100 - NORTH HOPEWELL TOWNSHIP",
    "PA0675800 - MANCHESTER BOROUGH (COVERED BY YORK COUNTY REGIONAL - PA0676600)",
    "PA0676000 - LOWER WINDSOR TOWNSHIP",
    "PA067619E - YORK COLLEGE OF PENNSYLVANIA",
    "PA0676400 - DCNR - SAMUEL S. LEWIS STATE PARK",
    "PA0676500 - PSU - YORK CAMPUS",
    "PA0676600 - YORK COUNTY REGIONAL POLICE",
    "PA067AG00 - (INACTIVE) YORK COUNTY ATTORNEY GENERALS OFFICE",
    "PA067BF0X - DCNR - BUREAU OF FORESTRY (YORK COUNTY) ",
    "PA067DE0X - PA OFFICE OF ATTORNEY GENERAL - REGION 2 (YORK COUNTY)",
    "PA1234500 - OPTIMUM SRS AGENCY",
    "PA5432100 - OPTIMUM NIBRS AGENCY",
    "PABCA010X - PSP - BCI - AUTO THEFT (ADAMS COUNTY)",
    "PABCA020X - PSP - BCI - AUTO THEFT (ALLEGHENY COUNTY) ",
    "PABCA030X - PSP - BCI - AUTO THEFT (ARMSTRONG COUNTY) ",
    "PABCA040X - PSP - BCI - AUTO THEFT (BEAVER COUNTY) ",
    "PABCA050X - PSP - BCI - AUTO THEFT (BEDFORD COUNTY) ",
    "PABCA060X - PSP - BCI - AUTO THEFT (BERKS COUNTY)",
    "PABCA070X - PSP - BCI - AUTO THEFT (BLAIR COUNTY)",
    "PABCA080X - PSP - BCI - AUTO THEFT (BRADFORD COUNTY) ",
    "PABCA090X - PSP - BCI - AUTO THEFT (BUCKS COUNTY)",
    "PABCA100X - PSP - BCI - AUTO THEFT (BUTLER COUNTY) ",
    "PABCA110X - PSP - BCI - AUTO THEFT (CAMBRIA COUNTY) ",
    "PABCA120X - PSP - BCI - AUTO THEFT (CAMERON COUNTY) ",
    "PABCA130X - PSP - BCI - AUTO THEFT (CARBON COUNTY) ",
    "PABCA140X - PSP - BCI - AUTO THEFT (CENTRE COUNTY) ",
    "PABCA150X - PSP - BCI - AUTO THEFT (CHESTER COUNTY) ",
    "PABCA160X - PSP - BCI - AUTO THEFT (CLARION COUNTY) ",
    "PABCA170X - PSP - BCI - AUTO THEFT (CLEARFIELD COUNTY) ",
    "PABCA180X - PSP - BCI - AUTO THEFT (CLINTON COUNTY) ",
    "PABCA190X - PSP - BCI - AUTO THEFT (COLUMBIA COUNTY) ",
    "PABCA200X - PSP - BCI - AUTO THEFT (CRAWFORD COUNTY) ",
    "PABCA210X - PSP - BCI - AUTO THEFT (CUMBERLAND COUNTY) ",
    "PABCA220X - PSP - BCI - AUTO THEFT (DAUPHIN COUNTY) ",
    "PABCA230X - PSP - BCI - AUTO THEFT (DELAWARE COUNTY) ",
    "PABCA240X - PSP - BCI - AUTO THEFT (ELK COUNTY)",
    "PABCA250X - PSP - BCI - AUTO THEFT (ERIE COUNTY)",
    "PABCA260X - PSP - BCI - AUTO THEFT (FAYETTE COUNTY) ",
    "PABCA270X - PSP - BCI - AUTO THEFT (FOREST COUNTY) ",
    "PABCA280X - PSP - BCI - AUTO THEFT (FRANKLIN COUNTY) ",
    "PABCA290X - PSP - BCI - AUTO THEFT (FULTON COUNTY) ",
    "PABCA300X - PSP - BCI - AUTO THEFT (GREENE COUNTY) ",
    "PABCA310X - PSP - BCI - AUTO THEFT (HUNTINGDON COUNTY) ",
    "PABCA320X - PSP - BCI - AUTO THEFT (INDIANA COUNTY) ",
    "PABCA330X - PSP - BCI - AUTO THEFT (JEFFERSON COUNTY) ",
    "PABCA340X - PSP - BCI - AUTO THEFT (JUNIATA COUNTY) ",
    "PABCA350X - PSP - BCI - AUTO THEFT (LACKAWANNA COUNTY) ",
    "PABCA360X - PSP - BCI - AUTO THEFT (LANCASTER COUNTY) ",
    "PABCA370X - PSP - BCI - AUTO THEFT (LAWRENCE COUNTY) ",
    "PABCA380X - PSP - BCI - AUTO THEFT (LEBANON COUNTY) ",
    "PABCA390X - PSP - BCI - AUTO THEFT (LEHIGH COUNTY) ",
    "PABCA400X - PSP - BCI - AUTO THEFT (LUZERNE COUNTY) ",
    "PABCA410X - PSP - BCI - AUTO THEFT (LYCOMING COUNTY) ",
    "PABCA420X - PSP - BCI - AUTO THEFT (MCKEAN COUNTY) ",
    "PABCA430X - PSP - BCI - AUTO THEFT (MERCER COUNTY) ",
    "PABCA440X - PSP - BCI - AUTO THEFT (MIFFLIN COUNTY) ",
    "PABCA450X - PSP - BCI - AUTO THEFT (MONROE COUNTY) ",
    "PABCA460X - PSP - BCI - AUTO THEFT (MONTGOMERY COUNTY) ",
    "PABCA470X - PSP - BCI - AUTO THEFT (MONTOUR COUNTY) ",
    "PABCA480X - PSP - BCI - AUTO THEFT (NORTHAMPTON COUNTY)",
    "PABCA490X - PSP - BCI - AUTO THEFT (NORTHUMBERLAND COUNTY)",
    "PABCA500X - PSP - BCI - AUTO THEFT (PERRY COUNTY)",
    "PABCA510X - PSP - BCI - AUTO THEFT (PHILADELPHIA COUNTY)",
    "PABCA520X - PSP - BCI - AUTO THEFT (PIKE COUNTY)",
    "PABCA530X - PSP - BCI - AUTO THEFT (POTTER COUNTY) ",
    "PABCA540X - PSP - BCI - AUTO THEFT (SCHUYLKILL COUNTY) ",
    "PABCA550X - PSP - BCI - AUTO THEFT (SNYDER COUNTY) ",
    "PABCA560X - PSP - BCI - AUTO THEFT (SOMERSET COUNTY) ",
    "PABCA570X - PSP - BCI - AUTO THEFT (SULLIVAN COUNTY) ",
    "PABCA580X - PSP - BCI - AUTO THEFT (SUSQUEHANNA COUNTY)",
    "PABCA590X - PSP - BCI - AUTO THEFT (TIOGA COUNTY)",
    "PABCA600X - PSP - BCI - AUTO THEFT (UNION COUNTY)",
    "PABCA610X - PSP - BCI - AUTO THEFT (VENANGO COUNTY) ",
    "PABCA620X - PSP - BCI - AUTO THEFT (WARREN COUNTY) ",
    "PABCA630X - PSP - BCI - AUTO THEFT (WASHINGTON COUNTY) ",
    "PABCA640X - PSP - BCI - AUTO THEFT (WAYNE COUNTY)",
    "PABCA650X - PSP - BCI - AUTO THEFT (WESTMORELAND COUNTY)",
    "PABCA660X - PSP - BCI - AUTO THEFT (WYOMING COUNTY) ",
    "PABCA670X - PSP - BCI - AUTO THEFT (YORK COUNTY)",
    "PABCC010X - PSP - BCI - COMPUTER CRIME (ADAMS COUNTY) ",
    "PABCC020X - PSP - BCI - COMPUTER CRIME (ALLEGHENY COUNTY)",
    "PABCC030X - PSP - BCI - COMPUTER CRIME (ARMSTRONG COUNTY)",
    "PABCC040X - PSP - BCI - COMPUTER CRIME (BEAVER COUNTY) ",
    "PABCC050X - PSP - BCI - COMPUTER CRIME (BEDFORD COUNTY)",
    "PABCC060X - PSP - BCI - COMPUTER CRIME (BERKS COUNTY) ",
    "PABCC070X - PSP - BCI - COMPUTER CRIME (BLAIR COUNTY) ",
    "PABCC080X - PSP - BCI - COMPUTER CRIME (BRADFORD COUNTY)",
    "PABCC090X - PSP - BCI - COMPUTER CRIME (BUCKS COUNTY) ",
    "PABCC100X - PSP - BCI - COMPUTER CRIME (BUTLER COUNTY) ",
    "PABCC110X - PSP - BCI - COMPUTER CRIME (CAMBRIA COUNTY)",
    "PABCC120X - PSP - BCI - COMPUTER CRIME (CAMERON COUNTY)",
    "PABCC130X - PSP - BCI - COMPUTER CRIME (CARBON COUNTY) ",
    "PABCC140X - PSP - BCI - COMPUTER CRIME (CENTRE COUNTY) ",
    "PABCC150X - PSP - BCI - COMPUTER CRIME (CHESTER COUNTY)",
    "PABCC160X - PSP - BCI - COMPUTER CRIME (CLARION COUNTY)",
    "PABCC170X - PSP - BCI - COMPUTER CRIME (CLEARFIELD COUNTY)",
    "PABCC180X - PSP - BCI - COMPUTER CRIME (CLINTON COUNTY)",
    "PABCC190X - PSP - BCI - COMPUTER CRIME (COLUMBIA COUNTY)",
    "PABCC200X - PSP - BCI - COMPUTER CRIME (CRAWFORD COUNTY)",
    "PABCC210X - PSP - BCI - COMPUTER CRIME (CUMBERLAND COUNTY)",
    "PABCC220X - PSP - BCI - COMPUTER CRIME (DAUPHIN COUNTY)",
    "PABCC230X - PSP - BCI - COMPUTER CRIME (DELAWARE COUNTY)",
    "PABCC240X - PSP - BCI - COMPUTER CRIME (ELK COUNTY) ",
    "PABCC250X - PSP - BCI - COMPUTER CRIME (ERIE COUNTY) ",
    "PABCC260X - PSP - BCI - COMPUTER CRIME (FAYETTE COUNTY)",
    "PABCC270X - PSP - BCI - COMPUTER CRIME (FOREST COUNTY) ",
    "PABCC280X - PSP - BCI - COMPUTER CRIME (FRANKLIN COUNTY)",
    "PABCC290X - PSP - BCI - COMPUTER CRIME (FULTON COUNTY) ",
    "PABCC300X - PSP - BCI - COMPUTER CRIME (GREENE COUNTY) ",
    "PABCC310X - PSP - BCI - COMPUTER CRIME (HUNTINGDON COUNTY)",
    "PABCC320X - PSP - BCI - COMPUTER CRIME (INDIANA COUNTY)",
    "PABCC330X - PSP - BCI - COMPUTER CRIME (JEFFERSON COUNTY)",
    "PABCC340X - PSP - BCI - COMPUTER CRIME (JUNIATA COUNTY)",
    "PABCC350X - PSP - BCI - COMPUTER CRIME (LACKAWANNA COUNTY)",
    "PABCC360X - PSP - BCI - COMPUTER CRIME (LANCASTER COUNTY)",
    "PABCC370X - PSP - BCI - COMPUTER CRIME (LAWRENCE COUNTY)",
    "PABCC380X - PSP - BCI - COMPUTER CRIME (LEBANON COUNTY)",
    "PABCC390X - PSP - BCI - COMPUTER CRIME (LEHIGH COUNTY) ",
    "PABCC400X - PSP - BCI - COMPUTER CRIME (LUZERNE COUNTY)",
    "PABCC410X - PSP - BCI - COMPUTER CRIME (LYCOMING COUNTY)",
    "PABCC420X - PSP - BCI - COMPUTER CRIME (MCKEAN COUNTY) ",
    "PABCC430X - PSP - BCI - COMPUTER CRIME (MERCER COUNTY) ",
    "PABCC440X - PSP - BCI - COMPUTER CRIME (MIFFLIN COUNTY)",
    "PABCC450X - PSP - BCI - COMPUTER CRIME (MONROE COUNTY) ",
    "PABCC460X - PSP - BCI - COMPUTER CRIME (MONTGOMERY COUNTY)",
    "PABCC470X - PSP - BCI - COMPUTER CRIME (MONTOUR COUNTY)",
    "PABCC480X - PSP - BCI - COMPUTER CRIME (NORTHAMPTON COUNTY)",
    "PABCC490X - PSP - BCI - COMPUTER CRIME (NORTHUMBERLAND COUNTY)",
    "PABCC500X - PSP - BCI - COMPUTER CRIME (PERRY COUNTY) ",
    "PABCC510X - PSP - BCI - COMPUTER CRIME (PHILADELPHIA COUNTY)",
    "PABCC520X - PSP - BCI - COMPUTER CRIME (PIKE COUNTY) ",
    "PABCC530X - PSP - BCI - COMPUTER CRIME (POTTER COUNTY) ",
    "PABCC540X - PSP - BCI - COMPUTER CRIME (SCHUYLKILL COUNTY)",
    "PABCC550X - PSP - BCI - COMPUTER CRIME (SNYDER COUNTY) ",
    "PABCC560X - PSP - BCI - COMPUTER CRIME (SOMERSET COUNTY)",
    "PABCC570X - PSP - BCI - COMPUTER CRIME (SULLIVAN COUNTY)",
    "PABCC580X - PSP - BCI - COMPUTER CRIME (SUSQUEHANNA COUNTY)",
    "PABCC590X - PSP - BCI - COMPUTER CRIME (TIOGA COUNTY) ",
    "PABCC600X - PSP - BCI - COMPUTER CRIME (UNION COUNTY) ",
    "PABCC610X - PSP - BCI - COMPUTER CRIME (VENANGO COUNTY)",
    "PABCC620X - PSP - BCI - COMPUTER CRIME (WARREN COUNTY) ",
    "PABCC630X - PSP - BCI - COMPUTER CRIME (WASHINGTON COUNTY)",
    "PABCC640X - PSP - BCI - COMPUTER CRIME (WAYNE COUNTY) ",
    "PABCC650X - PSP - BCI - COMPUTER CRIME (WESTMORELAND COUNTY)",
    "PABCC660X - PSP - BCI - COMPUTER CRIME (WYOMING COUNTY)",
    "PABCC670X - PSP - BCI - COMPUTER CRIME (YORK COUNTY) ",
    "PABCD010X - PSP - BCI - DRUG LAW (ADAMS COUNTY)",
    "PABCD020X - PSP - BCI - DRUG LAW (ALLEGHENY COUNTY) ",
    "PABCD030X - PSP - BCI - DRUG LAW (ARMSTRONG COUNTY) ",
    "PABCD040X - PSP - BCI - DRUG LAW (BEAVER COUNTY)",
    "PABCD050X - PSP - BCI - DRUG LAW (BEDFORD COUNTY)",
    "PABCD060X - PSP - BCI - DRUG LAW (BERKS COUNTY)",
    "PABCD070X - PSP - BCI - DRUG LAW (BLAIR COUNTY)",
    "PABCD080X - PSP - BCI - DRUG LAW (BRADFORD COUNTY) ",
    "PABCD090X - PSP - BCI - DRUG LAW (BUCKS COUNTY)",
    "PABCD100X - PSP - BCI - DRUG LAW (BUTLER COUNTY)",
    "PABCD110X - PSP - BCI - DRUG LAW (CAMBRIA COUNTY)",
    "PABCD120X - PSP - BCI - DRUG LAW (CAMERON COUNTY)",
    "PABCD130X - PSP - BCI - DRUG LAW (CARBON COUNTY)",
    "PABCD140X - PSP - BCI - DRUG LAW (CENTRE COUNTY)",
    "PABCD150X - PSP - BCI - DRUG LAW (CHESTER COUNTY)",
    "PABCD160X - PSP - BCI - DRUG LAW (CLARION COUNTY)",
    "PABCD170X - PSP - BCI - DRUG LAW (CLEARFIELD COUNTY) ",
    "PABCD180X - PSP - BCI - DRUG LAW (CLINTON COUNTY)",
    "PABCD190X - PSP - BCI - DRUG LAW (COLUMBIA COUNTY) ",
    "PABCD200X - PSP - BCI - DRUG LAW (CRAWFORD COUNTY) ",
    "PABCD210X - PSP - BCI - DRUG LAW (CUMBERLAND COUNTY) ",
    "PABCD220X - PSP - BCI - DRUG LAW (DAUPHIN COUNTY)",
    "PABCD230X - PSP - BCI - DRUG LAW (DELAWARE COUNTY) ",
    "PABCD240X - PSP - BCI - DRUG LAW (ELK COUNTY)",
    "PABCD250X - PSP - BCI - DRUG LAW (ERIE COUNTY)",
    "PABCD260X - PSP - BCI - DRUG LAW (FAYETTE COUNTY)",
    "PABCD270X - PSP - BCI - DRUG LAW (FOREST COUNTY)",
    "PABCD280X - PSP - BCI - DRUG LAW (FRANKLIN COUNTY) ",
    "PABCD290X - PSP - BCI - DRUG LAW (FULTON COUNTY)",
    "PABCD300X - PSP - BCI - DRUG LAW (GREENE COUNTY)",
    "PABCD310X - PSP - BCI - DRUG LAW (HUNTINGDON COUNTY) ",
    "PABCD320X - PSP - BCI - DRUG LAW (INDIANA COUNTY)",
    "PABCD330X - PSP - BCI - DRUG LAW (JEFFERSON COUNTY) ",
    "PABCD340X - PSP - BCI - DRUG LAW (JUNIATA COUNTY)",
    "PABCD350X - PSP - BCI - DRUG LAW (LACKAWANNA COUNTY) ",
    "PABCD360X - PSP - BCI - DRUG LAW (LANCASTER COUNTY) ",
    "PABCD370X - PSP - BCI - DRUG LAW (LAWRENCE COUNTY) ",
    "PABCD380X - PSP - BCI - DRUG LAW (LEBANON COUNTY)",
    "PABCD390X - PSP - BCI - DRUG LAW (LEHIGH COUNTY)",
    "PABCD400X - PSP - BCI - DRUG LAW (LUZERNE COUNTY)",
    "PABCD410X - PSP - BCI - DRUG LAW (LYCOMING COUNTY) ",
    "PABCD420X - PSP - BCI - DRUG LAW (MCKEAN COUNTY)",
    "PABCD430X - PSP - BCI - DRUG LAW (MERCER COUNTY)",
    "PABCD440X - PSP - BCI - DRUG LAW (MIFFLIN COUNTY)",
    "PABCD450X - PSP - BCI - DRUG LAW (MONROE COUNTY)",
    "PABCD460X - PSP - BCI - DRUG LAW (MONTGOMERY COUNTY) ",
    "PABCD470X - PSP - BCI - DRUG LAW (MONTOUR COUNTY)",
    "PABCD480X - PSP - BCI - DRUG LAW (NORTHAMPTON COUNTY) ",
    "PABCD490X - PSP - BCI - DRUG LAW (NORTHUMBERLAND COUNTY)",
    "PABCD500X - PSP - BCI - DRUG LAW (PERRY COUNTY)",
    "PABCD510X - PSP - BCI - DRUG LAW (PHILADELPHIA COUNTY) ",
    "PABCD520X - PSP - BCI - DRUG LAW (PIKE COUNTY)",
    "PABCD530X - PSP - BCI - DRUG LAW (POTTER COUNTY)",
    "PABCD540X - PSP - BCI - DRUG LAW (SCHUYLKILL COUNTY) ",
    "PABCD550X - PSP - BCI - DRUG LAW (SNYDER COUNTY)",
    "PABCD560X - PSP - BCI - DRUG LAW (SOMERSET COUNTY) ",
    "PABCD570X - PSP - BCI - DRUG LAW (SULLIVAN COUNTY) ",
    "PABCD580X - PSP - BCI - DRUG LAW (SUSQUEHANNA COUNTY) ",
    "PABCD590X - PSP - BCI - DRUG LAW (TIOGA COUNTY)",
    "PABCD600X - PSP - BCI - DRUG LAW (UNION COUNTY)",
    "PABCD610X - PSP - BCI - DRUG LAW (VENANGO COUNTY)",
    "PABCD620X - PSP - BCI - DRUG LAW (WARREN COUNTY)",
    "PABCD630X - PSP - BCI - DRUG LAW (WASHINGTON COUNTY) ",
    "PABCD640X - PSP - BCI - DRUG LAW (WAYNE COUNTY)",
    "PABCD650X - PSP - BCI - DRUG LAW (WESTMORELAND COUNTY) ",
    "PABCD660X - PSP - BCI - DRUG LAW (WYOMING COUNTY)",
    "PABCD670X - PSP - BCI - DRUG LAW (YORK COUNTY)",
    "PABCI010X - PSP - BCI - INTELLIGENCE (ADAMS COUNTY) ",
    "PABCI020X - PSP - BCI - INTELLIGENCE (ALLEGHENY COUNTY)",
    "PABCI030X - PSP - BCI - INTELLIGENCE (ARMSTRONG COUNTY)",
    "PABCI040X - PSP - BCI - INTELLIGENCE (BEAVER COUNTY) ",
    "PABCI050X - PSP - BCI - INTELLIGENCE (BEDFORD COUNTY) ",
    "PABCI060X - PSP - BCI - INTELLIGENCE (BERKS COUNTY) ",
    "PABCI070X - PSP - BCI - INTELLIGENCE (BLAIR COUNTY) ",
    "PABCI080X - PSP - BCI - INTELLIGENCE (BRADFORD COUNTY) ",
    "PABCI090X - PSP - BCI - INTELLIGENCE (BUCKS COUNTY) ",
    "PABCI100X - PSP - BCI - INTELLIGENCE (BUTLER COUNTY) ",
    "PABCI110X - PSP - BCI - INTELLIGENCE (CAMBRIA COUNTY) ",
    "PABCI120X - PSP - BCI - INTELLIGENCE (CAMERON COUNTY) ",
    "PABCI130X - PSP - BCI - INTELLIGENCE (CARBON COUNTY) ",
    "PABCI140X - PSP - BCI - INTELLIGENCE (CENTRE COUNTY) ",
    "PABCI150X - PSP - BCI - INTELLIGENCE (CHESTER COUNTY) ",
    "PABCI160X - PSP - BCI - INTELLIGENCE (CLARION COUNTY) ",
    "PABCI170X - PSP - BCI - INTELLIGENCE (CLEARFIELD COUNTY)",
    "PABCI180X - PSP - BCI - INTELLIGENCE (CLINTON COUNTY) ",
    "PABCI190X - PSP - BCI - INTELLIGENCE (COLUMBIA COUNTY) ",
    "PABCI200X - PSP - BCI - INTELLIGENCE (CRAWFORD COUNTY) ",
    "PABCI210X - PSP - BCI - INTELLIGENCE (CUMBERLAND COUNTY)",
    "PABCI220X - PSP - BCI - INTELLIGENCE (DAUPHIN COUNTY) ",
    "PABCI230X - PSP - BCI - INTELLIGENCE (DELAWARE COUNTY) ",
    "PABCI240X - PSP - BCI - INTELLIGENCE (ELK COUNTY)",
    "PABCI250X - PSP - BCI - INTELLIGENCE (ERIE COUNTY) ",
    "PABCI260X - PSP - BCI - INTELLIGENCE (FAYETTE COUNTY) ",
    "PABCI270X - PSP - BCI - INTELLIGENCE (FOREST COUNTY) ",
    "PABCI280X - PSP - BCI - INTELLIGENCE (FRANKLIN COUNTY) ",
    "PABCI290X - PSP - BCI - INTELLIGENCE (FULTON COUNTY) ",
    "PABCI300X - PSP - BCI - INTELLIGENCE (GREENE COUNTY) ",
    "PABCI310X - PSP - BCI - INTELLIGENCE (HUNTINGDON COUNTY)",
    "PABCI320X - PSP - BCI - INTELLIGENCE (INDIANA COUNTY) ",
    "PABCI330X - PSP - BCI - INTELLIGENCE (JEFFERSON COUNTY)",
    "PABCI340X - PSP - BCI - INTELLIGENCE (JUNIATA COUNTY) ",
    "PABCI350X - PSP - BCI - INTELLIGENCE (LACKAWANNA COUNTY)",
    "PABCI360X - PSP - BCI - INTELLIGENCE (LANCASTER COUNTY)",
    "PABCI370X - PSP - BCI - INTELLIGENCE (LAWRENCE COUNTY) ",
    "PABCI380X - PSP - BCI - INTELLIGENCE (LEBANON COUNTY) ",
    "PABCI390X - PSP - BCI - INTELLIGENCE (LEHIGH COUNTY) ",
    "PABCI400X - PSP - BCI - INTELLIGENCE (LUZERNE COUNTY) ",
    "PABCI410X - PSP - BCI - INTELLIGENCE (LYCOMING COUNTY) ",
    "PABCI420X - PSP - BCI - INTELLIGENCE (MCKEAN COUNTY) ",
    "PABCI430X - PSP - BCI - INTELLIGENCE (MERCER COUNTY) ",
    "PABCI440X - PSP - BCI - INTELLIGENCE (MIFFLIN COUNTY) ",
    "PABCI450X - PSP - BCI - INTELLIGENCE (MONROE COUNTY) ",
    "PABCI460X - PSP - BCI - INTELLIGENCE (MONTGOMERY COUNTY)",
    "PABCI470X - PSP - BCI - INTELLIGENCE (MONTOUR COUNTY) ",
    "PABCI480X - PSP - BCI - INTELLIGENCE (NORTHAMPTON COUNTY)",
    "PABCI490X - PSP - BCI - INTELLIGENCE (NORTHUMBERLAND COUNTY)",
    "PABCI500X - PSP - BCI - INTELLIGENCE (PERRY COUNTY) ",
    "PABCI510X - PSP - BCI - INTELLIGENCE (PHILADELPHIA COUNTY)",
    "PABCI520X - PSP - BCI - INTELLIGENCE (PIKE COUNTY) ",
    "PABCI530X - PSP - BCI - INTELLIGENCE (POTTER COUNTY) ",
    "PABCI540X - PSP - BCI - INTELLIGENCE (SCHUYLKILL COUNTY)",
    "PABCI550X - PSP - BCI - INTELLIGENCE (SNYDER COUNTY) ",
    "PABCI560X - PSP - BCI - INTELLIGENCE (SOMERSET COUNTY) ",
    "PABCI570X - PSP - BCI - INTELLIGENCE (SULLIVAN COUNTY) ",
    "PABCI580X - PSP - BCI - INTELLIGENCE (SUSQUEHANNA COUNTY)",
    "PABCI590X - PSP - BCI - INTELLIGENCE (TIOGA COUNTY) ",
    "PABCI600X - PSP - BCI - INTELLIGENCE (UNION COUNTY) ",
    "PABCI610X - PSP - BCI - INTELLIGENCE (VENANGO COUNTY) ",
    "PABCI620X - PSP - BCI - INTELLIGENCE (WARREN COUNTY) ",
    "PABCI630X - PSP - BCI - INTELLIGENCE (WASHINGTON COUNTY)",
    "PABCI640X - PSP - BCI - INTELLIGENCE (WAYNE COUNTY) ",
    "PABCI650X - PSP - BCI - INTELLIGENCE (WESTMORELAND COUNTY)",
    "PABCI660X - PSP - BCI - INTELLIGENCE (WYOMING COUNTY) ",
    "PABCI670X - PSP - BCI - INTELLIGENCE (YORK COUNTY) ",
    "PABCO010X - PSP - BCI - ORGANIZED CRIME (ADAMS COUNTY) ",
    "PABCO020X - PSP - BCI - ORGANIZED CRIME (ALLEGHENY COUNTY)",
    "PABCO030X - PSP - BCI - ORGANIZED CRIME (ARMSTRONG COUNTY)",
    "PABCO040X - PSP - BCI - ORGANIZED CRIME (BEAVER COUNTY)",
    "PABCO050X - PSP - BCI - ORGANIZED CRIME (BEDFORD COUNTY)",
    "PABCO060X - PSP - BCI - ORGANIZED CRIME (BERKS COUNTY) ",
    "PABCO070X - PSP - BCI - ORGANIZED CRIME (BLAIR COUNTY) ",
    "PABCO080X - PSP - BCI - ORGANIZED CRIME (BRADFORD COUNTY)",
    "PABCO090X - PSP - BCI - ORGANIZED CRIME (BUCKS COUNTY) ",
    "PABCO100X - PSP - BCI - ORGANIZED CRIME (BUTLER COUNTY)",
    "PABCO110X - PSP - BCI - ORGANIZED CRIME (CAMBRIA COUNTY)",
    "PABCO120X - PSP - BCI - ORGANIZED CRIME (CAMERON COUNTY)",
    "PABCO130X - PSP - BCI - ORGANIZED CRIME (CARBON COUNTY)",
    "PABCO140X - PSP - BCI - ORGANIZED CRIME (CENTRE COUNTY)",
    "PABCO150X - PSP - BCI - ORGANIZED CRIME (CHESTER COUNTY)",
    "PABCO160X - PSP - BCI - ORGANIZED CRIME (CLARION COUNTY)",
    "PABCO170X - PSP - BCI - ORGANIZED CRIME (CLEARFIELD COUNTY)",
    "PABCO180X - PSP - BCI - ORGANIZED CRIME (CLINTON COUNTY)",
    "PABCO190X - PSP - BCI - ORGANIZED CRIME (COLUMBIA COUNTY)",
    "PABCO200X - PSP - BCI - ORGANIZED CRIME (CRAWFORD COUNTY)",
    "PABCO210X - PSP - BCI - ORGANIZED CRIME (CUMBERLAND COUNTY)",
    "PABCO220X - PSP - BCI - ORGANIZED CRIME (DAUPHIN COUNTY)",
    "PABCO230X - PSP - BCI - ORGANIZED CRIME (DELAWARE COUNTY)",
    "PABCO240X - PSP - BCI - ORGANIZED CRIME (ELK COUNTY) ",
    "PABCO250X - PSP - BCI - ORGANIZED CRIME (ERIE COUNTY) ",
    "PABCO260X - PSP - BCI - ORGANIZED CRIME (FAYETTE COUNTY)",
    "PABCO270X - PSP - BCI - ORGANIZED CRIME (FOREST COUNTY)",
    "PABCO280X - PSP - BCI - ORGANIZED CRIME (FRANKLIN COUNTY)",
    "PABCO290X - PSP - BCI - ORGANIZED CRIME (FULTON COUNTY)",
    "PABCO300X - PSP - BCI - ORGANIZED CRIME (GREENE COUNTY)",
    "PABCO310X - PSP - BCI - ORGANIZED CRIME (HUNTINGDON COUNTY)",
    "PABCO320X - PSP - BCI - ORGANIZED CRIME (INDIANA COUNTY)",
    "PABCO330X - PSP - BCI - ORGANIZED CRIME (JEFFERSON COUNTY)",
    "PABCO340X - PSP - BCI - ORGANIZED CRIME (JUNIATA COUNTY)",
    "PABCO350X - PSP - BCI - ORGANIZED CRIME (LACKAWANNA COUNTY)",
    "PABCO360X - PSP - BCI - ORGANIZED CRIME (LANCASTER COUNTY)",
    "PABCO370X - PSP - BCI - ORGANIZED CRIME (LAWRENCE COUNTY)",
    "PABCO380X - PSP - BCI - ORGANIZED CRIME (LEBANON COUNTY)",
    "PABCO390X - PSP - BCI - ORGANIZED CRIME (LEHIGH COUNTY)",
    "PABCO400X - PSP - BCI - ORGANIZED CRIME (LUZERNE COUNTY)",
    "PABCO410X - PSP - BCI - ORGANIZED CRIME (LYCOMING COUNTY)",
    "PABCO420X - PSP - BCI - ORGANIZED CRIME (MCKEAN COUNTY)",
    "PABCO430X - PSP - BCI - ORGANIZED CRIME (MERCER COUNTY)",
    "PABCO440X - PSP - BCI - ORGANIZED CRIME (MIFFLIN COUNTY)",
    "PABCO450X - PSP - BCI - ORGANIZED CRIME (MONROE COUNTY)",
    "PABCO460X - PSP - BCI - ORGANIZED CRIME (MONTGOMERY COUNTY)",
    "PABCO470X - PSP - BCI - ORGANIZED CRIME (MONTOUR COUNTY)",
    "PABCO480X - PSP - BCI - ORGANIZED CRIME (NORTHAMPTON COUNTY)",
    "PABCO490X - PSP - BCI - ORGANIZED CRIME (NORTHUMBERLAND COUNTY)",
    "PABCO500X - PSP - BCI - ORGANIZED CRIME (PERRY COUNTY) ",
    "PABCO510X - PSP - BCI - ORGANIZED CRIME (PHILADELPHIA COUNTY)",
    "PABCO520X - PSP - BCI - ORGANIZED CRIME (PIKE COUNTY) ",
    "PABCO530X - PSP - BCI - ORGANIZED CRIME (POTTER COUNTY)",
    "PABCO540X - PSP - BCI - ORGANIZED CRIME (SCHUYLKILL COUNTY)",
    "PABCO550X - PSP - BCI - ORGANIZED CRIME (SNYDER COUNTY)",
    "PABCO560X - PSP - BCI - ORGANIZED CRIME (SOMERSET COUNTY)",
    "PABCO570X - PSP - BCI - ORGANIZED CRIME (SULLIVAN COUNTY)",
    "PABCO580X - PSP - BCI - ORGANIZED CRIME (SUSQUEHANNA COUNTY)",
    "PABCO590X - PSP - BCI - ORGANIZED CRIME (TIOGA COUNTY) ",
    "PABCO600X - PSP - BCI - ORGANIZED CRIME (UNION COUNTY) ",
    "PABCO610X - PSP - BCI - ORGANIZED CRIME (VENANGO COUNTY)",
    "PABCO620X - PSP - BCI - ORGANIZED CRIME (WARREN COUNTY)",
    "PABCO630X - PSP - BCI - ORGANIZED CRIME (WASHINGTON COUNTY)",
    "PABCO640X - PSP - BCI - ORGANIZED CRIME (WAYNE COUNTY) ",
    "PABCO650X - PSP - BCI - ORGANIZED CRIME (WESTMORELAND COUNTY)",
    "PABCO660X - PSP - BCI - ORGANIZED CRIME (WYOMING COUNTY)",
    "PABCO670X - PSP - BCI - ORGANIZED CRIME (YORK COUNTY) ",
    "PACO10000 - ALVERNIA UNIVERSITY - MAIN CAMPUS",
    "PACO10100 - (INACTIVE) ALVERNIA UNIVERSITY - PHILADELPHIA CENTER (CLOSED)",
    "PACO10200 - ALVERNIA UNIVERSITY - SCHUYLKILL CENTER (POTTSVILLE CAMPUS)",
    "PACO10300 - AMERICAN COLLEGE OF FINANCIAL SERVICES ",
    "PACO10400 - ARCADIA UNIVERSITY",
    "PACO10500 - (INACTIVE) ART INSTITUTE OF PITTSBURGH ",
    "PACO10600 - (INACTIVE) BEAVER COUNTY COMMUNITY COLLEGE ",
    "PACO10700 - (INACTIVE) BIBLICAL THEOLOGICAL SEMINARY ",
    "PACO10800 - Bryn Athyn College",
    "PACO10900 - Bryn Mawr College",
    "PACO11000 - BUCKS COUNTY COMMUNITY COLLEGE",
    "PACO11100 - (INACTIVE) CABRINI UNIVERSITY",
    "PACO11200 - CAIRN UNIVERSITY",
    "PACO11300 - (INACTIVE) CALVARY BASTIST THEOLOGICAL SEMINARY",
    "PACO11400 - CENTRAL PENN COLLEGE",
    "PACO11500 - CHESTNUT HILL COLLEGE",
    "PACO11600 - (INACTIVE) CHI INSTITUTE - BENSALEM CAMPUS ",
    "PACO11700 - (INACTIVE) CHI INSTITUTE - BROOMALL CAMPUS ",
    "PACO11800 - (INACTIVE) CHRIST THE SAVIOUR SEMINARY ",
    "PACO11900 - (INACTIVE) CLARION UNIVERSITY OF PENNSYLVANIA - VENANGO CAMPUS",
    "PACO12000 - (INACTIVE) CLARKS SUMMIT UNIVERSITY",
    "PACO12100 - COMMUNITY COLLEGE OF PHILADELPHIA",
    "PACO12200 - CURTIS INSTITUTE OF MUSIC",
    "PACO12300 - DELAWARE COUNTY COMMUNITY COLLEGE - DOWNINGTOWN CAMPUS",
    "PACO12400 - (INACTIVE) DELAWARE COUNTY COMMUNITY COLLEGE - EXTON CENTER (LOCATION CLOSED)",
    "PACO12500 - DELAWARE COUNTY COMMUNITY COLLEGE - MAIN CAMPUS (MARPLE)",
    "PACO12600 - DELAWARE COUNTY COMMUNITY COLLEGE - PENNOCKS BRIDGE",
    "PACO12700 - DELAWARE COUNTY COMMUNITY COLLEGE - PHOENIXVILLE CENTER",
    "PACO12800 - DELAWARE COUNTY COMMUNITY COLLEGE - SOUTHEAST",
    "PACO12900 - DELAWARE COUNTY COMMUNITY COLLEGE - UPPER DARBY",
    "PACO13000 - DREXEL UNIVERSITY - CENTER CITY CAMPUS ",
    "PACO13100 - DREXEL UNIVERSITY - LEBOW COLLEGE OF BUSINESS MALVERN CAMPUS",
    "PACO13200 - DREXEL UNIVERSITY - QUEEN LANE CAMPUS",
    "PACO13400 - (INACTIVE) EASTERN BAPTIST THEOLOGICAL SEMINARY",
    "PACO13500 - (INACTIVE) EASTERN UNIVERSITY - CENTER CITY CAMPUS (LOCATION CLOSED)",
    "PACO13600 - EASTERN UNIVERSITY - ESPERANZA COLLEGE ",
    "PACO13700 - (INACTIVE) EASTERN UNIVERSITY - FALLS CENTER CAMPUS (LOCATION CLOSED)",
    "PACO13800 - (INACTIVE) EASTERN UNIVERSITY - HARRISBURG CAMPUS (LOCATION CLOSED)",
    "PACO13900 - EASTERN UNIVERSITY - MAIN CAMPUS",
    "PACO14000 - (INACTIVE) EASTERN UNIVERSITY - PALMER SEMINARY (PART OF MAIN CAMPUS)",
    "PACO14100 - (INACTIVE) EASTERN UNIVERSITY - PEOPLE FOR PEOPLE INSTITUTE (CLOSED)",
    "PACO14200 - ELIZABETHTOWN UNIVERSITY",
    "PACO14300 - EVANGELICAL THEOLOGICAL SEMINARY",
    "PACO14400 - (INACTIVE) FAITH THEOLOGICAL SEMINARY",
    "PACO14500 - (INACTIVE) FRANKFORD HOSPITAL SCHOOL OF NURSING",
    "PACO14600 - GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - CENTRAL CAMPUS",
    "PACO14700 - (INACTIVE) GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - DOYLESTOWN CAMPUS (CLOSED)",
    "PACO14800 - GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - GUTHRIE CAMPUS",
    "PACO14900 - GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - NORTH CAMPUS",
    "PACO15000 - GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - SOUTH CAMPUS",
    "PACO15100 - (INACTIVE) GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - WILLIAMSPORT WEST CAMPUS (CLOSED)",
    "PACO15200 - GENEVA COLLEGE",
    "PACO15300 - GETTYSBURG COLLEGE",
    "PACO15400 - (INACTIVE) GRATZ COLLEGE (MONTGOMERY COUNTY)",
    "PACO15500 - GROVE CITY COLLEGE",
    "PACO15600 - GWYNEDD MERCY UNIVERSITY",
    "PACO15700 - (INACTIVE) GWYNEDD MERCY UNIVERSITY - BENSALEM",
    "PACO15800 - (INACTIVE) GWYNEDD MERCY UNIVERSITY - PUBLIC LEDGER BUILDING",
    "PACO15900 - HAHNEMANN UNIVERSITY",
    "PACO16000 - (INACTIVE) HARCUM COLLEGE",
    "PACO16100 - HARRISBURG AREA COMMUNITY COLLEGE - GETTYSBURG CAMPUS",
    "PACO16200 - HARRISBURG AREA COMMUNITY COLLEGE - HARRISBURG CAMPUS",
    "PACO16300 - HARRISBURG AREA COMMUNITY COLLEGE - LANCASTER CAMPUS",
    "PACO16400 - HARRISBURG AREA COMMUNITY COLLEGE - LEBANON CAMPUS",
    "PACO16500 - HARRISBURG AREA COMMUNITY COLLEGE - MIDTOWN CAMPUS",
    "PACO16600 - HARRISBURG AREA COMMUNITY COLLEGE - YORK CAMPUS",
    "PACO16700 - HARRISBURG UNIVERSITY OF SCIENCE AND TECHNOLOGY",
    "PACO16800 - Haverford College",
    "PACO16900 - (INACTIVE) HIRAM G. ANDREWS CENTER",
    "PACO17000 - HOLY FAMILY UNIVERSITY",
    "PACO17100 - (INACTIVE) ICM SCHOOL OF BUSINESS &amp; MEDICAL CAREERS OF PITTSBURGH",
    "PACO17200 - IMMACULATA UNIVERSITY",
    "PACO17300 - (INACTIVE) INDIANA UNIVERSITY OF PENNSYLVANIA - NORTHPOINTE CAMPUS",
    "PACO17400 - (INACTIVE) ITT TECHNICAL INSTITUTE - BENSALEM CAMPUS",
    "PACO17500 - (INACTIVE) ITT TECHNICAL INSTITUTE - KING OF PRUSSIA CAMPUS",
    "PACO17600 - (INACTIVE) ITT TECHNICAL INSTITUTE - MECHANICSBURG CAMPUS",
    "PACO17700 - (INACTIVE) ITT TECHNICAL INSTITUTE - MONROEVILLE CAMPUS",
    "PACO17800 - (INACTIVE) ITT TECHNICAL INSTITUTE - PITTSBURGH CAMPUS",
    "PACO17900 - (INACTIVE) KEYSTONE COLLEGE - DELAWARE VALLEY CAMPUS",
    "PACO18000 - KEYSTONE COLLEGE - MAIN CAMPUS",
    "PACO18100 - (INACTIVE) KEYSTONE COLLEGE - TOWANDA CAMPUS",
    "PACO18200 - King's College",
    "PACO18300 - (INACTIVE) LA SALLE UNIVERSITY - ALLENTOWN CAMPUS",
    "PACO18400 - (INACTIVE) LA SALLE UNIVERSITY - BUCKS CAMPUS",
    "PACO18500 - (INACTIVE) LA SALLE UNIVERSITY - MONTGOMERY CAMPUS",
    "PACO18600 - La Salle University - Philadelphia Campus ",
    "PACO18700 - LACKAWANNA COLLEGE - HAZLETON CAMPUS",
    "PACO18800 - LACKAWANNA COLLEGE - LAKE REGION CENTER (HAWLEY)",
    "PACO18900 - LACKAWANNA COLLEGE - TUNKHANOCK CAMPUS ",
    "PACO19000 - LACKAWANNA COLLEGE - SUNBURY CAMPUS",
    "PACO19100 - LACKAWANNA COLLEGE - TOWANDA CAMPUS",
    "PACO19200 - LANCASTER BIBLE COLLEGE",
    "PACO19300 - (INACTIVE) LANCASTER THEOLOGICAL SEMINARY ",
    "PACO19400 - LA ROCHE UNIVERSITY",
    "PACO19500 - LEHIGH-CARBON COMMUNITY COLLEGE - DONLEY CENTER",
    "PACO19600 - LEHIGH-CARBON COMMUNITY COLLEGE - MAIN CAMPUS",
    "PACO19700 - LEHIGH-CARBON COMMUNITY COLLEGE - MORGAN CENTER",
    "PACO19800 - Lock Haven University - Clearfield Campus ",
    "PACO19900 - LUTHERAN THEOLOGICAL SEMINARY OF PHILADELPHIA",
    "PACO20000 - LYCOMING COLLEGE",
    "PACO20100 - MANOR COLLEGE",
    "PACO20200 - MARYWOOD UNIVERSITY",
    "PACO20300 - MESSIAH UNIVERSITY",
    "PACO20400 - MISERICORDIA UNIVERSITY",
    "PACO20500 - MISERICORDIA UNIVERSITY - NURSING PITTSBURGH",
    "PACO20600 - MONTGOMERY COUNTY COMMUNITY COLLEGE - BLUE BELL CAMPUS",
    "PACO20700 - MONTGOMERY COUNTY COMMUNITY COLLEGE - POTTSTOWN CAMPUS",
    "PACO20800 - MOORE COLLEGE OF ART &amp; DESIGN",
    "PACO20900 - Muhlenberg University",
    "PACO21000 - Neumann University",
    "PACO21100 - NORTHAMPTON COMMUNITY COLLEGE - BETHLEHEM CAMPUS",
    "PACO21200 - NORTHAMPTON COMMUNITY COLLEGE - MONROE CAMPUS",
    "PACO21300 - (INACTIVE) NORTHWEST PENNSYLVANIA TECHNICAL INSTITUTE",
    "PACO21400 - PEIRCE COLLEGE",
    "PACO21500 - Penn Highlands Community College - Johnstown Campus",
    "PACO21600 - Pennsylvania Academy of the Fine Arts",
    "PACO21700 - SAINT JOSEPHS UNIVERSITY - LANCASTER",
    "PACO21800 - Pennsylvania College of Optometry",
    "PACO21900 - PENNSYLVANIA INSTITUTE OF TECHNOLOGY - MAIN CAMPUS",
    "PACO22000 - Pennsylvania Institute of Technology - Philadelphia Campus",
    "PACO22100 - PHILADELPHIA COLLEGE OF OSTEOPATHIC MEDICINE",
    "PACO22200 - (INACTIVE) PITTSBURGH INSTITUTE OF AERONAUTICS",
    "PACO22300 - PITTSBURGH INSTITUTE OF MORTUARY SCIENCE ",
    "PACO22400 - (INACTIVE) PITTSBURGH TECHNICAL COLLEGE ",
    "PACO22500 - Pittsburgh Theological Seminary",
    "PACO22600 - PSU - CARLISLE CAMPUS (COVERED BY PSU - CARLISLE CAMPUS - PA0213800)",
    "PACO22700 - PSU - College of Medicine",
    "PACO22800 - READING AREA COMMUNITY COLLEGE",
    "PACO22900 - RECONSTRUCTIONIST RABBINICAL COLLEGE",
    "PACO23000 - REFORMED PRESBYTERIAN THEOLOGICAL SEMINARY ",
    "PACO23100 - ROSEMONT COLLEGE",
    "PACO23200 - (INACTIVE) SAINT CHARLES BORROMEO SEMINARY ",
    "PACO23300 - SAINT JOSEPHS UNIVERSITY - HAWK HILL",
    "PACO23400 - SAINT TIKHONS ORTHODOX THEOLOGICAL SEMINARY",
    "PACO23500 - SAINT VINCENT COLLEGE",
    "PACO23600 - (INACTIVE) SALUS UNIVERSITY",
    "PACO23700 - STRAYER UNIVERSITY - PENN CENTER WEST CAMPUS",
    "PACO23800 - STRAYER UNIVERSITY - ALLENTOWN CAMPUS",
    "PACO23900 - Strayer University - Center City Campus ",
    "PACO24000 - STRAYER UNIVERSITY - CRANBERRY WOODS CAMPUS",
    "PACO24100 - STRAYER UNIVERSITY - DELAWARE COUNTY CAMPUS",
    "PACO24200 - (INACTIVE) STRAYER UNIVERSITY - KING OF PRUSSIA CAMPUS",
    "PACO24300 - Strayer University - Lower Bucks County Campus",
    "PACO24400 - (INACTIVE) STRAYER UNIVERSITY - WARRENDALE CAMPUS",
    "PACO24500 - SUSQUEHANNA UNIVERSITY",
    "PACO24600 - SWARTHMORE COLLEGE",
    "PACO24700 - TEMPLE UNIVERSITY - AMBLER CAMPUS",
    "PACO24800 - TEMPLE UNIVERSITY - CENTER CITY CAMPUS ",
    "PACO24900 - TEMPLE UNIVERSITY - HARRISBURG CAMPUS",
    "PACO25000 - TEMPLE UNIVERSITY - HEALTH SCIENCES CENTER ",
    "PACO25100 - TEMPLE UNIVERSITY - SCHOOL OF PODIATRIC MEDICINE",
    "PACO25200 - TEMPLE UNIVERSITY - TYLER CAMPUS",
    "PACO25300 - Thaddeus Stevens College of Technology ",
    "PACO25400 - THE ART INSTITUTE OF PHILADELPHIA",
    "PACO25500 - THOMAS JEFFERSON UNIVERSITY - CENTER CITY CAMPUS",
    "PACO25600 - THOMAS JEFFERSON UNIVERSITY - EAST FALLS CAMPUS",
    "PACO25700 - (INACTIVE) THOMPSON INSTITUTE - CHAMBERSBURG CAMPUS",
    "PACO25800 - (INACTIVE) THOMPSON INSTITUTE - HARRISBURG CAMPUS",
    "PACO25900 - THOMPSON INSTITUTE OF PHILADELPHIA",
    "PACO26000 - (INACTIVE) TRINITY SCHOOL FOR MINISTRY ",
    "PACO26100 - (INACTIVE) UNITED LUTHERAN SEMINARY - GETTYSBURG",
    "PACO26200 - (INACTIVE) UNIVERSITY OF PENNSYLVANIA",
    "PACO26300 - (INACTIVE) UNIVERSITY OF PHOENIX - HARRISBURG CAMPUS",
    "PACO26400 - UNIVERSITY OF PHOENIX - PHILADELPHIA MAIN CAMPUS",
    "PACO26500 - (INACTIVE) UNIVERSITY OF PHOENIX - PITTSBURGH CAMPUS",
    "PACO26600 - (INACTIVE) UNIVERSITY OF PHOENIX - WARRINGTON CAMPUS",
    "PACO26700 - (INACTIVE) UNIVERSITY OF PHOENIX - WAYNE CAMPUS",
    "PACO26800 - (INACTIVE) UNIVERSITY OF THE ARTS",
    "PACO26900 - (INACTIVE) UNIVERSITY OF THE SCIENCES (MERGED WITH SAINT JOSEPHS UNIVERSITY)",
    "PACO27000 - UNIVERSITY OF VALLEY FORGE",
    "PACO27100 - URSINUS COLLEGE",
    "PACO27200 - VALLEY FORGE MILITARY ACADEMY &amp; COLLEGE",
    "PACO27300 - ( INACTIVE) WASHINGTON &amp; JEFFERSON COLLEGE",
    "PACO27400 - WAYNESBURG UNIVERSITY - MAIN CAMPUS",
    "PACO27500 - WAYNESBURG UNIVERSITY - SOUTHPOINTE CAMPUS ",
    "PACO27600 - (INACTIVE) WESTMINSTER THEOLOGICAL SEMINARY",
    "PACO27700 - WESTMORELAND COMMUNITY COLLEGE",
    "PACO27800 - WIDENER LAW - COMMONWEALTH &amp; HARRISBURG CAMPUS",
    "PACO27900 - WIDENER UNIVERSITY",
    "PACO28000 - (INACTIVE) WIDENER UNIVERSITY - MAIN CAMPUS",
    "PACO28100 - (INACTIVE) WILSON COLLEGE",
    "PACO28200 - EASTERN UNIVERSITY - CITY LINE",
    "PACO28300 - DREXEL UNIVERSITY COLLEG OF MEDICINE - WEST READING CAMPUS",
    "PACO28400 - NORTHAMPTON COMMUNITY COLLEGE - FOWLER CENTER",
    "PACO28500 - ALVERNIA UNIVERSITY - COLLEGETOWNE",
    "PACO28600 - SAINT JOSEPHS UNIVERSITY - UNIVERSITY CITY ",
    "PACO28700 - GEISINGER COMMONWEALTH SCHOOL OF MEDICINE - WEST CAMPUS",
    "PACO28800 - PENN VIEW BIBLE INSTITUTE",
    "PACO28900 - ALBRIGHT UNIVERSITY",
    "PACO29000 - DREXEL UNIVERSITY - ELKINS PARK CAMPUS ",
    "PACO29100 - GEISINGER COLLEGE OF HEALTH SCIENCES - SCHOOL OF NURSING",
    "PADAA010X - (INACTIVE) ADAMS COUNTY DISTRICT ATTORNEY (PA001043A)",
    "PADAA020X - Allegheny County District Attorney - Investigation Division (PA002013A)",
    "PADAA210X - CUMBERLAND COUNTY DISTRICT ATTORNEY (PA021013A)",
    "PADAA280X - (INACTIVE) FRANKLIN COUNTY DISTRICT ATTORNEYS OFFICE (PA028013A)",
    "PADAA390X - LEHIGH COUNTY DISTRICT ATTORNEY (PA039013A)",
    "PADAA440X - MIFFLIN COUNTY DETECTIVES (PA044013A)",
    "PADAA450X - (INACTIVE) MONROE COUNTY DETECTIVES (PA045013A)",
    "PADAA540X - (INACTIVE) NORTHAMPTON COUNTY DISTRICT ATTORNEYS OFFICE (PA048013A)",
    "PADTF010X - ADAMS COUNTY DRUG TASK FORCE (PA001013A) ",
    "PAGEA0200 - PSP - BGE - VALLEY FORGE CASINO",
    "PAGEA0400 - PSP - BGE - RIVERS CASINO - PITTSBURGH ",
    "PAGEA0500 - PSP - BGE - NEMACOLIN CASINO",
    "PAGEA0600 - PSP - BGE - MOUNT AIRY CASINO",
    "PAGEA0700 - PSP - BGE - WIND CREEK CASINO",
    "PAGEA0900 - PSP - BGE - RIVERS CASINO - PHILADELPHIA ",
    "PAGEA1000 - PSP - BGE - HOLLYWOOD CASINO",
    "PAGEA1100 - PSP - BGE - PRESQUE ISLE CASINO",
    "PAGEA1200 - (INACTIVE) PSP - BGE - HARRAHS CASINO - CHESTER",
    "PAGEA1300 - PSP - BGE - MEADOWS CASINO",
    "PAGEA1400 - PSP - BGE - POCONO DOWNS CASINO",
    "PAGEA1500 - PSP - BGE - PARX CASINO",
    "PAGMDHQ0X - PSP - BGE - DHQ Employee Data",
    "PALCE010X - PSP - BLCE - DISTRICT 3 (ADAMS COUNTY) ",
    "PALCE030X - PSP - BLCE - DISTRICT 7 (ARMSTRONG COUNTY) ",
    "PALCE040X - PSP - BLCE - DISTRICT 4 (BEAVER COUNTY) ",
    "PALCE050X - PSP - BLCE - DISTRICT 5 (BEDFORD COUNTY) ",
    "PALCE060X - PSP - BLCE - DISTRICT 9 (BERKS COUNTY) ",
    "PALCE080X - PSP - BLCE - DISTRICT 6 (BRADFORD COUNTY) ",
    "PALCE090X - PSP - BLCE - DISTRICT 9 (BUCKS COUNTY) ",
    "PALCE100X - PSP - BLCE - DISTRICT 7 (BUTLER COUNTY) ",
    "PALCE110X - PSP - BLCE - DISTRICT 5 (CAMBRIA COUNTY) ",
    "PALCE120X - PSP - BLCE - DISTRICT 7 (CAMERON COUNTY) ",
    "PALCE130X - PSP - BLCE - DISTRICT 2 (CARBON COUNTY) ",
    "PALCE140X - (INACTIVE) PSP - BLCE - DISTRICT 5 (CENTRE COUNTY)",
    "PALCE150X - PSP - BLCE - DISTRICT 1 (CHESTER COUNTY) ",
    "PALCE160X - PSP - BLCE - DISTRICT 7 (CLARION COUNTY) ",
    "PALCE170X - PSP - BLCE - DISTRICT 7 (CLEARFIELD COUNTY)",
    "PALCE180X - PSP - BLCE - DISTRICT 6 (CLINTON COUNTY) ",
    "PALCE190X - PSP - BLCE - DISTRICT 6 (COLUMBIA COUNTY) ",
    "PALCE200X - PSP - BLCE - DISTRICT 8 (CRAWFORD COUNTY) ",
    "PALCE210X - PSP - BLCE - DISTRICT 3 (CUMBERLAND COUNTY)",
    "PALCE230X - PSP - BLCE - DISTRICT 1 (DELAWARE COUNTY) ",
    "PALCE240X - PSP - BLCE - DISTRICT 7 (ELK COUNTY)",
    "PALCE260X - PSP - BLCE - DISTRICT 4 (FAYETTE COUNTY) ",
    "PALCE270X - PSP - BLCE - DISTRICT 7 (FOREST COUNTY) ",
    "PALCE280X - PSP - BLCE - DISTRICT 3 (FRANKLIN COUNTY) ",
    "PALCE290X - PSP - BLCE - DISTRICT 5 (FULTON COUNTY) ",
    "PALCE300X - PSP - BLCE - DISTRICT 4 (GREENE COUNTY) ",
    "PALCE310X - PSP - BLCE - DISTRICT 5 (HUNTINGDON COUNTY)",
    "PALCE320X - PSP - BLCE - DISTRICT 7 (INDIANA COUNTY) ",
    "PALCE340X - PSP - BLCE - DISTRICT 3 (JUNIATA COUNTY) ",
    "PALCE350X - PSP - BLCE - DISTRICT 2 (LACKAWANNA COUNTY)",
    "PALCE360X - PSP - BLCE - DISTRICT 3 (LANCASTER COUNTY) ",
    "PALCE370X - PSP - BLCE - DISTRICT 8 (LAWRENCE COUNTY) ",
    "PALCE380X - PSP - BLCE - DISTRICT 3 (LEBANON COUNTY) ",
    "PALCE420X - PSP - BLCE - DISTRICT 8 (MCKEAN COUNTY) ",
    "PALCE430X - PSP - BLCE - DISTRICT 8 (MERCER COUNTY) ",
    "PALCE440X - PSP - BLCE - DISTRICT 5 (MIFFLIN COUNTY) ",
    "PALCE450X - PSP - BLCE - DISTRICT 2 (MONROE COUNTY) ",
    "PALCE460X - PSP - BLCE - DISTRICT 9 (MONTGOMERY COUNTY)",
    "PALCE470X - PSP - BLCE - DISTRICT 6 (MONTOUR COUNTY) ",
    "PALCE480X - PSP - BLCE - DISTRICT 9 (NORTHAMPTON COUNTY)",
    "PALCE490X - PSP - BLCE - DISTRICT 6 (NORTHUMBERLAND COUNTY)",
    "PALCE500X - PSP - BLCE - DISTRICT 3 (PERRY COUNTY) ",
    "PALCE520X - PSP - BLCE - DISTRICT 2 (PIKE COUNTY)",
    "PALCE530X - PSP - BLCE - DISTRICT 6 (POTTER COUNTY) ",
    "PALCE540X - PSP - BLCE - DISTRICT 9 (SCHUYLKILL COUNTY)",
    "PALCE550X - PSP - BLCE - DISTRICT 6 (SNYDER COUNTY) ",
    "PALCE560X - PSP - BLCE - DISTRICT 5 (SOMERSET COUNTY) ",
    "PALCE570X - PSP - BLCE - DISTRICT 6 (SULLIVAN COUNTY) ",
    "PALCE580X - PSP - BLCE - DISTRICT 2 (SUSQUEHANNA COUNTY)",
    "PALCE590X - PSP - BLCE - DISTRICT 6 (TIOGA COUNTY) ",
    "PALCE600X - PSP - BLCE - DISTRICT 6 (UNION COUNTY) ",
    "PALCE610X - PSP - BLCE - DISTRICT 8 (VENANGO COUNTY) ",
    "PALCE620X - PSP - BLCE - DISTRICT 8 (WARREN COUNTY) ",
    "PALCE630X - PSP - BLCE - DISTRICT 4 (WASHINGTON COUNTY)",
    "PALCE640X - PSP - BLCE - DISTRICT 2 (WAYNE COUNTY) ",
    "PALCE650X - PSP - BLCE - DISTRICT 4 (WESTMORELAND COUNTY)",
    "PALCE660X - PSP - BLCE - DISTRICT 2 (WYOMING COUNTY) ",
    "PALCE670X - PSP - BLCE - DISTRICT 3 (YORK COUNTY)",
    "PAPD0600X - ONTELAUNEE TOWNSHIP (COVERED BY NORTHERN BERKS REGIONAL - PA0062800)",
    "PAPD0800X - READING NORTHERN RAILROAD POLICE (BRADFORD)",
    "PAPD1300X - READING NORTHERN RAILROAD POLICE (CARBON) ",
    "PAPD2000X - READING NORTHERN RAILROAD POLICE (COLUMBIA)",
    "PAPD3500X - READING NORTHERN RAILROAD POLICE (LACKAWANNA)",
    "PAPD4000X - READING NORTHERN RAILROAD POLICE (LUZERNE) ",
    "PAPD4900X - READING NORTHERN RAILROAD POLICE (NORTHUMBERLAND)",
    "PAPD5400X - READING NORTHERN RAILROAD POLICE (SCHUYLKILL)",
    "PAPD6600X - READING NORTHERN RAILROAD POLICE (WYOMING) ",
    "PAPDA640X - NEW ALEXANDRIA BOROUGH (COVERED BY PSP - TROOP A - KISKI VALLEY STATION - PAPSP4500)",
    "PAPDA650X - NORFOLK SOUTHERN - WYOMING COUNTY",
    "PAPDA660X - SOUTHWEST REGIONAL - WESTMORELAND CO. (COVERED BY SOUTHWEST REGIONAL POLICE - PA0260400)",
    "PAPDB010X - GREENE COUNTY REGIONAL - WASHINGTON COUNTY (PA0301200) (COVERED BY SOUTHWEST REGIONAL - PA0260400)",
    "PAPDB030X - NORFOLK SOUTHERN - ALLEGHENY COUNTY (PA0023G9E)",
    "PAPDB620X - MORRIS TOWNSHIP (WASHINGTON COUNTY) (COVERED BY PSP - TROOP B - WASHINGTON - PAPSP9100)",
    "PAPDC010X - ST. PETERSBURG BORO (COVERED BY PSP - TROOP C - CLARION STATION - PAPSP8200)",
    "PAPDC020X - FOXBURG BOROUGH (COVERED BY PSP - TROOP C - CLARION STATION - PAPSP8200)",
    "PAPDC110X - RICHLAND TOWNSHIP (CLARION COUNTY) (COVERED BY PSP - TROOP C - CLARION - PAPSP8200)",
    "PAPDD010X - EAST VALE BOROUGH (PA0043100) (COVERED BY BEAVER FALLS CITY - PA0040500)",
    "PAPDD100X - JEFFERSON TOWNSHIP (BUTLER COUNTY) (COVERED BY PSP - TROOP D - BUTLER - PAPSP0900)",
    "PAPDD430X - HERMITAGE TWP (COVERED BY HERMITAGE CITY - PA0430400)",
    "PAPDD570X - NORFOLK SOUTHERN - SUSQUEHANNA COUNTY",
    "PAPDE250X - LAKE ERIE COLLEGE OF OSTEOPATHIC MEDICINE ",
    "PAPDF490X - NORFOLK SOUTHERN - NORTHUMBERLAND COUNTY ",
    "PAPDG070X - (INACTIVE) ALTOONA HOSPITAL (PA007019P) ",
    "PAPDG080X - NORFOLK SOUTHERN - BLAIR COUNTY (PA0072G9E)",
    "PAPDH010X - (INACTIVE) LATIMORE-YORK SPRINGS REGIONAL ",
    "PAPDH020X - ADAMS COUNTY DETECTIVES",
    "PAPDH220X - NORFOLK SOUTHERN - DAUPHIN COUNTY (PA0222G9E)",
    "PAPDH280X - NORFOLK SOUTHERN - FRANKLIN COUNTY (PA028159E)",
    "PAPDJ360X - (INACTIVE) CONESTOGA TOWNSHIP (LANCASTER COUNTY) (COVERED BY PSP - TROOP J - LANCASTER - PAPSP4700)",
    "PAPDJ380X - EAST DRUMORE TOWNSHIP (COVERED BY PSP - TROOP J - LANCASTER STATION - PAPSP4700)",
    "PAPDJ390X - LANCASTER COUNTY DETECTIVES DRUG TASK FORCE",
    "PAPDM090X - NORFOLK SOUTHERN - BUCKS COUNTY (PA0092G9E)",
    "PAPDM390X - NORFOLK SOUTHERN - LEHIGH COUNTY (PA0392G9E)",
    "PAPDN190X - MIFFLIN TOWNSHIP (COVERED BY SOUTH CENTRE TOWNSHIP - PA0191000)",
    "PAPDP400X - NORFOLK SOUTHERN - LUZERNE COUNTY",
    "PAPDR350X - LACKAWANNA COLLEGE - SCRANTON CAMPUS (PA035PP01)",
    "PAPEP0000 - PHILADELPHIA CITY",
    "PAPPD0000 - PITTSBURGH CITY",
    "PAPSP0000 - PENNSYLVANIA STATE POLICE (TOTAL EMPLOYEE) ",
    "PAPSP0200 - PSP - TROOP J - AVONDALE STATION",
    "PAPSP0300 - PSP - TROOP D - BEAVER STATION",
    "PAPSP0400 - PSP - TROOP G - BEDFORD STATION",
    "PAPSP0500 - PSP - TROOP B - BELLE VERNON STATION",
    "PAPSP0600 - PSP - TROOP M - BETHLEHEM STATION",
    "PAPSP0700 - PSP - TROOP N - BLOOMSBURG STATION",
    "PAPSP0800 - PSP - TROOP T - BOWMANSVILLE STATION",
    "PAPSP0900 - PSP - TROOP D - BUTLER STATION",
    "PAPSP0L00 - PSP - BLCE - DISTRICT 1 (PHILADELPHIA COUNTY)",
    "PAPSP0M10 - PSP - BLCE - DISTRICT 9 (LEHIGH COUNTY) ",
    "PAPSP0N00 - PSP - BLCE - DISTRICT 3 (DAUPHIN COUNTY) ",
    "PAPSP0P00 - PSP - BLCE - DISTRICT 5 (BLAIR COUNTY) ",
    "PAPSP0R00 - PSP - BLCE - DISTRICT 2 (LUZERNE COUNTY) ",
    "PAPSP0T00 - PSP - BLCE - DISTRICT 8 (ERIE COUNTY)",
    "PAPSP0V00 - PSP - BLCE - DISTRICT 6 (LYCOMING COUNTY) ",
    "PAPSP0W00 - PSP - BLCE - DISTRICT 7 (JEFFERSON COUNTY) ",
    "PAPSP0X00 - PSP - TROOP T - NEW STANTON STATION",
    "PAPSP0Y00 - PSP - BLCE - DISTRICT 4 (ALLEGHENY COUNTY) ",
    "PAPSP1000 - PSP - TROOP H - CARLISLE STATION",
    "PAPSP1100 - PSP - TROOP B - PITTSBURGH STATION",
    "PAPSP1200 - PSP - TROOP H - CHAMBERSBURG STATION",
    "PAPSP1300 - PSP - TROOP C - CLEARFIELD STATION",
    "PAPSP1400 - PSP - TROOP E - CORRY STATION",
    "PAPSP1500 - PSP - TROOP F - COUDERSPORT STATION",
    "PAPSP1600 - PSP - TROOP M - DUBLIN STATION",
    "PAPSP1700 - PSP - TROOP C - DUBOIS STATION",
    "PAPSP1800 - PSP - TROOP H - NEWPORT STATION",
    "PAPSP1900 - PSP - TROOP R - DUNMORE STATION",
    "PAPSP2000 - PSP - TROOP P - LAPORTE STATION",
    "PAPSP2100 - PSP - TROOP M - BELFAST STATION",
    "PAPSP2200 - PSP - TROOP A - EBENSBURG STATION",
    "PAPSP2300 - PSP - TROOP F - EMPORIUM STATION",
    "PAPSP2400 - (INACTIVE) PSP - TROOP J - EPHRATA STATION ",
    "PAPSP2500 - PSP - TROOP E - ERIE STATION",
    "PAPSP2600 - PSP - TROOP T - EVERETT STATION",
    "PAPSP2700 - PSP - TROOP J - EMBREEVILLE STATION",
    "PAPSP2800 - PSP - TROOP N - FERN RIDGE STATION",
    "PAPSP2900 - PSP - TROOP M - FOGELSVILLE STATION",
    "PAPSP3000 - PSP - TROOP E - FRANKLIN STATION",
    "PAPSP3100 - PSP - TROOP H - GETTYSBURG STATION",
    "PAPSP3200 - PSP - TROOP T - GIBSONIA STATION",
    "PAPSP3300 - PSP - TROOP E - GIRARD STATION",
    "PAPSP3400 - PSP - TROOP A - GREENSBURG STATION",
    "PAPSP3500 - PSP - TROOP L - HAMBURG STATION",
    "PAPSP3600 - PSP - TROOP H - HARRISBURG STATION",
    "PAPSP3700 - PSP - TROOP N - HAZLETON STATION",
    "PAPSP3800 - (INACTIVE) PSP - TROOP T - HIGHSPIRE STATION",
    "PAPSP3900 - PSP - TROOP G - HOLLIDAYSBURG STATION",
    "PAPSP4000 - PSP - TROOP R - HONESDALE STATION",
    "PAPSP4100 - PSP - TROOP G - HUNTINGDON STATION",
    "PAPSP4200 - PSP - TROOP A - INDIANA STATION",
    "PAPSP4300 - PSP - TROOP L - JONESTOWN STATION",
    "PAPSP4400 - PSP - TROOP C - LEWIS RUN STATION",
    "PAPSP4500 - PSP - TROOP A - KISKI VALLEY STATION",
    "PAPSP4600 - PSP - TROOP D - KITTANNING STATION",
    "PAPSP4700 - PSP - TROOP J - LANCASTER STATION",
    "PAPSP4800 - PSP - TROOP N - LEHIGHTON STATION",
    "PAPSP4900 - PSP - TROOP G - LEWISTOWN STATION",
    "PAPSP5000 - PSP - TROOP F - LAMAR STATION",
    "PAPSP5100 - PSP - TROOP H - LYKENS STATION",
    "PAPSP5200 - PSP - TROOP L - FRACKVILLE STATION",
    "PAPSP5300 - PSP - TROOP F - MANSFIELD STATION",
    "PAPSP5400 - PSP - TROOP G - MCCONNELLSBURG STATION ",
    "PAPSP5500 - PSP - TROOP E - MEADVILLE STATION",
    "PAPSP5600 - PSP - TROOP K - MEDIA STATION",
    "PAPSP5700 - PSP - TROOP D - MERCER STATION",
    "PAPSP5800 - PSP - TROOP R - BLOOMING GROVE STATION ",
    "PAPSP5900 - PSP - TROOP F - MILTON STATION",
    "PAPSP6000 - PSP - TROOP F - MONTOURSVILLE STATION",
    "PAPSP6200 - PSP - TROOP D - NEW CASTLE STATION",
    "PAPSP6300 - PSP - TROOP R - GIBSON STATION",
    "PAPSP6400 - PSP - TROOP T - NEWVILLE STATION",
    "PAPSP6500 - PSP - TROOP K - PHILADELPHIA STATION",
    "PAPSP6600 - PSP - TROOP G - PHILIPSBURG STATION (COVERED BY PSP - TROOP G - ROCKVIEW STATION - PAPSP7400)",
    "PAPSP6800 - PSP - TROOP T - KING OF PRUSSIA STATION ",
    "PAPSP6900 - PSP - TROOP T - POCONO STATION",
    "PAPSP7000 - PSP - TROOP C - PUNXSUTAWNEY STATION",
    "PAPSP7200 - PSP - TROOP L - READING STATION",
    "PAPSP7300 - PSP - TROOP C - RIDGWAY STATION",
    "PAPSP7400 - PSP - TROOP G - ROCKVIEW STATION",
    "PAPSP7500 - PSP - TROOP L - SCHUYLKILL HAVEN STATION ",
    "PAPSP7600 - PSP - TROOP K - SKIPPACK STATION",
    "PAPSP7700 - PSP - TROOP F - SELINSGROVE STATION",
    "PAPSP7800 - PSP - TROOP F - STONINGTON STATION",
    "PAPSP7900 - PSP - TROOP P - SHICKSHINNY STATION",
    "PAPSP8000 - PSP - TROOP A - SOMERSET STATION",
    "PAPSP8100 - PSP - TROOP T - SOMERSET STATION",
    "PAPSP8200 - PSP - TROOP C - CLARION STATION",
    "PAPSP8300 - PSP - TROOP N - STROUDSBURG STATION",
    "PAPSP8400 - PSP - TROOP C - MARIENVILLE STATION",
    "PAPSP8500 - PSP - TROOP P - TOWANDA STATION",
    "PAPSP8600 - PSP - TROOP M - TREVOSE STATION",
    "PAPSP8700 - PSP - TROOP P - TUNKHANNOCK STATION",
    "PAPSP8800 - PSP - TROOP B - UNIONTOWN STATION",
    "PAPSP8900 - PSP - TROOP E - WARREN STATION",
    "PAPSP9100 - PSP - TROOP B - WASHINGTON STATION",
    "PAPSP9200 - PSP - TROOP B - WAYNESBURG STATION",
    "PAPSP9300 - PSP - TROOP P - WILKES-BARRE STATION",
    "PAPSP9400 - PSP - TROOP J - YORK STATION",
    "PAPSPA100 - PSP - TROOP T - JEFFERSON HILLS STATION ",
    "PAPSPHQ01 - PENNSYLVANIA STATE POLICE - UCR UNIT - NIBRS",
    "PASPB020X - PSP - TROOP B - BELLE VERNON STATION (ALLEGHENY COUNTY)",
    "PASPB620X - PSP - TROOP B - PITTSBURGH STATION (WASHINGTON COUNTY)",
    "PASPB630X - PSP - TROOP B - BELLE VERNON STATION (WASHINGTON COUNTY)",
    "PASPB640X - PSP - TROOP B - BELLE VERNON STATION (WESTMORELAND COUNTY)",
    "PASPC160X - PSP - TROOP C - MARIENVILLE (CLARION COUNTY)",
    "PASPC170X - PSP - TROOP C - PUNXSUTAWNEY STATION (CLEARFIELD COUNTY)",
    "PASPC320X - PSP - TROOP C - PUNXSUTAWNEY STATION (INDIANA COUNTY)",
    "PASPC330X - PSP - TROOP C - MARIENVILLE (JEFFERSON COUNTY)",
    "PASPC340X - PSP - TROOP C - DUBOIS STATION (JEFFERSON COUNTY)",
    "PASPD020X - PSP - TROOP D - KITTANNING STATION (ALLEGHENY COUNTY)",
    "PASPDQT0X - PSP - DHQ LEGACY EMPLOYEE DATA",
    "PASPE200X - PSP - TROOP E - CORRY STATION (CRAWFORD COUNTY)",
    "PASPE630X - PSP - TROOP E - CORRY STATION (WARREN COUNTY)",
    "PASPF470X - PSP - TROOP F - MILTON STATION (MONTOUR COUNTY)",
    "PASPF590X - PSP - TROOP F - MILTON STATION (UNION COUNTY)",
    "PASPG440X - PSP - TROOP G - LEWISTOWN STATION (MIFFLIN COUNTY)",
    "PASPH210X - PSP - TROOP H - CHAMBERSBURG STATION (CUMBERLAND COUNTY)",
    "PASPK460X - PSP - TROOP K - PHILADELPHIA STATION (MONTGOMERY COUNTY)",
    "PASPM480X - PSP - TROOP M - BETHLEHEM STATION (NORTHAMPTON COUNTY)",
    "PASPN130X - PSP - TROOP N - FERN RIDGE STATION (CARBON COUNTY)",
    "PASPN140X - PSP - TROOP N - HAZLETON STATION (CARBON COUNTY)",
    "PASPN450X - (INACTIVE) PSP - TROOP N - LEHIGHTON STATION (MONROE COUNTY)",
    "PASPT020X - PSP - TROOP T - NEW STANTON STATION (ALLEGHENY COUNTY)",
    "PASPT040X - PSP - TROOP T - GIBSONIA STATION (BEAVER COUNTY)",
    "PASPT060X - PSP - TROOP T - BOWMANSVILLE STATION (BERKS COUNTY)",
    "PASPT090X - PSP - TROOP T - KING OF PRUSSIA STATION (BUCKS COUNTY)",
    "PASPT100X - PSP - TROOP T - GIBSONIA STATION (BUTLER COUNTY)",
    "PASPT1500 - PSP - TROOP T - KING OF PRUSSIA STATION (CHESTER COUNTY)",
    "PASPT160X - PSP - TROOP T - BOWMANSVILLE STATION (CHESTER COUNTY)",
    "PASPT220X - PSP - TROOP T - NEWVILLE STATION (DAUPHIN COUNTY)",
    "PASPT230X - PSP - TROOP T - BOWMANSVILLE STATION (DAUPHIN COUNTY)",
    "PASPT280X - PSP - TROOP T - NEWVILLE STATION (FRANKLIN COUNTY)",
    "PASPT290X - PSP - TROOP T - EVERETT STATION (FULTON COUNTY)",
    "PASPT300X - PSP - TROOP T - NEW STANTON STATION (FAYETTE COUNTY)",
    "PASPT310X - PSP - TROOP T - NEWVILLE STATION (HUNTINGDON COUNTY)",
    "PASPT350X - PSP - TROOP T - POCONO STATION (LACKAWANNA COUNTY)",
    "PASPT370X - PSP - TROOP T - GIBSONIA STATION (LAWRENCE COUNTY)",
    "PASPT380X - PSP - TROOP T - BOWMANSVILLE STATION (LEBANON COUNTY)",
    "PASPT390X - PSP - TROOP T - POCONO STATION (LEHIGH COUNTY)",
    "PASPT400X - PSP - TROOP T - POCONO STATION (LUZERNE COUNTY)",
    "PASPT410X - PSP - TROOP T - KING OF PRUSSIA STATION (LEHIGH COUNTY)",
    "PASPT620X - PSP - TROOP T - NEW STANTON STATION (WASHINGTON COUNTY)",
    "PASPT640X - PSP - TROOP T - SOMERSET STATION (WESTMORELAND COUNTY)",
    "PASPT660X - PSP - TROOP T - NEWVILLE STATION (YORK COUNTY)",
    "PAUCRUNIT - PENNSYLVANIA STATE POLICE - UCR UNIT - SRS ",
    "PAUNI020X - ROBERT MORRIS UNIVERSITY (PA002019E)",
    "PAUNI090X - DELAWARE VALLEY UNIVERSITY (PA00901VS) ",
    "PAUNI110X - SAINT FRANCIS UNIVERSITY (PA011019E)",
    "PAUNI360X - FRANKLIN &amp; MARSHALL COLLEGE (PA036579E)",
    "PAUNI400X - LUZERNE COUNTY COMMUNITY COLLEGE (PA04002VS)",
    "PAUNI600X - BUCKNELL UNIVERSITY (PA060019E)",
    "PAUNI650X - SETON HILL UNIVERSITY (PA065609P)",
    "PAVEND006 - VERSATERM RMS",
]


report_links = {
    "home" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/",
    "home" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Home/Index",
    "faq" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/FAQ/SearchResults?searchTerm=",

    "crime_in_pennsylvania" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/CrimePublication/CrimePublicationReports",
    "reports" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/ReportsIndex/List",
    "pccd_data_extracts" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/ExtractFile/ListPCCDExtractFiles",
    "crime_map" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/CrimeAnalytics/index.html",
    "ad-hoc_queries" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/AdvancedSearch/AdvancedSearch",
    "arrest_distribution_breakdown_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/ArrestDrillDown",
    "arrest_distribution_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/ArrestDistribution",
    "arrest_trends_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/ArrestTrends",
    "group_a_offense_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/GroupACrimeReport",
    "hate_crime_incidents_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/HateCrimeByORIReport",
    "leoka_incidents_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/LEOKAByORIReport",
    "murder_incidents_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/MurderCrimeByORIReport",
    "offense_and_arrest_summary_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/AgencySummaryReport",
    "offense_distribution_breakdown_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/DrillDownReports",
    "offense_distribution_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/CrimeDistributionReport",
    "offense_trends_comparison_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/CrimesIndex",
    "offense_trends_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/CrimeTrends",
    "annual_srs_summary_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/SRSReport/AnnualSRSSummary",
    "arrest_distribution_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/SRSReport/CrimeDistribution",
    "hate_crime_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/HateCrime",
    "offense_density_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/SRSReport/CrimeDensity",
    "offense_trends_comparison" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/SRSReport/CrimesIndex",
    "offense_trends_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/SRSReport/CrimeTrends",
    "ytd_comparison_report" : "https://www.ucr.pa.gov/PAUCRSPUBLIC/Report/UCRSummary",
}


#### special selection
agency_options = [
    "PA0010100 - GETTYSBURG BOROUGH",
    "PA0010200 - LITTLESTOWN BOROUGH",
    "PA0010400 - MCSHERRYSTOWN BOROUGH",
    "PA0020200 - BELLEVUE BOROUGH",
    "PA0020500 - CARNEGIE BOROUGH",
    "PA0020900 - DUQUESNE CITY",
    "PA0020F00 - KENNEDY TOWNSHIP",
    "PA0021400 - MCKEES ROCKS BOROUGH",
    "PA0021I00 - LEET TOWNSHIP",
    "PA0022200 - ROSS TOWNSHIP",
    "PA0022600 - WHITEHALL BOROUGH",
    "PA0022700 - WILKINSBURG BOROUGH",
    "PA0023500 - MOON TOWNSHIP",
    "PA0024400 - SWISSVALE BOROUGH",
    "PA0024700 - WEST MIFFLIN BOROUGH",
    "PA0024900 - BELL ACRES BOROUGH",
    "PA0025600 - BRIDGEVILLE BOROUGH",
    "PA0026100 - CRAFTON BOROUGH",
    "PA0029000 - SHARPSBURG BOROUGH",
    "PA0030400 - KITTANNING BOROUGH",
    "PA0032000 - WEST KITTANNING BOROUGH",
    "PA0040200 - AMBRIDGE BOROUGH",
    "PA0040600 - MIDLAND BOROUGH",
    "PA0040900 - BRIGHTON TOWNSHIP",
    "PA0041900 - CHIPPEWA TOWNSHIP",
    "PA0043800 - NEW SEWICKLEY TOWNSHIP",
    "PA0050200 - EVERETT BOROUGH",
    "PA0050400 - SAXTON BOROUGH",
    "PA0061300 - MUHLENBERG TOWNSHIP",
    "PA0061400 - READING CITY",
    "PA0062100 - WEST READING BOROUGH",
    "PA0062200 - WYOMISSING BOROUGH",
    "PA0063100 - BERN TOWNSHIP",
    "PA0063600 - LOWER HEIDELBERG TOWNSHIP",
    "PA0066000 - TULPEHOCKEN TOWNSHIP",
    "PA0070100 - ALTOONA CITY",
    "PA0070800 - DUNCANSVILLE BOROUGH",
    "PA0080100 - ATHENS BOROUGH",
    "PA0080300 - SAYRE BOROUGH",
    "PA0080400 - TOWANDA BOROUGH",
    "PA0080800 - ATHENS TOWNSHIP",
    "PA0090300 - BRISTOL TOWNSHIP",
    "PA0090700 - LOWER MAKEFIELD TOWNSHIP",
    "PA0090900 - MIDDLETOWN TOWNSHIP",
    "PA0091100 - NEW HOPE BOROUGH",
    "PA0091300 - NORTHAMPTON TOWNSHIP",
    "PA0092000 - UPPER SOUTHAMPTON TOWNSHIP",
    "PA0092200 - YARDLEY BOROUGH",
    "PA0092900 - HILLTOWN TOWNSHIP",
    "PA0093200 - NEW BRITAIN TOWNSHIP",
    "PA0094300 - PLUMSTEAD TOWNSHIP",
    "PA0094700 - BUCKINGHAM TOWNSHIP",
    "PA0095300 - BEDMINSTER TOWNSHIP",
    "PA0095700 - DUBLIN BOROUGH",
    "PA0100300 - SLIPPERY ROCK BOROUGH",
    "PA0101200 - CRANBERRY TOWNSHIP",
    "PA0110400 - JOHNSTOWN CITY",
    "PA0111800 - UPPER YODER TOWNSHIP",
    "PA0130400 - LEHIGHTON BOROUGH",
    "PA0130700 - WEATHERLY BOROUGH",
    "PA0140300 - STATE COLLEGE BOROUGH",
    "PA0140700 - FERGUSON TOWNSHIP",
    "PA0150400 - EAST WHITELAND TOWNSHIP",
    "PA0150500 - KENNETT SQUARE BOROUGH",
    "PA0150600 - MALVERN BOROUGH",
    "PA0150800 - OXFORD BOROUGH",
    "PA0151000 - PHOENIXVILLE BOROUGH",
    "PA0151400 - WEST GOSHEN TOWNSHIP",
    "PA0152100 - EAST FALLOWFIELD TOWNSHIP",
    "PA0152300 - SOUTH COATESVILLE BOROUGH",
    "PA0153200 - EAST PIKELAND TOWNSHIP",
    "PA0160100 - CLARION BOROUGH",
    "PA0170300 - CURWENSVILLE BOROUGH",
    "PA0170800 - SANDY TOWNSHIP",
    "PA0180100 - LOCK HAVEN CITY",
    "PA0190100 - BENTON BOROUGH",
    "PA0190200 - BERWICK BOROUGH",
    "PA0190400 - CATAWISSA BOROUGH",
    "PA0190500 - BRIAR CREEK TOWNSHIP",
    "PA0191500 - LOCUST TOWNSHIP",
    "PA0200100 - MEADVILLE CITY",
    "PA0200200 - TITUSVILLE CITY",
    "PA0200300 - CAMBRIDGE SPRINGS BOROUGH",
    "PA0201400 - WEST MEAD TOWNSHIP",
    "PA0210100 - CAMP HILL BOROUGH",
    "PA0210200 - CARLISLE BOROUGH",
    "PA0210300 - EAST PENNSBORO TOWNSHIP",
    "PA0210600 - LOWER ALLEN TOWNSHIP",
    "PA0211500 - NORTH MIDDLETON TOWNSHIP",
    "PA0211700 - SILVER SPRING TOWNSHIP",
    "PA0220200 - HARRISBURG CITY",
    "PA0220400 - LOWER PAXTON TOWNSHIP",
    "PA0220500 - MIDDLETOWN BOROUGH",
    "PA0220700 - STEELTON BOROUGH",
    "PA0220900 - SWATARA TOWNSHIP",
    "PA0221000 - HIGHSPIRE BOROUGH",
    "PA0221300 - PENBROOK BOROUGH",
    "PA0221700 - LOWER SWATARA TOWNSHIP",
    "PA0222100 - ROYALTON BOROUGH",
    "PA0230400 - CHESTER CITY",
    "PA0230600 - COLLINGDALE BOROUGH",
    "PA0231000 - EAST LANSDOWNE BOROUGH",
    "PA0231300 - GLENOLDEN BOROUGH",
    "PA0231400 - HAVERFORD TOWNSHIP",
    "PA0231500 - LANSDOWNE BOROUGH",
    "PA0231700 - MARPLE TOWNSHIP",
    "PA0232000 - MARCUS HOOK BOROUGH",
    "PA0232400 - NORWOOD BOROUGH",
    "PA0232500 - PROSPECT PARK BOROUGH",
    "PA0232600 - RADNOR TOWNSHIP",
    "PA0233600 - UPPER CHICHESTER TOWNSHIP",
    "PA0233700 - UPPER DARBY TOWNSHIP",
    "PA0240300 - SAINT MARYS CITY",
    "PA0250200 - ERIE CITY",
    "PA0250400 - NORTH EAST BOROUGH",
    "PA0250500 - WESLEYVILLE BOROUGH",
    "PA0250600 - LAWRENCE PARK TOWNSHIP",
    "PA0250700 - UNION CITY BOROUGH",
    "PA0250900 - EDINBORO BOROUGH",
    "PA0251100 - GIRARD BOROUGH",
    "PA0260100 - BROWNSVILLE BOROUGH",
    "PA0260200 - CONNELLSVILLE CITY",
    "PA0260300 - UNIONTOWN CITY",
    "PA0264100 - GERMAN TOWNSHIP",
    "PA0280200 - GREENCASTLE BOROUGH",
    "PA0310100 - HUNTINGDON BOROUGH",
    "PA0320300 - INDIANA BOROUGH",
    "PA0350400 - SCRANTON CITY",
    "PA0351300 - THROOP BOROUGH",
    "PA0352300 - MOOSIC BOROUGH",
    "PA035399E - UNIVERSITY OF SCRANTON",
    "PA0360500 - LANCASTER CITY",
    "PA0361100 - NEW HOLLAND BOROUGH",
    "PA0363700 - EAST COCALICO TOWNSHIP",
    "PA0370200 - NEW CASTLE CITY",
    "PA0380200 - LEBANON CITY",
    "PA0390100 - ALLENTOWN CITY",
    "PA0390200 - CATASAUQUA BOROUGH",
    "PA0390400 - EMMAUS BOROUGH",
    "PA0390800 - WHITEHALL TOWNSHIP",
    "PA0390900 - COPLAY BOROUGH",
    "PA0391300 - SOUTH WHITEHALL TOWNSHIP",
    "PA0391400 - ALBURTIS BOROUGH",
    "PA0400100 - DALLAS BOROUGH",
    "PA0400300 - DURYEA BOROUGH",
    "PA0400700 - HAZLETON CITY",
    "PA0400800 - KINGSTON BOROUGH",
    "PA0401100 - PITTSTON CITY",
    "PA0401800 - FORTY FORT BOROUGH",
    "PA0401900 - LARKSVILLE BOROUGH",
    "PA0402300 - SWOYERSVILLE BOROUGH",
    "PA0402700 - WILKES-BARRE TOWNSHIP",
    "PA0403900 - WRIGHT TOWNSHIP",
    "PA0406500 - HUGHESTOWN BOROUGH",
    "PA0410300 - MUNCY BOROUGH",
    "PA0410400 - SOUTH WILLIAMSPORT BOROUGH",
    "PA0410500 - WILLIAMSPORT CITY",
    "PA0412800 - LYCOMING REGIONAL POLICE DEPARTMENT",
    "PA0430400 - HERMITAGE CITY",
    "PA0430700 - SHARPSVILLE BOROUGH",
    "PA0430800 - PYMATUNING TOWNSHIP",
    "PA0440700 - GRANVILLE TOWNSHIP",
    "PA0441200 - LEWISTOWN BOROUGH",
    "PA0450700 - POCONO TOWNSHIP",
    "PA0460100 - ABINGTON TOWNSHIP",
    "PA0460500 - CHELTENHAM TOWNSHIP",
    "PA0460900 - HATBORO BOROUGH",
    "PA0461100 - JENKINTOWN BOROUGH",
    "PA0461200 - LANSDALE BOROUGH",
    "PA0461300 - LOWER GWYNEDD TOWNSHIP",
    "PA0461500 - LOWER MORELAND TOWNSHIP",
    "PA0461600 - LOWER POTTSGROVE TOWNSHIP",
    "PA0461700 - LOWER PROVIDENCE TOWNSHIP",
    "PA0461800 - MONTGOMERY TOWNSHIP",
    "PA0461800 - MONTGOMERY TOWNSHIP",
    "PA0462200 - POTTSTOWN BOROUGH",
    "PA0462800 - UPPER GWYNEDD TOWNSHIP",
    "PA0462900 - UPPER MERION TOWNSHIP",
    "PA0464100 - HATFIELD TOWNSHIP",
    "PA0464600 - UPPER POTTSGROVE TOWNSHIP",
    "PA0464800 - WEST POTTSGROVE TOWNSHIP",
    "PA0480300 - BETHLEHEM CITY",
    "PA0480400 - EASTON CITY",
    "PA0480700 - NAZARETH BOROUGH",
    "PA0480800 - NORTHAMPTON BOROUGH",
    "PA0481200 - BETHLEHEM TOWNSHIP",
    "PA0481300 - BUSHKILL TOWNSHIP",
    "PA0481500 - FORKS TOWNSHIP",
    "PA0482200 - ROSETO BOROUGH",
    "PA0490100 - MILTON BOROUGH",
    "PA0490200 - MOUNT CARMEL BOROUGH",
    "PA0490500 - SUNBURY CITY",
    "PA0490600 - COAL TOWNSHIP",
    "PA0491400 - WATSONTOWN BOROUGH",
    "PA0491600 - POINT TOWNSHIP",
    "PA0491800 - MOUNT CARMEL TOWNSHIP",
    "PA0540200 - FRACKVILLE BOROUGH",
    "PA0540600 - MINERSVILLE BOROUGH",
    "PA0540800 - SAINT CLAIR BOROUGH",
    "PA0542000 - MAHANOY TOWNSHIP",
    "PA0543400 - RYAN TOWNSHIP",
    "PA0560500 - BOSWELL BOROUGH",
    "PA0590700 - WESTFIELD BOROUGH",
    "PA0600300 - MIFFLINBURG BOROUGH",
    "PA0600600 - NEW BERLIN BOROUGH",
    "PA0610100 - FRANKLIN CITY",
    "PA0610200 - OIL CITY",
    "PA0630500 - WASHINGTON CITY",
    "PA0630900 - MCDONALD BOROUGH",
    "PA0633300 - SOUTH STRABANE TOWNSHIP",
    "PA0650400 - JEANNETTE CITY",
    "PA0650500 - LATROBE CITY",
    "PA0650700 - MONESSEN CITY",
    "PA0650900 - NORTH HUNTINGDON TOWNSHIP",
    "PA0654500 - ROSTRAVER TOWNSHIP",
    "PA0670200 - YORK CITY",
    "PA0674700 - NEWBERRY TOWNSHIP",
    "PAPDE250X - LAKE ERIE COLLEGE OF OSTEOPATHIC MEDICINE",
    "PAPPD0000 - PITTSBURGH CITY",
]
print(dt.now())
print(len(agency_options))

2025-10-22 20:22:56.617090
219


In [2]:
########## SETUP FOR THIS RUN 
### Set download directory for exported report files: 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
here = os.path.abspath('') # the directory of this file 
### save directories by data type / geography:
monthly_file_dir = os.path.join(here, 'ReturnA','MonthlyData')
annual_file_dir = os.path.join(here, 'ReturnA', 'YearlyData')
agency_fiscal_year_data = os.path.join(here, 'ReturnA', 'ByAgency', 'FiscalYearData')
# set which one to use:
file_download_dir = agency_fiscal_year_data 
### when chrome is launched it changes the default download directory - does not affect user's chrome settings
chrome_options.add_experimental_option('prefs', {'download.default_directory': file_download_dir})

### set the actual parameters you want to use for this loop
report_url = report_links['annual_srs_summary_report']

report_type_selection = 'Return A'
report_by_selection = 'County'
report_by_selection = 'Agency'

### Time parameters
### for running individual months - unused right now
years = ['2025','2024','2023','2022'] # ['2023','2024','2025']
### build list of Jan - Dec parameter tuples for years in years list 
calendar_year_parameters = [(f'Jul - {int(year)-1}', f'Jun - {year}') for year in years]
###### ::: 2008 - 2024 captured already... 

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Dec']
# build parameter option format as list of month - year combinations 
month_by_month_parameters = [(f'{month} - {year}', f'{month} - {year}') for year in years for month in months]

### define which time pd type you want to use
time_pd_parameters = calendar_year_parameters
# time_pd_parameters = month_by_month_parameters

### subset counties if needed
counties = county_options
agencies = agency_options

### build a list of dictionaries to use as parameters for the main loop
### srs summary - combination of counties and time periods 
#### TODO: first parameter - county vs agency vs abstract - what to call it? keeping it county even when running by agency for now... #TECHDEBT
parameters = [ 
    # {'county':county, 'start': pd[0], 'end': pd[1]} for pd in time_pd_parameters for county in counties
    {'county':agency, 'start': pd[0], 'end': pd[1]} for pd in time_pd_parameters for agency in agencies
]

### Parameter override for re-running failures from a previous run 
### (county time period dictionaries are logged if they fail, reload from the json file if needed)
# import json 
# if os.path.exists('LastRunFailures.json'):
#     with open('LastRunFailures.json', 'r') as f:
#         prev_failure_parameters = json.loads(f.read())
#         # print(json.dumps(parameters, indent=2))
#     for p in prev_failure_parameters:
#         print(p)
#     # print(prev_failure_parameters)
#     ###check if file / prev_fail is empty; if empty use parameters above;  else retry failures
#     if len(prev_failure_parameters) > 0:
#         ## failures found, retry:
#         parameters = prev_failure_parameters
#     else:
#         pass # parameters already set
    
print(dt.now().isoformat())
print(len(parameters), parameters)


2025-10-22T20:22:56.638061
876 [{'county': 'PA0010100 - GETTYSBURG BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0010200 - LITTLESTOWN BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0010400 - MCSHERRYSTOWN BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0020200 - BELLEVUE BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0020500 - CARNEGIE BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0020900 - DUQUESNE CITY', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0020F00 - KENNEDY TOWNSHIP', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0021400 - MCKEES ROCKS BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0021I00 - LEET TOWNSHIP', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0022200 - ROSS TOWNSHIP', 'start': 'Jul - 2024', 'end': 'Jun - 2025'}, {'county': 'PA0022600 - WHITEHALL BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 20

In [3]:
### function definition - this gets executed once per loop for each parameter option
def export_iframe_report_to_excel(iframe_element):
    '''This can be re-used across multiple report types 
    Once the parameters have been selected and the report is generated on the page
    Find the iframe element in which the report is loaded
    pass that element here and it will wait for the report to fully load and trigger an excel export 
    '''
    ### switch context to iframe to find elements within it 
    driver.switch_to.frame(iframe_element.get_attribute('id'))
    if print_debug: print('waiting for report/export...', dt.now().isoformat())
    ### once iframe is loaded we can wait for elements within it
    # wait for report viewer to complete the report and show the export button 
    excel_export = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,'//table[@title="Export drop down menu"]')))
    # wait for the copyright state of PA text to load at the bottom of the report - indicator loading is done, before exporting

    end_of_report_text = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//span[text()="© STATE OF Pennsylvania"]')))
    if print_debug: print('end of report visibility', dt.now())
    ### EXPORT: this is the onclick event for the excel export in the drop down, just exeucte this directly (after all loading is done:): 
    driver.execute_script("$find('rvSiteMapping').exportReport('EXCELOPENXML');")

def get_report_data(parameter, print_debug=False):
    '''This is specific tot he SRSAnnual Summary report parameters
    '''
    # extract individual values from paremter dict
    county = parameter['county']
    start_pd = parameter['start']
    end_pd = parameter['end']
    # prep log record with start time 
    county_run = {
        'Start':start_pd,
        'County':county,
        'StartTime':dt.now().isoformat()
    }
    if print_debug: print('\n', start_pd, end_pd, county, dt.now(), end='...')
    
    # load page: 
    driver.get(report_url)

    if print_debug: print('getting elements...', dt.now().isoformat())
    ### find all of the relevant parameter objects on page (not all used, but at least validates they're all present as a test)
    start_month_year = driver.find_element(By.ID,'StartMonthYear') # date / text = set values from parameter list 
    end_month_year = driver.find_element(By.ID, 'EndMonthYear') # date / text = set values from parameter list 
    report_by = Select(driver.find_element(By.ID, 'ReportBy')) # option change "selected" value # SET BEFORE AGENCY
    chosen_choices = driver.find_element(By.CLASS_NAME, 'chosen-choices') # need to add one chosen choice (county) per loop 
    generate_report = driver.find_element(By.XPATH,'//*[@title="Generate Report"]') # button to generate report 
    report_type = Select(driver.find_element(By.ID, 'SRSSummaryReportType')) # report type selection box
    county_input = driver.find_element(By.CLASS_NAME, 'chosen-choices') # county input box 
    county_drop_down = driver.find_element(By.CLASS_NAME, 'chosen-results') # list of county options that appears from input box

    if print_debug: print('setting report by and report type...', dt.now().isoformat())
    ### set report by and report type once: 
    sleep(3) # make sure page is loaded BEFORE changing report type to county...
    report_type.select_by_visible_text(report_type_selection)
    report_by.select_by_visible_text(report_by_selection) 
    sleep(2) # let county options load after changing report by value

    if print_debug: print('set year month parameters...', dt.now().isoformat())
    ### set year month parameters: 
    driver.execute_script(f"document.getElementsByName('StartMonthYear')[0].value= '{start_pd}'")
    driver.execute_script(f"document.getElementsByName('EndMonthYear')[0].value= '{end_pd}'")

    if print_debug: print('County Click and select...')
    ### click county box to load dropdown of choices
    ### if there is a previous choice - need to clear it...
    try:
        existing_county_choice = driver.find_element(By.CLASS_NAME, 'search-choice-close')
        existing_county_choice.click()
    except:
        # no existing county selection
        pass

    ### click input to load drop down selection. 
    county_input.click()
    sleep(1)
    ### get the element of the county one you want and simulate click (assumes county option list has loaded - will timeout if not)
    if print_debug: print('first try county...', dt.now().isoformat())
    county_choice = WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH, f"//li[contains(text(), '{county}')]")))
    county_choice.click()

    if print_debug: print('clicking generate...', dt.now().isoformat())
    ### run report: 
    generate_report.click()

    if print_debug: print('waiting for iframe...', dt.now().isoformat())
    ### wait for the iframe to load:
    iframe_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'frmReport')))
    export_iframe_report_to_excel(iframe_element)

    end_time = dt.now().isoformat()
    if print_debug: print('exported and done', dt.now().isoformat())
    county_run['EndTime'] = end_time
    county_run['Status'] = 'Success'
    with open(log_file, 'a') as f:
        f.write(f',\n{json.dumps(county_run, indent=2)}')
    print('done.', end='')
print(dt.now().isoformat())

2025-10-22T20:22:56.656978


In [4]:
### begin html session 
run_log = []
driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(5)
print(dt.now())

2025-10-22 20:22:59.495593


In [ ]:
#### MAIN DRIVER - 
# enables a bunch of print statements in the process, turn on if troubleshooting a single run - don't print this much in a big loop
print_debug = False 

start_time = dt.now().isoformat()
print(start_time)
log_file = 'RunLog_Annual.json'
fails = []
for n, parameter in enumerate(parameters):
    print(f'\n{n+1}/{len(parameters)} : {parameter}', dt.now().isoformat())
    
    try:
        get_report_data(parameter)
    except Exception as e:
        print(n, e)
        print('trying again...')
        try:
            get_report_data(parameter)
        except Exception as e:
            print(e)
            fails.append((parameter))
print(len(fails), fails)
with open('LastRunFailures.json', 'w') as f:
    f.write(json.dumps(fails, indent=2))
    print('failures logged')


2025-10-22T20:23:06.590267

1/876 : {'county': 'PA0010100 - GETTYSBURG BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'} 2025-10-22T20:23:06.590401
0 name 'json' is not defined
trying again...
name 'json' is not defined

2/876 : {'county': 'PA0010200 - LITTLESTOWN BOROUGH', 'start': 'Jul - 2024', 'end': 'Jun - 2025'} 2025-10-22T20:25:12.906227
